<div align="right">
Massimo Nocentini
<br>
<br>September 12, 2016: search hints, table data representation
<br>September 7, 2016: module encapsulation, refactoring and filtering
<br>September 6, 2016: pretty printer
<br>September 5, 2016: big bang
</div>
<br>
<div align="center">
<b>Abstract</b><br>
This document collects examples of interaction with the <a href="http://oeis.org/">OEIS</a>.<br>
We aim to provide an <i>async</i> interface and a <i>pretty printer</i> for search results<br> to be rendered in a working notebook instead of switching between browser tabs.
</div>

# A _pretty printer_ for OEIS search results

Our idea to integrate OEIS search results in a working notebook lies on a existing attempt to talk with http://oeis.org, under review in [this Stack Overflow thread][stack:overflow]. Evaluate the following cell to work with most recent definitions according to version history:

[stack:overflow]:http://codereview.stackexchange.com/questions/120102/an-oeis-lookup-tool-in-python

In [1]:
from oeis import oeis_search, pretty_print

Instead, for *debug and develop* sessions evaluate the following one:

In [39]:
%run ../python-libs/oeis.py

## Search hints

OEIS provide a [page full of search hints][hints]. In order to gain performance and search capabilities already provided by the online encyclopedia, we show how to use those hints with our implementation.

The following content is kept from the *Advanced search syntax* section of the [reference page][hints]:
   - **Examples of searches**
   
        ```
        1,4,9,16,25,36,64
        5 8 13 233 39088169
        "fermat's little theorem"
        author:Guy keyword:nice
        keyword:nice keyword:more -keyword:base
        keyword:new -keyword:base
        id:A64413
        A64413
        id:A028284|id:A066948
        author:njas keyword:less|keyword:dumb
        ```
        
        - The search text is a list of space-separated search terms, as with popular web search engines.
        - The search terms can be single numbers, sequences of numbers, words or strings of words.
        - Sequences of numbers should be separated with commas.
        - Strings of words should be separated by spaces (NOT commas).
        - Strings of words may be enclosed in double quotes.
        - You can separate search terms with | (no spaces around the |) and it means sequences that match either term.
   
   - **Commas versus spaces** <br>
        Commas and spaces have different meanings. These two searches produce different results:
        
        ```
        1,4,9,16,25,36,49,64,81,100
        1 4 9 16 25 36 49 64 81 100
        ```

        The first matches sequences in which the numbers appear consecutively, in the order given.<br>
        The second matches sequences containing all the numbers, in any order. <br>
        Both searches will match the squares (A000290), but the second will also match other sequences,
        such as the composite numbers (A002808), which contain all those numbers but not necessarily consecutively or in the given order.<br>
        The same is true for word searches. Suppose you want to find sequences that cite Stanley's book Enumerative Combinatorics.<br>
        The search string Stanley, Combinatorics (with the comma) gets no hits, but searching for Stanley Combinatorics (without the comma) gets several hundred hits.
        
   - **Prefixes**<br>
    Simple search terms will be matched against every field of the entry, 
    though fields such as the name and the sequence data are given more weight than others.
    Prefixes restrict the search to particular lines, as in:

        ```
        keyword:nice author:LeBrun
        ```

    The prefixes mostly correspond to the lines in the display, as follows:

        ```
        id:                     ref:                    program:
        seq:                    link:                   xref:
        signed:                 formula:                keyword:
        name:                   example:                author:
        offset:                 maple:                  extension:
        comment:                mathematica:
        ```
        
        - `id:A001006` displays sequence A001006. A001006 displays sequence A001006 
            followed by all other sequences that mention it.
        - `seq:` matches the sequence data, possibly by ignoring signs.
        - `signed:` matches sequence data and requires that any signs match as well. 
            For example: `signed:1,-1,-1,0,-1,1,-1,0,0,1,-1,0,-1,1,1,0`
            will find the Moebius function mu(n), A008683.
        - Put a minus sign in front of a prefix to exclude those matches.
            Put a tilde (~) in front of a number to exclude it (only applies if the 
            numbers are separated by spaces). For example, a search for 2 4 ~8 16 returns 
            sequences such as 0, 1, 2, 4, 16, 65536 that do not contain 8.
        - You can say `prefix:` by itself to match sequences that have a line of that type. 
            For example `ref:` will match all sequences with reference lines.
        - Another example: you can say, e.g. `-seq:5` in your search to exclude all 
            sequences containing the number 5. So if you know your sequence consists only 
            of 1, 2, 3, and 4 (and does include all of them), you could search 
            `seq:1 seq:2 seq:3 seq:4 -seq:5 -seq:6 -seq:0`.

    Two other prefixes are concerned with subsequences:
        
        ```
        subseq:                 signedsubseq:
        ```

    You can say, for example, `subseq:377,8` to get those numbers in sequence data in that order.
   
   - **Wild cards**<br>
        - The wildcard \_ will match any number in a sequence, as in 1,2,_,4,5
        - The wildcard \__ will match any list of numbers in a sequence, as in 1,2,__,7,8
        - Search terms are highlighted in the results
        
   - **Punctuation**<br>
     is not indexed and is treated as identical to whitespace during matching;
     for example, searching for fermat's is the same as searching for "fermat s".
     
   - **Further examples**<br>
     To find sequences containing both terms 1759 and 1957: search for `seq:1759 seq:1957`.



[hints]:https://oeis.org/hints.html

## Use cases

### Search by sequence `id`entifier

In [2]:
searchable = oeis_search(id=45,)

In [3]:
searchable(comment=lambda i,c: "binomial" in c)

_Results for query: <a href='https://oeis.org/search?fmt=json&start=0&q=id%3AA000045'>https://oeis.org/search?fmt=json&start=0&q=id%3AA000045</a>_<br><hr><div align='center'><b><a href='http://oeis.org/A000045'>A000045</a></b>: <i>Fibonacci numbers: F(n) = F(n-1) + F(n-2) with F(0) = 0 and F(1) = 1.</i><br></div>

by _N. J. A. Sloane_, 1964

_Keywords_: `core,nonn,nice,easy,hear,changed`

_Data_:

$$
\begin{array}{c|cccccccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 & 15 & 16 & 17 & 18 & 19 \\
\hline
A000045(n) & 0 & 1 & 1 & 2 & 3 & 5 & 8 & 13 & 21 & 34 & 55 & 89 & 144 & 233 & 377 & 610 & 987 & 1597 & 2584 & 4181
\end{array}
$$


_Comments_:
   - Fib(n+2) = Sum_{k=0..n} binomial(floor((n+k)/2),k), row sums of <a href="http://oeis.org/A046854">A046854</a>. - _Paul Barry_, Mar 11 2003
   - The sequence F(n) is the binomial transformation of the alternating sequence (-1)^(n-1)*F(n), whereas the sequence F(n+1) is the binomial transformation of the alternating sequence (-1)^n*F(n-1). Both of these facts follow easily from the equalities a(n;1)=F(n+1) and b(n;1)=F(n) where a(n;d) and b(n;d) are so-called "delta-Fibonacci" numbers as defined in comments to <a href="http://oeis.org/A014445">A014445</a> (see also the papers of Witula et al.). - _Roman Witula_, Jul 24 2012
   - From _Tom Copeland_, Nov 02 2014: 
       - Let P(x) = x/(1+x) with comp. inverse Pinv(x) = x/(1-x) = -P[-x], and C(x)= [1-sqrt(1-4x)]/2, an o.g.f. for the shifted Catalan numbers <a href="http://oeis.org/A000108">A000108</a>, with inverse Cinv(x) = x * (1-x).
       - Fin(x) = P[C(x)] = C(x)/[1 + C(x)] is an o.g.f. for the Fine numbers, <a href="http://oeis.org/A000957">A000957</a> with inverse Fin^(-1)(x) = Cinv[Pinv(x)] = Cinv[-P(-x)].
       - Mot(x) = C[P(x)] = C[-Pinv(-x)] gives an o.g.f. for shifted <a href="http://oeis.org/A005043">A005043</a>, the Motzkin or Riordan numbers with comp. inverse Mot^(-1)(x) = Pinv[Cinv(x)] = (x - x^2) / (1 - x + x^2) (cf. <a href="http://oeis.org/A057078">A057078</a>).
       - BTC(x) = C[Pinv(x)] gives <a href="http://oeis.org/A007317">A007317</a>, a binomial transform of the Catalan numbers, with BTC^(-1)(x) = P[Cinv(x)].
       - Fib(x) = -Fin[Cinv(Cinv(-x))] = -P[Cinv(-x)] = x + 2 x^2 + 3 x^3 + 5 x^4 + ... = (x+x^2)/[1-x-x^2] is an o.g.f. for the shifted Fibonacci sequence <a href="http://oeis.org/A000045">A000045</a>, so the comp. inverse is Fib^(-1)(x) = -C[Pinv(-x)] = -BTC(-x) and Fib(x) = -BTC^(-1)(-x).
       - Generalizing to P(x,t) = x /(1 + t*x) and Pinv(x,t) = x /(1 - t*x) = -P(-x,t) gives other relations to lattice paths, such as the o.g.f. for <a href="http://oeis.org/A091867">A091867</a>, C[P[x,1-t]], and that for <a href="http://oeis.org/A104597">A104597</a>, Pinv[Cinv(x),t+1].

_Formulae_:
   - G.f.: x / (1 - x - x^2).
   - G.f.: Sum_{n>=0} x^n * Product_{k=1..n} (k + x)/(1 + k*x). - _Paul D. Hanna_, Oct 26 2013
   - F(n) = ((1+sqrt(5))^n-(1-sqrt(5))^n)/(2^n*sqrt(5)).
   - Alternatively, F(n) = ((1/2+sqrt(5)/2)^n-(1/2-sqrt(5)/2)^n)/sqrt(5).
   - F(n) = F(n-1) + F(n-2) = -(-1)^n F(-n).
   - F(n) = round(phi^n/sqrt(5)).
   - F(n+1) = Sum_{j=0..[n/2]} binomial(n-j, j).
   - This is a divisibility sequence; that is, if n divides m, then a(n) divides a(m). - _Michael Somos_, Apr 07 2012
   - E.g.f.: (2/sqrt(5))*exp(x/2)*sinh(sqrt(5)*x/2). - _Len Smiley_, Nov 30 2001
   - [0 1; 1 1]^n [0 1] = [F(n); F(n+1)]
   - x | F(n) ==> x | F(kn).
   - A sufficient condition for F(m) to be divisible by a prime p is (p - 1) divides m, if p == 1 or 4 (mod 5); (p + 1) divides m, if p == 2 or 3 (mod 5); or 5 divides m, if p = 5. (This is essentially Theorem 180 in Hardy and Wright.) - Fred W. Helenius (fredh(AT)ix.netcom.com), Jun 29 2001
   - a(n)=F(n) has the property: F(n)*F(m) + F(n+1)*F(m+1) = F(n+m+1). - _Miklos Kristof_, Nov 13 2003
   - _Kurmang. Aziz. Rashid_, Feb 21 2004, makes 4 conjectures and gives 3 theorems:
   - Conjecture 1: for n>=2 sqrt{F(2n+1)+F(2n+2)+F(2n+3)+F(2n+4)+2*(-1)^n}={F(2n+1)+2*(-1)^n}/F(n-1). [For a proof see Comments section.]
   - Conjecture 2: for n>=0, {F(n+2)* F(n+3)}-{F(n+1)* F(n+4)}+ (-1)^n = 0.
   - Conjecture 3: for n>=0, F(2n+1)^3 - F(2n+1)*[(2*A^2)-1] - [A + A^3]=0, where A = {F(2n+1)+sqrt{5*F(2n+1)^2 +4}}/2.
   - Conjecture 4: for x>=5, if x is a Fibonacci number >= 5 then g*x*[{x+sqrt{5*(x^2) +- 4}}/2]*[2x+{{x+sqrt{5*(x^2) +- 4}}/2}]*[2x+{{3x+3*sqrt {5*(x^2) +- 4}}/2}]^2+[2x+{{x+sqrt{5*(x^2) +- 4}}/2}] +- x*[2x+{{3x+3*sqrt{5*(x^2) +- 4}}/2}]^2 -x*[2x+{{x+sqrt{5*(x^2) +- 4}}/2}]*[x+{{x+sqrt{5*(x^2) +- 4}}/2}]* [2x+{{3x+3*sqrt{5*(x^2) +- 4}}/2}]^2= 0, where g = {1 + sqrt(5)/2}.
   - Theorem 1: for n>=0, {F(n+3)^ 2 - F(n+1)^ 2}/F(n+2)={F(n+3)+ F(n+1)}.
   - Theorem 2: for n>=0, F(n+10) = 11*F(n+5) + F(n).
   - Theorem 3: for n>=6, F(n) = 4*F(n-3) + F(n-6).
   - Conjecture 2 of Rashid is actually a special case of the general law F(n)*F(m) + F(n+1)*F(m+1) = F(n+m+1) (take n <- n+1 and m <- -(n+4) in this law). - Harmel Nestra (harmel.nestra(AT)ut.ee), Apr 22 2005
   - Conjecture 2 of Rashid Kurmang simplified: F(n)*F(n+3) = F(n+1)*F(n+2)-(-1)^n. Follows from d'Ocagne's identity: m=n+2. - _Alex Ratushnyak_, May 06 2012
   - Conjecture: for all c such that 2-Phi <= c < 2*(2-Phi) we have F(n) = floor(Phi*a(n-1)+c) for n > 2. - _Gerald McGarvey_, Jul 21 2004
   - |2*Fib(n) - 9*Fib(n+1)| = 4*<a href="http://oeis.org/A000032">A000032</a>(n) + <a href="http://oeis.org/A000032">A000032</a>(n+1). - _Creighton Dement_, Aug 13 2004
   - For x > Phi, Sum_{n=0..inf} F(n)/x^n = x/(x^2 - x - 1) - _Gerald McGarvey_, Oct 27 2004
   - F(n+1) = exponent of the n-th term in the series f(x, 1) determined by the equation f(x, y) = xy + f(xy, x). - _Jonathan Sondow_, Dec 19 2004
   - a(n-1) = Sum_{k=0..n} (-1)^k*binomial(n-ceil(k/2), floor(k/2)). - _Benoit Cloitre_, May 05 2005
   - F(n+1) = Sum_{k=0..n} binomial((n+k)/2, (n-k)/2)(1+(-1)^(n-k))/2. - _Paul Barry_, Aug 28 2005
   - Fibonacci(n) = Product(1 + 4[cos(j*Pi/n)]^2, j=1..ceil(n/2)-1). [Bicknell and Hoggatt, pp. 47-48.] - _Emeric Deutsch_, Oct 15 2006
   - F(n) = 2^-(n-1)*Sum_{k=0..floor((n-1)/2)} binomial(n,2*k+1)*5^k. - _Hieronymus Fischer_, Feb 07 2006
   - a(n) = (b(n+1)+b(n-1))/n where {b(n)} is the sequence <a href="http://oeis.org/A001629">A001629</a>. - _Sergio Falcon_, Nov 22 2006
   - F(n*m) = Sum_{k = 0..m} binomial(m,k)*F(n-1)^k*F(n)^(m-k)*F(m-k). The generating function of F(n*m) (n fixed, m = 0,1,2...) is G(x) = F(n)*x / ((1-F (n-1)*x)^2-F(n)*x*(1-F(n-1)*x)-( F(n)*x)^2). E.g., F(15) = 610 = F(5*3) = binomial(3,0)* F(4)^0*F(5)^3*F(3) + binomial(3,1)* F(4)^1*F(5)^2*F(2) + binomial(3,2)* F(4)^2*F(5)^1*F(1) + binomial(3,3)* F(4)^3*F(5)^0*F(0) = 1*1*125*2 + 3*3*25*1 + 3*9*5*1 + 1*27*1*0 = 250 + 225 + 135 + 0 = 610. - _Miklos Kristof_, Feb 12 2007
   - From _Miklos Kristof_, Mar 19 2007: 
       -   Let L(n) = <a href="http://oeis.org/A000032">A000032</a>(n) = Lucas numbers. Then:
       -   For a>=b and odd b, F(a+b)+F(a-b)=L(a)*F(b).
       -   For a>=b and even b, F(a+b)+F(a-b)=F(a)*L(b).
       -   For a>=b and odd b, F(a+b)-F(a-b)=F(a)*L(b).
       -   For a>=b and even b, F(a+b)-F(a-b)=L(a)*F(b).
       -   F(n+m)+(-1)^m*F(n-m)=F(n)*L(m);
       -   F(n+m)-(-1)^m*F(n-m)=L(n)*F(m);
       -   F(n+m+k)+(-1)^k*F(n+m-k)+(-1)^m*(F(n-m+k)+(-1)^k*F(n-m-k)) =F(n)*L(m)*L(k);
       -   F(n+m+k)-(-1)^k*F(n+m-k)+(-1)^m*(F(n-m+k)-(-1)^k*F(n-m-k)) =L(n)*L(m)*F(k);
       -   F(n+m+k)+(-1)^k*F(n+m-k)-(-1)^m*(F(n-m+k)+(-1)^k*F(n-m-k)) =L(n)*F(m)*L(k);
       -   F(n+m+k)-(-1)^k*F(n+m-k)-(-1)^m*(F(n-m+k)-(-1)^k*F(n-m-k)) =5*F(n)*F(m)*F(k). 
   - A corollary to Kristof 2007 is 2*F(a+b)=F(a)*L(b)+L(a)*F(b). - _Graeme McRae_, Apr 24 2014
   - For n>m, the sum of the 2m consecutive Fibonacci numbers F(n-m-1) thru F(n+m-2) is F(n)*L(m) if m is odd, and L(n)*F(m) if m is even (see the McRae link). - _Graeme McRae_, Apr 24 2014.
   - Fib(n) = b(n)+(p-1)*Sum_{1<k<n} floor(b(k)/p)*Fib(n-k+1) where b(k) is the digital sum analog of the Fibonacci recurrence, defined by b(k)=ds_p(b(k-1))+ds_p(b(k-2)), b(0)=0, b(1)=1, ds_p=digital sum base p. Example for base p=10: Fib(n)=<a href="http://oeis.org/A010077">A010077</a>(n)+9*Sum_{1<k<n} <a href="http://oeis.org/A059995">A059995</a>(<a href="http://oeis.org/A010077">A010077</a>(k))*Fib(n-k+1). - _Hieronymus Fischer_, Jul 01 2007
   - Fib(n) = b(n)+p*Sum_{1<k<n} floor(b(k)/p)*Fib(n-k+1) where b(k) is the digital product analog of the Fibonacci recurrence, defined by b(k)=dp_p(b(k-1))+dp_p(b(k-2)), b(0)=0, b(1)=1, dp_p=digital product base p. Example for base p=10: Fib(n)=<a href="http://oeis.org/A074867">A074867</a>(n)+10*Sum_{1<k<n} <a href="http://oeis.org/A059995">A059995</a>(<a href="http://oeis.org/A074867">A074867</a>(k))*Fib(n-k+1). - _Hieronymus Fischer_, Jul 01 2007
   - a(n) = denominator of continued fraction [1,1,1,...] (with n ones); e.g., 2/3 = continued fraction [1,1,1]; where barover[1] = [1,1,1...] = 0.6180339.... - _Gary W. Adamson_, Nov 29 2007
   - F(n + 3) = 2F(n + 2) - F(n), F(n + 4) = 3F(n + 2) - F(n), F(n + 8) = 7F(n + 4) - F(n), F(n + 12) = 18F(n + 6) - F(n). - _Paul Curtz_, Feb 01 2008
   - 1 = 1/(1*2) + 1/(1*3) + 1/(2*5) + 1/(3*8) + 1/(5*13) + ... = 1/2 + 1/3 + 1/10 + 1/24 + 1/65 + 1/168 + ...; where <a href="http://oeis.org/A059929">A059929</a> = (0, 2, 3, 10, 24, 65, 168,...). - _Gary W. Adamson_, Mar 16 2008
   - a(2^n) = prod{i=0}^{n-2}B(i) where B(i) is <a href="http://oeis.org/A001566">A001566</a>. Example 3*7*47 = Fib(16). - _Kenneth J Ramsey_, Apr 23 2008
   - F(n) = (1/(n-1)!) * (n^(n-1) - (C(n-2,0) + 4*C(n-2,1) + 3*C(n-2,2))*n^(n-2) + (10*C(n-3,0) + 49*C(n-3,1) + 95*C(n-3,2) + 83*C(n-3,3) + 27*C(n-3,4))*n^(n-3) - (90*C(n-4,0) + 740*C(n-4,1) + 2415*C(n-4,2) + 4110*C(n-4,3) + 3890*C(n-4,4) + 1950*C(n-4,5) + 405*C(n-4,6))*n^(n-4) + ... ). - _André F. Labossière_, Nov 24 2004
   - a(n+1) = Sum_{k, 0<=k<=n} <a href="http://oeis.org/A109466">A109466</a>(n,k)*(-1)^(n-k). -_Philippe Deléham_, Oct 26 2008
   - a(n) = Sum_{l_1=0..n+1} Sum_{l_2=0..n}...Sum_{l_i=0..n-i}... Sum_{l_n=0..1} delta(l_1,l_2,...,l_i,...,l_n), where delta(l_1,l_2,...,l_i,...,l_n) = 0 if any l_i + l_(i+1) >= 2 for i=1..n-1 and delta(l_1,l_2,...,l_i,...,l_n) = 1 otherwise. - _Thomas Wieder_, Feb 25 2009
   - a(n+1) = 2^n sqrt(Product_{k=1..n} cos(k Pi/(n+1))^2+1/4)) (Kasteleyn's formula specialized). - Sarah-Marie Belcastro (smbelcas(AT)toroidalsnark.net), Jul 04 2009
   - a(n+1) = Sum_{k=floor[n/2] mod 5} C(n,k) - Sum_{k=floor[(n+5)/2] mod 5} C(n,k) = <a href="http://oeis.org/A173125">A173125</a>(n) - <a href="http://oeis.org/A173126">A173126</a>(n) = |<a href="http://oeis.org/A054877">A054877</a>(n)-<a href="http://oeis.org/A052964">A052964</a>(n-1)|. - _Henry Bottomley_, Feb 10 2010
   - If p[i]=modp(i,2) and if A is Hessenberg matrix of order n defined by: A[i,j]=p[j-i+1], (i<=j), A[i,j]=-1, (i=j+1), and A[i,j]=0 otherwise. Then, for n>=1, a(n)=det A. - _Milan Janjic_, May 02 2010
   - Limit(F(k+n)/F(k), k = infinity) = (L(n) + F(n)*sqrt(5))/2 with the Lucas numbers L(n)= <a href="http://oeis.org/A000032">A000032</a>(n). - _Johannes W. Meijer_, May 27 2010
   - For n>=1, F(n)=round(log_2(2^{phi*F(n-1)} + 2^{phi*F(n-2)})), where phi is the golden ratio. - _Vladimir Shevelev_, Jun 24 2010, Jun 27 2010
   - For n>=1, a(n+1)=ceil(phi*a(n)), if n is even and a(n+1)=floor(phi*a(n)), if n is odd (phi = golden ratio). - _Vladimir Shevelev_, Jul 01 2010
   - a(n) = 2*a(n-2) + a(n-3), n>2. - _Gary Detlefs_, Sep 08 2010
   - a(2^n) = Prod_{i=0..n-1} <a href="http://oeis.org/A000032">A000032</a>(2^i). - _Vladimir Shevelev_, Nov 28 2010
   - a(n)^2 - a(n-1)^2 = a(n+1)*a(n-2), see <a href="http://oeis.org/A121646">A121646</a>.
   - a(n) = sqrt((-1)^k*(a(n+k)^2 - a(k)*a(2n+k))), for any k. - _Gary Detlefs_, Dec 03 2010
   - F(2*n) = F(n+2)^2 - F(n+1)^2 - 2*F(n)^2. - _Richard R. Forberg_, Jun 04 2011
   - (-1)^(n+1) = F(n)^2 + F(n)*F(1+n) - F(1+n)^2.
   -   F(n) = -F(n+2)(-2 + (F(n+1))^4 + 2*(F(n+1)^3*F(n+2)) - (F(n+1)*F(n+2))^2 2*F(n+1)(F(n+2))^3 + (F(n+2))^4)- F(n+1). - _Artur Jasinski_, Nov 17 2011
   - F(n) = 1 + Sum_{x=1..n-2} F(x). - _Joseph P. Shoulak_, Feb 05 2012
   - F(n) = 4*F(n-2) - 2*F(n-3) - F(n-6). - _Gary Detlefs_, Apr 01 2012
   - F(n) = round(phi^(n+1)/(phi+2)). - _Thomas Ordowski_, Apr 20 2012
   - From _Sergei N. Gladkovskii_, Jun 03 2012: 
       - G.f. A(x) = x/(1-x-x^2) = G(0)/sqrt(5) where G(k)= 1 -((-1)^k)*2^k/(a^k - b*x*a^k*2^k/(b*x*2^k - 2*((-1)^k)*c^k/G(k+1))) and a=3+sqrt(5), b=1+sqrt(5), c=3-sqrt(5); (continued fraction, 3rd kind, 3-step).
       - Let E(x) be the e.g.f., i.e.,
       - E(x) = 1*x + 1/2*x^2 + 1/3*x^3 + 1/8*x^4 + 1/24*x^5 + 1/90*x^6 + 13/5040*x^7 + ...; then
       - E(x) = G(0)/sqrt(5); G(k)= 1 -((-1)^k)*2^k/(a^k - b*x*a^k*2^k/(b*x*2^k - 2*((-1)^k)*(k+1)*c^k/G(k+1))), where a=3+sqrt(5), b=1+sqrt(5), c=3-sqrt(5); (continued fraction, 3rd kind, 3-step).
   - From _Hieronymus Fischer_, Nov 30 2012: 
       - Fib(n) = 1 + Sum_{j_1=1..n-2} 1 + Sum_{j_1=1..n-2} Sum_{j_2=1..j_1-2} 1 + Sum_{j_1=1..n-2} Sum_{j_2=1..j_1-2} Sum_{j_3=1..j_2-2} 1 + ... + Sum_{j_1=1..n-2} Sum_{j_2=1..j_1-2} Sum_{j_3=1..j_2-2} ... Sum_{j_k=1..j_(k-1)-2} 1, where k = floor((n-1)/2).
       - Example: Fib(6) = 1 + Sum_{j=1..4} 1 + Sum_{j=1..4} Sum_{k=1..(j-2)} 1 + 0 = 1 + (1 + 1 + 1 + 1) + (1 + (1 + 1)) = 8.
       - Fib(n) = Sum_{j=0..k} S(j+1,n-2j), where k = floor((n-1)/2) and the S(j,n) are the n-th j-simplex sums: S(1,n) = 1 is the 1-simplex sum, S(2,n) = Sum_{k=1..n} S(1,k) = 1+1+...+1 = n is the 2-simplex sum, S(3,n) = Sum_{k=1..n} S(2,k) = 1+2+3+...+n is the 3-simplex sum (= triangular numbers = <a href="http://oeis.org/A000217">A000217</a>), S(4,n) = Sum_{k=1..n} S(3,k) = 1+3+6+...+n(n+1)/2 is the 4-simplex sum (= tetrahedral numbers = <a href="http://oeis.org/A000292">A000292</a>) and so on.
       - Since S(j,n) = binomial(n-2+j,j-1), the formula above equals the well-known binomial formula, essentially. 
   - G.f. A(x) = x / (1 - x / (1 - x / (1 + x))). - _Michael Somos_, Jan 04 2013
   - Sum_{n>=1} (-1)^(n-1)/(a(n)*a(n+1)) = 1/phi (phi=golden ratio). - _Vladimir Shevelev_, Feb 22 2013
   - From _Vladimir Shevelev_, Feb 24 2013: 
       - (1) Expression a(n+1) via a(n): a(n+1) = (a(n) + sqrt(5*(a(n))^2 + 4*(-1)^n))/2;
       - (2) Sum_{k=1...n} (-1)^(k-1)/(a(k)*a(k+1)) = a(n)/a(n+1);
       - (3) a(n)/a(n+1) = 1/phi + r(n), where |r(n)| < 1/(a(n+1)*a(n+2)). 
   - F(n+1) = F(n)/2 + sqrt((-1)^n + 5*F(n)^2/4), n>=0. F(n+1) = U_n(i/2)/i^n, (U:= Chebyshef 2nd kind). - _Bill Gosper_, Mar 04 2013
   - G.f.: -Q(0) where Q(k) = 1 - (1+x)/(1 - x/(x - 1/Q(k+1) )); (continued fraction). - _Sergei N. Gladkovskii_, Mar 06 2013
   - G.f.: x-1-1/x + 1/x/Q(0), where Q(k) = 1 - (k+1)*x/(1 - x/(x - (k+1)/Q(k+1))); (continued fraction). - _Sergei N. Gladkovskii_, Apr 23 2013
   - G.f.: x*G(0), where G(k)= 1 + x*(1+x)/(1 - x*(1+x)/(x*(1+x) + 1/G(k+1) )); (continued fraction). - _Sergei N. Gladkovskii_, Jul 08 2013
   - G.f.: x^2 - 1 + 2*x^2/(W(0)-2), where W(k) = 1 + 1/(1 - x*(k + x)/( x*(k+1 + x) + 1/W(k+1) )); (continued fraction). - _Sergei N. Gladkovskii_, Aug 28 2013
   - G.f.: Q(0) -1, where Q(k) = 1 + x^2 + (k+2)*x -x*(k+1 + x)/Q(k+1); (continued fraction). - _Sergei N. Gladkovskii_, Oct 06 2013
   - Let b(n) = b(n-1) + b(n-2), with b(0) = 0, b(1) = phi. Then, for n>=2, F(n)= floor(b(n-1)) if n is even, F(n) = ceil(b(n-1)), if n is odd, with convergence. - _Richard R. Forberg_, Jan 19 2014
   - a(n) = Sum_{t1*g(1)+t2*g(2)+...+tn*g(n)=n} multinomial(t1+t2 +...+tn,t1,t2,...,tn), where g(k)=2*k-1. - _Mircea Merca_, Feb 27 2014
   - F(n) = round(sqrt(F(n-1)^2 + F(n)^2 + F(n+1)^2)/2), for n > 0. This rule appears to apply to any sequence of the form a(n) = a(n-1) + a(n-2), for any two values of a(0) and a(1), if n is sufficiently large. - _Richard R. Forberg_, Jul 27 2014
   - F(n) = round(2/(1/F(n) + 1/F(n+1) + 1/F(n+2)), for n > 0. This rule also appears to apply to any sequence of the form a(n) = a(n-1) + a(n-2), for any two values of a(0) and a(1), if n is sufficiently large. - _Richard R. Forberg_, Aug 03 2014
   - F(n) = round(1/(Sum_{j>=n+2} 1/F(j))). - _Richard R. Forberg_, Aug 14 2014
   - a(n) = hypergeometric([-n/2+1/2, -n/2+1], [-n+1], -4) for n>=2. - _Peter Luschny_, Sep 19 2014
   - F(n) = (L(n+1)^2 - L(n-1)^2)/(5*L(n)), where L(n) is <a href="http://oeis.org/A000032">A000032</a>(n), with a similar inverse relationship. - _Richard R. Forberg_, Nov 17 2014
   - Consider the graph G[1-vertex;1-loop,2-loop] in comment above. Construct the power matrix array T(n,j)=[A^*j]*[S^*(j-1)] where A=(1,1,0,...) and S=(0,1,0,...)(<a href="http://oeis.org/A063524">A063524</a>). [* is convolution operation] Define S^*0=I with I=(1,0,...). Then T(n,j) counts n-walks containing (j) loops and a(n-1) = Sum_{j=1...n} T(n,j). - _David Neil McGrath_, Nov 21 2014
   - Define F(-n) to be F(n) for n odd and -F(n) for n even. Then for all n and k, F(n) = F(k)*F(n-k+3) - F(k-1)*F(n-k+2) - F(k-2)*F(n-k) + (-1)^k*F(n-2k+2). - _Charlie Marion_, Dec 04 2014
   - F(n+k)^2 - L(k)*F(n)*F(n+k) + (-1)^k*F(n)^2 = (-1)^n*F(k)^2, if L(k) = <a href="http://oeis.org/A000032">A000032</a>(k). - _Alexander Samokrutov_, Jul 20 2015
   - F(2*n) = F(n+1)^2 - F(n-1)^2, similar to Koshy (D) and Forberg 2011, but different. - _Hermann Stamm-Wilbrandt_, Aug 12 2015
   - F(n+1) = ceiling( (1/phi)*Sum_{k=0..n} F(k) ). - _Tom Edgar_, Sep 10 2015
   - a(n) = (L(n-3) + L(n+3))/10 where L(n)=<a href="http://oeis.org/A000032">A000032</a>(n). - _J. M. Bergot_, Nov 25 2015
   - From _Bob Selcoe_, Mar 27 2016 :
       - F(n) = (F(2n+k+1) - F(n+1)*F(n+k+1))/F(n+k), k>=0.
       - Thus when k=0: F(n) = sqrt(F(2n+1) - F(n+1)^2).
       - F(n) = cbrt(F(3n) - F(n+1)^3 + F(n-1)^3).
       - F(n+2k) = binomial transform of any subsequence starting with F(n). Example F(6)=8: 1*8 = F(6)=8; 1*8 + 1*13 = F(8)=21; 1*8 + 2*13 + 1*21 = F(10)=55; 1*8 + 3*13 + 3*21 + 1*34 = F(12)=144, etc. This formula applies to Fibonacci-type sequences with any two seed values for a(0) and a(1) (e.g., Lucas sequence <a href="http://oeis.org/A000032">A000032</a>: a(0)=2, a(1)=1).
   - F(n) = L(k)*F(n-k) + (-1)^(k+1)*F(n-2k) for all k>=0, where L(k) = <a href="http://oeis.org/A000032">A000032</a>(k). - _Anton Zakharov_, Aug 02 2016
   - From _Ilya Gutkovskiy_, Aug 03 2016: 
       - a(n) = F_n(1), where F_n(x) are the Fibonacci polynomials.
       - Inverse binomial transform of <a href="http://oeis.org/A001906">A001906</a>.
       - Number of zeros in substitution system {0 -> 11, 1 -> 1010} at step n from initial string "1" (1 -> 1010 -> 101011101011 -> ...) multiplied by 1/<a href="http://oeis.org/A000079">A000079</a>(n). 

_Cross references_:
   - Cf. <a href="http://oeis.org/A039834">A039834</a> (signed Fibonacci numbers), <a href="http://oeis.org/A001690">A001690</a> (complement), <a href="http://oeis.org/A000213">A000213</a>, <a href="http://oeis.org/A000288">A000288</a>, <a href="http://oeis.org/A000322">A000322</a>, <a href="http://oeis.org/A000383">A000383</a>, <a href="http://oeis.org/A060455">A060455</a>, <a href="http://oeis.org/A030186">A030186</a>, <a href="http://oeis.org/A020695">A020695</a>, <a href="http://oeis.org/A020701">A020701</a>, <a href="http://oeis.org/A071679">A071679</a>, <a href="http://oeis.org/A099731">A099731</a>, <a href="http://oeis.org/A100492">A100492</a>, <a href="http://oeis.org/A094216">A094216</a>, <a href="http://oeis.org/A094638">A094638</a>, <a href="http://oeis.org/A000108">A000108</a>, <a href="http://oeis.org/A101399">A101399</a>, <a href="http://oeis.org/A101400">A101400</a>, <a href="http://oeis.org/A001611">A001611</a>, <a href="http://oeis.org/A000071">A000071</a>, <a href="http://oeis.org/A157725">A157725</a>, <a href="http://oeis.org/A001911">A001911</a>, <a href="http://oeis.org/A157726">A157726</a>, <a href="http://oeis.org/A006327">A006327</a>, <a href="http://oeis.org/A157727">A157727</a>, <a href="http://oeis.org/A157728">A157728</a>, <a href="http://oeis.org/A157729">A157729</a>, <a href="http://oeis.org/A167616">A167616</a>, <a href="http://oeis.org/A059929">A059929</a>, <a href="http://oeis.org/A144152">A144152</a>, <a href="http://oeis.org/A152063">A152063</a>, <a href="http://oeis.org/A114690">A114690</a>, <a href="http://oeis.org/A003893">A003893</a>, <a href="http://oeis.org/A000032">A000032</a>, <a href="http://oeis.org/A060441">A060441</a>, <a href="http://oeis.org/A000930">A000930</a>, <a href="http://oeis.org/A003269">A003269</a>, <a href="http://oeis.org/A000957">A000957</a>, <a href="http://oeis.org/A057078">A057078</a>, <a href="http://oeis.org/A007317">A007317</a>, <a href="http://oeis.org/A091867">A091867</a>, <a href="http://oeis.org/A104597">A104597</a>, <a href="http://oeis.org/A249548">A249548</a>, <a href="http://oeis.org/A262342">A262342</a>.
   - First row of arrays <a href="http://oeis.org/A103323">A103323</a>, <a href="http://oeis.org/A234357">A234357</a>. Second row of arrays <a href="http://oeis.org/A099390">A099390</a>, <a href="http://oeis.org/A048887">A048887</a>, and <a href="http://oeis.org/A092921">A092921</a> (k-generalized Fibonacci numbers).
   - a(n) = <a href="http://oeis.org/A094718">A094718</a>(4, n). a(n) = <a href="http://oeis.org/A101220">A101220</a>(0, j, n).
   - a(n) = <a href="http://oeis.org/A090888">A090888</a>(0, n+1) = <a href="http://oeis.org/A118654">A118654</a>(0, n+1) = <a href="http://oeis.org/A118654">A118654</a>(1, n-1) = <a href="http://oeis.org/A109754">A109754</a>(0, n) = <a href="http://oeis.org/A109754">A109754</a>(1, n-1), for n > 0.
   - Fibonacci-Pascal triangles: <a href="http://oeis.org/A027926">A027926</a>, <a href="http://oeis.org/A036355">A036355</a>, <a href="http://oeis.org/A037027">A037027</a>, <a href="http://oeis.org/A074829">A074829</a>, <a href="http://oeis.org/A105809">A105809</a>, <a href="http://oeis.org/A109906">A109906</a>, <a href="http://oeis.org/A111006">A111006</a>, <a href="http://oeis.org/A114197">A114197</a>, <a href="http://oeis.org/A162741">A162741</a>, <a href="http://oeis.org/A228074">A228074</a>.
   - Boustrophedon transforms: <a href="http://oeis.org/A000738">A000738</a>, <a href="http://oeis.org/A000744">A000744</a>.
   - Powers: <a href="http://oeis.org/A103323">A103323</a>, <a href="http://oeis.org/A105317">A105317</a>, <a href="http://oeis.org/A254719">A254719</a>.
   - Numbers of prime factors: <a href="http://oeis.org/A022307">A022307</a> and <a href="http://oeis.org/A038575">A038575</a>.

_Links_:
   - N. J. A. Sloane, <a href="/<a href="http://oeis.org/A000045">A000045</a>/b000045.txt">The first 2000 Fibonacci numbers: Table of n, F(n) for n = 0..2000</a>
   - Amazing Mathematical Object Factory, <a href="http://www.schoolnet.ca/vp-pv/amof/e_fiboI.htm">Information on the Fibonacci sequences</a>
   - P. G. Anderson, <a href="http://www.cs.rit.edu/~pga/Fibo/fact_sheet.html">Fibonacci Facts</a>
   - Joerg Arndt, <a href="http://www.jjj.de/fxt/#fxtbook">Matters Computational (The Fxtbook)</a>
   - J.-L. Baril, <a href="http://www.combinatorics.org/Volume_18/PDF/v18i1p178.pdf">Classical sequences revisited with permutations avoiding dotted pattern</a>, Electronic Journal of Combinatorics, 18 (2011), #P178.
   - Paul Barry, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL8/Barry/barry84.html">A Catalan Transform and Related Transformations on Integer Sequences</a>, Journal of Integer Sequences, Vol. 8 (2005), Article 05.4.5.
   - Paul Barry, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL9/Barry/barry91.html">On Integer-Sequence-Based Constructions of Generalized Pascal Triangles</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.2.4.
   - A. T. Benjamin, A. K. Eustis, M. A. Shattuck, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL12/Shattuck/shattuck20.html">Compression Theorems for Periodic Tilings and Consequences</a>, JIS 12 (2009) 09.6.3
   - E. R. Berlekamp, <a href="/<a href="http://oeis.org/A257113">A257113</a>/a257113.pdf">A contribution to mathematical psychometrics</a>, Unpublished Bell Labs Memorandum, Feb 08 1968 [Annotated scanned copy]
   - Kh. Bibak, M. H. Shirdareh Haghighi, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL12/Bibak/bibak4.html">Some Trigonometric Identities Involving Fibonacci and Lucas Numbers</a>, JIS 12 (2009) 09.8.4
   - Marjorie Bicknell and Verner E Hoggatt Jr, <a href="http://www.fq.math.ca/Books/Primer/bicknell6.pdf">To Prove: F(n) Divides F(nk)</a>, A Primer for the Fibonacci Numbers: IX (1973)
   - J. Bodeen, S. Butler, T. Kim, X. Sun, S. Wang, <a href="http://www.combinatorics.org/ojs/index.php/eljc/article/view/v21i1p7">Tiling a strip with triangles</a>, El. J. Combinat. 21 (1) (2014) P1.7
   - H. Bottomley and N. J. A. Sloane, <a href="/<a href="http://oeis.org/A000045">A000045</a>/a000045.html">Illustration of initial terms: the Fibonacci tree</a>
   - S. Brlek, E. Duchi, E. Pergola and S. Rinaldi, <a href="http://dx.doi.org/10.1016/j.disc.2004.07.019">On the equivalence problem for succession rules</a>, Discr. Math., 298 (2005), 142-154.
   - N. D. Cahill, J. R. D'Errico, J. P. Spence, <a href="http://www.fq.math.ca/41-1.html">Complex factorizations of the Fibonacci and Lucas numbers</a>, Fib. Quart. 41 (2003) 13.
   - N. D. Cahill and D. A. Narayan, <a href="http://www.fq.math.ca/Papers1/42-3/quartcahill03_2004.pdf">Fibonacci and Lucas Numbers as Tridiagonal Matrix Determinants</a>, Fibonacci Quarterly, 42(3):216-221, 2004.
   - C. K. Caldwell, The Prime Glossary, <a href="http://primes.utm.edu/glossary/page.php/FibonacciNumber.html">Fibonacci number</a>
   - P. J. Cameron, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL3/groups.html">Sequences realized by oligomorphic permutation groups</a>, J. Integ. Seqs. Vol. 3 (2000), #00.1.5.
   - Hongwei Chen, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL9/Chen/chen78.html">Evaluations of Some Variant Euler Sums</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.2.3.
   - J. H. Conway and N. J. A. Sloane, <a href="/<a href="http://oeis.org/A019586">A019586</a>/a019586.pdf">Notes on the Para-Fibonacci and related sequences</a>
   - J. H. Conway, Allan Wechsler, Marc LeBrun, Dan Hoey, N. J. A. Sloane, <a href="/<a href="http://oeis.org/A269725">A269725</a>/a269725.txt">On Kimberling Sums and Para-Fibonacci Sequences</a>, Correspondence and Postings to Math-Fun Mailing List, Nov 1996 to Jan 1997
   - E. S. Croot, <a href="http://www.math.gatech.edu/~ecroot/recurrence_notes2.pdf">Notes on Linear Recurrence Sequences</a>
   - Paul Cubre, <a href="http://wakespace.lib.wfu.edu/bitstream/handle/10339/37313/Cubre_wfu_0248M_10299.pdf">The Z-densities of the Fibonacci sequence</a>, M. A. Thesis, Wake Forest University, May 2012;
   - R. M. Dickau, <a href="http://mathforum.org/advanced/robertd/fibboard.html">Fibonacci numbers</a>
   - R. Doroslovacki, <a href="http://www.emis.de/journals/MV/9434/mv943407.ps">Binary sequences without 011...110 (k-1 1's) for fixed k</a>, Mat. Vesnik 46 (1994), no. 3-4, 93-98.
   - A.-S. Elsenhans and J. Jahnel, <a href="http://www.uni-math.gwdg.de/tschinkel/gauss/Fibon.pdf">The Fibonacci sequence modulo p^2 - an investigation by computer for p < 10^14</a>.
   - Nathaniel D. Emerson, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL9/Emerson/emerson6.html">A Family of Meta-Fibonacci Sequences Defined by Variable-Order Recursions</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.1.8.
   - Reinhardt Euler, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL8/Euler/euler1.html">The Fibonacci Number of a Grid Graph and a New Class of Integer Sequences</a>, Journal of Integer Sequences, Vol. 8 (2005), Article 05.2.6.
   - G. Everest, A. J. van der Poorten, Y. Puri and T. Ward, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL5/Ward/ward2.html">Integer Sequences and Periodic Points</a>, Journal of Integer Sequences, Vol. 5 (2002), Article 02.2.3
   - G. Everest, A. van der Poorten, I. Shparlinski and T. Ward, Recurrence Sequences, Amer. Math. Soc., 2003; see esp. p. 255. <a href="http://www.ams.org/mathscinet-getitem?mr=1990179">MR1990179</a>
   - Philipp Fahr and Claus Michael Ringel, <a href="http://www.mathematik.uni-bielefeld.de/~ringel/opus/fr-zwei.pdf">Categorification of the Fibonacci Numbers Using Representations of Quivers</a>
   - John Farrier, <a href="http://uploads.neatorama.com/wp-content/uploads/2010/09/1nT6a-500x296.jpg">Fibonacci Pigeons</a> [From Sarah Spolaor, Sep 30 2010]
   - Helaman and Claire Ferguson, <a href="http://www.ams.org/notices/201007/rtx100700840p.pdf">Celebrating Mathematics in Stone and Bronze</a>, Notices of the American Mathematical Society, 57 (2010), 840-850. See page 844
   - Emmanuel Ferrand, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL10/Ferrand/ferrand8.html">Deformations of the Taylor Formula</a>, Journal of Integer Sequences, Vol. 10 (2007), Article 07.1.7.
   - D. Foata and G.-N. Han, <a href="http://www-irma.u-strasbg.fr/~foata/paper/pub71.html">Nombres de Fibonacci et polynomes orthogonaux</a>,
   - I. Galkin, <a href="http://ulcar.uml.edu/~iag/CS/Fibonacci.html">"Fibonacci Numbers Spelled Out"</a>
   - Dale Gerdemann, <a href="http://www.youtube.com/watch?v=YpTSfkqS7zg">Video of Fibonacci tree</a>
   - Dale Gerdemann, <a href="http://bit.ly/lw7pP2">Video of Fibonacci tree(s)</a> (another version)
   - Dale Gerdemann, <a href="http://www.youtube.com/watch?v=AvRdzZMcUrI">Golden Ratio Base Contains Zeckendorf and Negative Indexed Bunder Forms</a>
   - C. J. Glasby, S. P. Glasby, F. Pleijel, <a href="http://dx.doi.org/10.1098/rspb.2008.0418">Worms by number</a>, Proc. Roy. Soc. B, Proc. Biol. Sci. 275 (1647) (2008) 2071-2076.
   - Hank Green, <a href="https://www.youtube.com/watch?v=wTlw7fNcO-0">The Fibonacci Sequence: Nature's Code</a> (2012).
   - M. Griffiths, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL14/Griffiths/griffiths16.html">A Restricted Random Walk defined via a Fibonacci Process</a>, Journal of Integer Sequences, Vol. 14 (2011), #11.5.4.
   - Nancy S. S. Gu, Nelson Y. Li, and Toufik Mansour, <a href="http://dx.doi.org/10.1016/j.disc.2007.04.007">2-Binary trees: bijections and related issues</a>, Discr. Math., 308 (2008), 1209-1221.
   - Guo-Niu Han, <a href="/<a href="http://oeis.org/A196265">A196265</a>/a196265.pdf">Enumeration of Standard Puzzles</a> [Cached copy]
   - S. Happersett, <a href="http://dx.doi.org/10.1080/17513470701210626">Mathematical meditations</a>
   - Brady Haran and Colm Mulcahy, <a href="https://www.youtube.com/watch?v=4izjrtR8Ozg">Little Fibs - Numberphile</a> (2016)
   - A. M. Hinz, S. Klavžar, U. Milutinović, C. Petr, <a href="http://dx.doi.org/10.1007/978-3-0348-0237-6">The Tower of Hanoi - Myths and Maths</a>, Birkhäuser 2013. See page 12. <a href="http://tohbook.info">Book's website</a>
   - V. E. Hoggatt and C. T. Long, <a href="http://www.fq.math.ca/Scanned/12-2/hoggatt1.pdf">Divisibility Properties of Generalized Fibonacci Polynomials</a>, Fibonacci Quarterly, 12:113-120, 1974.
   - C. W. Huegy and D. B. West, <a href="http://dx.doi.org/10.1016/S0012-365X(01)00239-4">A Fibonacci tiling of the plane</a>, Discrete Math., 249 (2002), 111-116.
   - Q.-H. Hou, Z.-W. Sun and H.-M. Wen, <a href="http://arxiv.org/abs/1208.3903">On monotonicity of some combinatorial sequences</a>, arXiv:1208.3903 [math.CO], 2012-2014.
   - James P. Jones, <a href="http://www.fq.math.ca/Scanned/13-1/jones.pdf">Diophantine representation of the Fibonacci numbers</a>, Fibonacci Quarterly 13:1 (1975), pp. 84-88.
   - S. Kak, <a href="http://arXiv.org/abs/physics/0411195">The Golden Mean and the Physics of Aesthetics</a>, arXiv:physics/0411195 [physics.hist-ph], 2004.
   - Louis H. Kauffman and Pedro Lopes, <a href="http://arXiv.org/abs/0710.3765">Graded forests and rational knots</a>, arXiv:0710.3765 [math.GT], 2007-2009.
   - Blair Kelly, <a href="http://mersennus.net/fibonacci//">Fibonacci and Lucas factorizations</a>
   - Tanya Khovanova, <a href="http://www.tanyakhovanova.com/RecursiveSequences/RecursiveSequences.html">Recursive Sequences</a>
   - C. Kimberling, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL6/Kimberling/kimberling24.html">Matrix Transformations of Integer Sequences</a>, J. Integer Seqs., Vol. 6, 2003.
   - Ron Knott, <a href="http://www.maths.surrey.ac.uk/hosted-sites/R.Knott/Fibonacci/">Fibonacci numbers and the golden section</a>
   - Ron Knott, <a href="http://www.maths.surrey.ac.uk/hosted-sites/R.Knott/Fibonacci/fibmaths.html">Mathematics of the Fibonacci Series</a>
   - Ron Knott, <a href="http://www.maths.surrey.ac.uk/hosted-sites/R.Knott/Fibonacci/fibtable.html">Fibonacci numbers with tables of F(0)-F(500).</a>
   - Hendrik Lenstra, <a href="http://math.berkeley.edu/~hwl/papers/fibo.pdf">Profinite Fibonacci Numbers</a>
   - M. A. Lerma, <a href="http://www.math.northwestern.edu/~mlerma/problem_solving/results/recurrences.pdf">Recurrence Relations</a>
   - F. Luca, V. J. M. Huguet, F. Nicolae, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL12/Mejia/luca31.html">On the Euler Function of Fibonacci Numbers</a>, JIS 12 (2009) 09.6.6
   - Charles P. McKeague, <a href="http://www.youtube.com/watch?v=_NmSEEhtc1U">Fibonacci numbers from MathTV</a>
   - Graeme McRae, <a href="http://oeis.org/wiki/User:Graeme_McRae/Sum_of_2m_Consecutive_Fibonacci_Numbers">Sum of 2m consecutive Fibonacci numbers</a>
   - Jean-Christophe Michel, <a href="http://framy.free.fr/fibonacci%20dans%20mandelbrot.htm">Le nombre d'or dans l'ensemble de Mandelbrot</a> (in French, 'The golden number in the Mandelbrot set')
   - Kerry Mitchell, <a href="http://kerrymitchellart.com/articles/Spirolateral-Type_Images_from_Integer_Sequences.pdf">Spirolateral-Type Images from Integer Sequences</a>, 2013
   - H. Mishima, Factorizations of Fibonacci numbers <a href="http://www.asahi-net.or.jp/~KC2H-MSM/mathland/matha1/matha108.htm">n=1..100</a>, <a href="http://www.asahi-net.or.jp/~KC2H-MSM/mathland/matha1/matha109.htm">n=101..200</a>, <a href="http://www.asahi-net.or.jp/~KC2H-MSM/mathland/matha1/matha110.htm">n=201..300</a>, <a href="http://www.asahi-net.or.jp/~KC2H-MSM/mathland/matha1/matha111.htm">n=301..400</a>, <a href="http://www.asahi-net.or.jp/~KC2H-MSM/mathland/matha1/matha112.htm">n=401..480</a>
   - P. Moree, <a href="http://arXiv.org/abs/math.CO/0311205">Convoluted convolved Fibonacci numbers</a>, arXiv:math/0311205 [math.CO], 2003.
   - R. Mullen, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL12/Mullen/mullen2.html">On Determining Paint by Numbers Puzzles with Nonunique Solutions</a>, JIS 12 (2009) 09.6.5
   - Newton Institute, <a href="http://www.newton.cam.ac.uk/wmy2kposters/january">Posters in the London Underground</a>
   - Tony D. Noe and Jonathan Vos Post, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL8/Noe/noe5.html">Primes in Fibonacci n-step and Lucas n-step Sequences</a>, Journal of Integer Sequences, Vol. 8 (2005), Article 05.4.4.
   - J. J. O'Connor and E. F. Robertson, Mac Tutor History of Mathematics, <a href="http://www-history.mcs.st-andrews.ac.uk/Biographies/Hemchandra.html">Archarya Hemachandra</a>
   - Arzu Özkoç, <a href="http://link.springer.com/article/10.1186/s13662-015-0486-7/fulltext.html">Some algebraic identities on quadra Fibona-Pell integer sequence</a>, Advances in Difference Equations, 2015, 2015:148.
   - Ram Krishna Pandey, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL16/Pandey/pandey7.html">On Some Magnified Fibonacci Numbers Modulo a Lucas Number</a>, Journal of Integer Sequences, Vol. 16 (2013), #13.1.7.
   - Ed Pegg, Jr., <a href="http://www.mathpuzzle.com/MAA/07-Sequence%20Pictures/mathgames_12_08_03.html">Sequence Pictures</a>, Math Games column, Dec 08 2003.
   - Ed Pegg, Jr., <a href="/<a href="http://oeis.org/A000043">A000043</a>/a000043_2.pdf">Sequence Pictures</a>, Math Games column, Dec 08 2003 [Cached copy, with permission (pdf only)]
   - T. K. Petersen and B. E. Tenner, <a href="http://arxiv.org/abs/1202.4765">The depth of a permutation</a>, arXiv:1202.4765v1 [math.CO], 2012-2014.
   - Ivars Peterson, <a href="http://www.sciencenews.org/articles/20060603/mathtrek.asp">Fibonacci's Missing Flowers</a>.
   - Akos Pinter and Volker Ziegler, <a href="http://arxiv.org/abs/1005.3624">On Arithmetic Progressions in Recurrences - A new characterization of the Fibonacci sequence</a>, arXiv:1005.3624 [math.NT], 2010.
   - Project Nayuki, <a href="http://www.nayuki.io/page/fast-fibonacci-algorithms">Fast Fibonacci algorithms</a> (fast doubling is faster than matrix multiplication).
   - S. Rabinowitz, <a href="http://www.mathpropress.com/stan/bibliography/algorithmicFib.pdf">Algorithmic Manipulation of Fibonacci Identities</a> (1996).
   - Arulalan Rajan, R. Vittal Rao, Ashok Rao and H. S. Jamadagni, <a href="http://arxiv.org/abs/1205.5398">Fibonacci Sequence, Recurrence Relations, Discrete Probability Distributions and Linear Convolution</a>, arXiv preprint arXiv:1205.5398 [math.PR], 2012. - From _N. J. A. Sloane_, Oct 23 2012
   - Marc Renault, <a href="http://www.math.temple.edu/~renault/fibonacci/thesis.html">Properties of the Fibonacci sequence under various moduli</a>, MSc Thesis, Wake Forest U, 1996.
   - N. Renton, <a href="http://www.users.bigpond.net.au/renton/903.htm">The fibonacci Series</a>
   - B. Rittaud, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL10/Rittaud2/rittaud11.pdf">On the Average Growth of Random Fibonacci Sequences</a>, Journal of Integer Sequences, 10 (2007), Article 07.2.4.
   - Rosetta Code, <a href="http://rosettacode.org/wiki/Fibonacci_sequence">A collection of codes to compute fibonacci numbers with different computer languages</a>
   - E. S. Rowland, <a href="http://people.hofstra.edu/Eric_Rowland/investigations/fibonacci.html">Fibonacci Sequence Calculator up to n=1474</a>
   - Michelle Rudolph-Lilith, <a href="http://arxiv.org/abs/1508.07894">On the Product Representation of Number Sequences, with Application to the Fibonacci Family</a>, arXiv preprint arXiv:1508.07894, 2015
   - Shiva Samieinia, <a href="http://www.math.su.se/reports/2007/6/">Digital straight line segments and curves</a>. Licentiate Thesis. Stockholm University, Department of Mathematics, Report 2007:6.
   - A. Sapounakis, I. Tasoulas and P. Tsikouras, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL9/Tsikouras/tsikouras67.html">On the Dominance Partial Ordering of Dyck Paths</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.2.5.
   - Mark A. Shattuck, <a href="http://www.emis.de/journals/INTEGERS/papers/j5/j5.Abstract.html">Tiling proofs of some formulas for the Pell numbers of odd index</a>, Integers, 9 (2009), 53-64.
   - Mark A. Shattuck and Carl G. Wagner, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL9/Shattuck/shattuck56.html">Periodicity and Parity Theorems for a Statistic on r-Mino Arrangements</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.3.6.
   - Parmanand Singh, <a href="http://dx.doi.org/10.1016/0315-0860(85)90021-7">The so-called Fibonacci numbers in ancient and medieval India</a>, Historia Mathematica, Volume 12 (3), 1985, 229-244.
   - Jaap Spies, <a href="http://www.jaapspies.nl/oeis/a000045.sage">Sage program for computing <a href="http://oeis.org/A000045">A000045</a></a>
   - Michael Z. Spivey and Laura L. Steil, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL9/Spivey/spivey7.html">The k-Binomial Transforms and the Hankel Transform</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.1.1.
   - Z.-H. Sun, <a href="http://202.195.112.2/xsjl/szh/ConFn.pdf">Congruences For Fibonacci Numbers</a>
   - Roberto Tauraso, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL7/Tauraso/tauraso3.html">A New Domino Tiling Sequence</a>, Journal of Integer Sequences, Vol. 7 (2004), Article 04.2.3.
   - K. Tognetti, <a href="/<a href="http://oeis.org/A000045">A000045</a>/a000045.pdf">Letter to N. J. A. Sloane (with attachments), May 25 1994</a>
   - K. Tognetti, <a href="/<a href="http://oeis.org/A000045">A000045</a>/a000045_2.pdf">The Search for the Golden Sequence</a>, Draft Manuscript, May 25 1994.
   - K. Tognetti, <a href="http://www.austms.org.au/Modules/Fib">Fibonacci-His Rabbits and His Numbers and Kepler</a>
   - Tony van Ravenstein, <a href="/<a href="http://oeis.org/A000045">A000045</a>/a000045_1.pdf">The three gap theorem (Steinhaus conjecture)</a>, Journal of the Australian Mathematical Society (Series A) 45.03 (1988): 360-370. [Annotated scanned copy]
   - C. Vila, <a href="http://www.boingboing.net/2010/03/22/dreamlike-animation.html">Nature by numbers</a> (animation).
   - Christobal Vila, <a href="http://wimp.com/naturenumbers/">Nature Numbers</a> (Video related to Fibonacci numbers)
   - N. N. Vorob'ev, <a href="http://eom.springer.de/F/f040020.htm">Fibonacci numbers</a>, Springer's Encyclopaedia of Mathematics.
   - Carl G. Wagner, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL7/Wagner/wagner3.html">Partition Statistics and q-Bell Numbers (q = -1)</a>, J. Integer Seqs., Vol. 7, 2004.
   - Robert Walker, <a href="http://www.youtube.com/watch?v=Wx4ZfuMl-FI&amp;NR=1">Inharmonic "Golden Rhythmicon" - Fibonacci Sequence in Pairs Approaching Golden Ratio - With Bounce</a>
   - Eric Weisstein's World of Mathematics, <a href="http://mathworld.wolfram.com/FibonacciNumber.html">Fibonacci Number</a>, <a href="http://mathworld.wolfram.com/Double-FreeSet.html">Double-Free Set</a>, <a href="http://mathworld.wolfram.com/Fibonaccin-StepNumber.html">Fibonacci n-Step Number</a>, <a href="http://mathworld.wolfram.com/ResistorNetwork.html">Resistor Network</a>
   - Eric Weisstein's World of Mathematics, <a href="http://mathworld.wolfram.com/HosoyaIndex.html">Hosoya Index</a>
   - Eric Weisstein's World of Mathematics, <a href="http://mathworld.wolfram.com/PathGraph.html">Path Graph</a>
   - Wikipedia, <a href="http://www.wikipedia.org/wiki/Fibonacci_number">Fibonacci number</a>
   - Wikipedia, <a href="https://en.wikipedia.org/wiki/Cassini_and_Catalan_identities">Cassini and Catalan identities</a>
   - Willem's Fibonacci site, <a href="http://home.zonnet.nl/LeonardEuler/fiboe.htm">Fibonacci</a>
   - Mike Winkler, <a href="http://arxiv.org/abs/1412.0519">On the structure and the behaviour of Collatz 3n + 1 sequences - Finite subsequences and the role of the Fibonacci sequence</a>, arXiv:1412.0519 [math.GM], 2014.
   - Roman Witula, Damian Slota and Edyta Hetmaniok, <a href="http://ami.ektf.hu/uploads/papers/finalpdf/AMI_41_from255to263.pdf">Bridges between different known integer sequences</a>, Annales Mathematicae et Informaticae, 41 (2013) pp. 255-263.
   - Aimei Yu and Xuezheng Lv, <a href="http://dx.doi.org/10.1007/s10910-006-9088-7">The Merrifield-Simmons indices and Hosoya indices of trees with k pendant vertices</a>, J. Math. Chem., Vol. 41 (2007), pp. 33-43. See page 35.
   - Tianping Zhang and Yuankui Ma, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL8/Zhang/zhang56.html">On Generalized Fibonacci Polynomials and Bernoulli Numbers</a>, Journal of Integer Sequences, Vol. 8 (2005), Article 05.5.3.
   - <a href="/index/Cor#core">Index entries for "core" sequences</a>
   - <a href="/index/Di#divseq">Index to divisibility sequences</a>
   - <a href="/index/Par#partN">Index entries for related partition-counting sequences</a>
   - <a href="/index/Rec#order_02">Index entries for linear recurrences with constant coefficients</a>, signature (1,1).
   - <a href="/index/Tu#2wis">Index entries for two-way infinite sequences</a>

_References_:
   - Abrate, Marco; Barbero, Stefano; Cerruti, Umberto; Murru, Nadir. Colored compositions, Invert operator and elegant compositions with the "black tie". Discrete Math. 335 (2014), 1-7. MR3248794
   - Andrews, George E. Fibonacci numbers and the Rogers-Ramanujan identities. Fibonacci Quart. 42 (2004), no. 1, 3-19. MR2060558(2005b:11161)
   - S. V. Ault and C. Kicey, Counting paths in corridors using circular Pascal arrays, Discrete Mathematics (2014).
   - Mohammad K. Azarian, The Generating Function for the Fibonacci Sequence, Missouri Journal of Mathematical Sciences, Vol. 2, No. 2, Spring 1990, pp. 78-79. Zentralblatt MATH, Zbl 1097.11516.
   - Mohammad K. Azarian, A Generalization of the Climbing Stairs Problem II, Missouri Journal of Mathematical Sciences, Vol. 16, No. 1, Winter 2004, pp. 12-17.
   - Mohammad K. Azarian, Fibonacci Identities as Binomial Sums, International Journal of Contemporary Mathematical Sciences, Vol. 7,  No. 38, 2012, pp. 1871-1876.
   - Mohammad K. Azarian, Fibonacci Identities as Binomial Sums II, International Journal of Contemporary Mathematical Sciences, Vol. 7,  No. 42, 2012, pp. 2053-2059.
   - Mohammad K. Azarian, Identities Involving Lucas or Fibonacci and Lucas Numbers as Binomial Sums, International Journal of Contemporary Mathematical Sciences, Vol. 7, No. 45, 2012, pp. 2221-2227.
   - P. Bachmann, Niedere Zahlentheorie (1902, 1910), reprinted Chelsea, NY, 1968, vol. 2, p. 70.
   - R. B. Banks, Slicing Pizzas, Racing Turtles and Further Adventures in Applied Mathematics, Princeton Univ. Press, 1999. See p. 84.
   - A. T. Benjamin and J. J. Quinn, Proofs that really count: the art of combinatorial proof, M.A.A. 2003, id. 4.
   - Marjorie Bicknell and Verner E Hoggatt, Fibonacci's Problem Book, Fibonacci Association, San Jose, Calif., 1974.
   - A. Cayley, Theorems in Trigonometry and on Partitions, Messenger of Mathematics, 5 (1876), pp. 164, 188 = Mathematical Papers Vol. 10, n. 634, p. 16.
   - D. Chmiela, K. Kaczmarek, R. Witula, Binomials Transformation Formulas of Scaled Fibonacci Numbers (submitted 2012).
   - B. A. Davey and H. A. Priestley, Introduction to Lattices and Order (2nd edition), CUP, 2002. (See Exercise 1.15.)
   - B. Davis, 'The law of first digits' in 'Science Today'(subsequently renamed '2001') March 1980 p. 55, Times of India, Mumbai.
   - S. R. Finch, Mathematical Constants, Cambridge, 2003, Section 1.2.
   - R. P. Grimaldi, Compositions without the summand 1, Proceedings Thirty-second Southeastern International Conference on Combinatorics, Graph Theory and Computing (Baton Rouge, LA, 2001). Congr. Numer. 152 (2001), 33-43.
   - H. Halberstam and K. F. Roth, Sequences, Oxford, 1966; see Appendix.
   - S. Happersett, "Mathematical meditations", Journal of Mathematics and the Arts, 1 (2007), 29 - 33.
   - G. H. Hardy and E. M. Wright, An Introduction to the Theory of Numbers. 3rd ed., Oxford Univ. Press, 1954; see esp. p. 148.
   - J. Hermes, Anzahl der Zerlegungen einer ganzen rationalen Zahl in Summanden, Math. Ann., 45 (1894), 371-380.
   - V. E. Hoggatt, Jr., Fibonacci and Lucas Numbers. Houghton, Boston, MA, 1969.
   - E. Horowitz and S. Sahni, Fundamentals of Data Structures, Computer Science Press, 1976; p. 338.
   - P. W. Kasteleyn, The statistics of dimers on a lattice. I. The number of dimer arrangements on a quadratic lattice, Physica, 27 (1961), 1209-1225.
   - M. Kauers and P. Paule, The Concrete Tetrahedron, Springer 2011, p. 63.
   - C. Kicey and K. Klimko, Some geometry of Pascal's triangle, Pi Mu Epsilon Journal, 13(4):229-245 (2011).
   - D. E. Knuth, The Art of Computer Programming. Addison-Wesley, Reading, MA, Vol. 1, p. 78; Vol. 3, Section 6.2.1.
   - Thomas Koshy, "Fibonacci and Lucas Numbers with Applications", John Wiley and Sons, 2001.
   - Leonardo of Pisa [Leonardo Pisano], Liber Abaci [The Book of Calculation], 1202.
   - Lukovits et al., Nanotubes: Number of Kekulé structures and aromaticity, J. Chem. Inf. Comput. Sci, (2003), vol. 43, 609-614. See eq. 2 on page 610.
   - I. Lukovits and D. Janezic, "Enumeration of conjugated circuits in nanotubes", J. Chem. Inf. Comput. Sci., vol. 44, 410-414 (2004). See Table 1, second column.
   - B. Malesevic: Some combinatorial aspects of differential operation composition on the space R^n, Univ. Beograd, Publ. Elektrotehn. Fak., Ser. Mat. 9 (1998), 29-33.
   - G. Mantel, Resten van wederkeerige Reeksen, Nieuw Archief v. Wiskunde, 2nd series, I (1894), 172-184.
   - C. N. Menhinick, The Fibonacci Resonance and other new Golden Ratio discoveries, Onperson, (2015), pages 200-206.
   - A. Milicevic and N. Trinajstic, "Combinatorial Enumeration in Chemistry", Chem. Modell., Vol. 4, (2006), pp. 405-469.
   - S. Mneimneh, Fibonacci in The Curriculum: Not Just a Bad Recurrence, in Proceeding SIGCSE '15 Proceedings of the 46th ACM Technical Symposium on Computer Science Education, Pages 253-258.
   - Clifford A. Pickover, A Passion for Mathematics, Wiley, 2005; see p. 49.
   - Clifford A. Pickover, The Math Book, From Pythagoras to the 57th Dimension, 250 Milestones in the History of Mathematics, Sterling Publ., NY, 2009, page 274.
   - A. S. Posamentier & I. Lehmann, The Fabulous Fibonacci Numbers, Prometheus Books, Amherst, NY 2007.
   - P. Ribenboim, The New Book of Prime Number Records, Springer, 1996.
   - J. Riordan, An Introduction to Combinatorial Analysis, Princeton University Press, Princeton, NJ, 1978.
   - A. M. Robert, A Course in p-adic Analysis, Springer-Verlag, 2000; p. 213.
   - J. Roberts, Lure of the Integers, Math. Assoc. America, 1992, p. 288.
   - Manfred R. Schroeder, "Number Theory in Science and Communication", 5th ed., Springer-Verlag, 2009
   - L. E. Sigler, Fibonacci's Liber Abaci, Springer, 2003, pp. 404-405 and [26] on p. 627.
   - Simson, [No first name given], An explanation of an obscure passage in Albert Girard's Commentary ..., Phil. Trans. Royal Soc., 10 (1753), 430-433.
   - N. J. A. Sloane, A Handbook of Integer Sequences, Academic Press, 1973 (includes this sequence).
   - N. J. A. Sloane and Simon Plouffe, The Encyclopedia of Integer Sequences, Academic Press, 1995 (includes this sequence).
   - S. Vajda, Fibonacci and Lucas numbers and the Golden Section, Ellis Horwood Ltd., Chichester, 1989.
   - Van Ravenstein, Tony. "The three gap theorem (Steinhaus conjecture)." Journal of the Australian Mathematical Society (Series A) 45.03 (1988): 360-370.
   - N. N. Vorob'ev, Chisla fibonachchi [Russian], Moscow, 1951. English translation, Fibonacci Numbers, Blaisdell, New York and London, 1961.
   - N. N. Vorobiev, Fibonacci Numbers, Birkhauser (Basel;Boston) 2002.
   - A. Y. Z. Wang, P. Wen, On the partial finite sums of the reciprocals of the Fibonacci numbers, Journal of Inequalities and Applications, 2015; http://www.journalofinequalitiesandapplications.com/content/2015/1/73
   - D. Wells, The Penguin Dictionary of Curious and Interesting Numbers, pp. 61-7, Penguin Books 1987.
   - R. Witula, D. Slota, delta-Fibonacci Numbers, Appl. Anal. Discrete Math., 3 (2009), 310-329.

### Search by sequence of numbers

In [25]:
searchable = oeis_search(seq=[1,1,2,5,14,42, 132, 429])

In [26]:
searchable(data_upper_limit=15, comment=lambda i,c: i in range(10))

_Results for query: <a href='https://oeis.org/search?q=1%2C+1%2C+2%2C+5%2C+14%2C+42%2C+132%2C+429&fmt=json&start=0'>https://oeis.org/search?q=1%2C+1%2C+2%2C+5%2C+14%2C+42%2C+132%2C+429&fmt=json&start=0</a>_<br><hr><div align='center'><b><a href='http://oeis.org/A000108'>A000108</a></b>: <i>Catalan numbers: C(n) = binomial(2n,n)/(n+1) = (2n)!/(n!(n+1)!). Also called Segner numbers.</i><br></div>

by _N. J. A. Sloane_

_Keywords_: `core,nonn,easy,eigen,nice,changed`

_Data_:

$$
\begin{array}{c|ccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 \\
\hline
A000108(n) & 1 & 1 & 2 & 5 & 14 & 42 & 132 & 429 & 1430 & 4862 & 16796 & 58786 & 208012 & 742900 & 2674440
\end{array}
$$


_Comments_:
   - The solution to Schröder's first problem. A very large number of combinatorial interpretations are known - see references, esp. Stanley, Enumerative Combinatorics, Volume 2. This is probably the longest entry in the OEIS, and rightly so.
   - Number of ways to insert n pairs of parentheses in a word of n+1 letters. E.g., for n=2 there are 2 ways: ((ab)c) or (a(bc)); for n=3 there are 5 ways: ((ab)(cd)), (((ab)c)d), ((a(bc))d), (a((bc)d)), (a(b(cd))).
   - Consider all the binomial(2n,n) paths on squared paper that (i) start at (0, 0), (ii) end at (2n, 0) and (iii) at each step, either make a (+1,+1) step or a (+1,-1) step. Then the number of such paths that never go below the x-axis (Dyck paths) is C(n). [Chung-Feller]
   - Number of noncrossing partitions of the n-set. For example, of the 15 set partitions of the 4-set, only [{13},{24}] is crossing, so there are a(4)=14 noncrossing partitions of 4 elements. - _Joerg Arndt_, Jul 11 2011
   - a(n-1) is the number of ways of expressing an n-cycle in the symmetric group S_n as a product of n-1 transpositions (u_1,v_1)*(u_2,v_2)*...*(u_{n-1},v_{n-1}) where u_k<=u_j and v_k<=v_j for k<j; see example.  If the condition is dropped, one obtains <a href="http://oeis.org/A000272">A000272</a>. - _Joerg Arndt_ and Greg Stevenson, Jul 11 2011
   - a(n) is the number of ordered rooted trees with n nodes, not including the root. See the Conway-Guy reference where these rooted ordered trees are called plane bushes. See also the Bergeron et al. reference, Example 4, p. 167. - _Wolfdieter Lang_, Aug 07 2007
   - As shown in the paper from Beineke and Pippert (1971), a(n-2)=D(n) is the number of labeled dissections of a disk, related to the number R(n)=<a href="http://oeis.org/A001761">A001761</a>(n-2) of labeled planar 2-trees having n vertices and rooted at a given exterior edge, by the formula D(n)=R(n)/(n-2)!. - _M. F. Hasler_, Feb 22 2012
   - Shifts one place left when convolved with itself.
   - For n >= 1 a(n) is also the number of rooted bicolored unicellular maps of genus 0 on n edges. - Ahmed Fares (ahmedfares(AT)my-deja.com), Aug 15 2001
   - Ways of joining 2n points on a circle to form n nonintersecting chords. (If no such restriction imposed, then ways of forming n chords is given by (2n-1)!!=(2n)!/n!2^n=<a href="http://oeis.org/A001147">A001147</a>(n).)

_Formulae_:
   - a(n) = <a href="http://oeis.org/A000984">A000984</a>(n)/(n+1) = binomial(2*n, n)/(n+1) = (2*n)!/(n!*(n+1)!).
   - a(n) = binomial(2*n, n)-binomial(2*n, n-1).
   - a(n) = Sum_{k=0..n-1} a(k)a(n-1-k).
   - a(n) = Product_{k=2..n} (1 + n/k), if n>1.
   - G.f.: A(x) = (1 - sqrt(1 - 4*x)) / (2*x). G.f. A(x) satisfies A = 1 + x*A^2.
   - a(n+1) = Sum_{i} binomial(n, 2*i)*2^(n-2*i)*a(i). - Touchard
   - 2*(2*n-1)*a(n-1) = (n+1)*a(n).
   - It is known that a(n) is odd if and only if n=2^k-1, k=0, 1, 2, 3, ... - _Emeric Deutsch_, Aug 04 2002, corrected by _M. F. Hasler_, Nov 08 2015
   - Using the Stirling approximation in <a href="http://oeis.org/A000142">A000142</a> we get the asymptotic expansion a(n) ~ 4^n / (sqrt(Pi * n) * (n + 1)). - Dan Fux (dan.fux(AT)OpenGaia.com or danfux(AT)OpenGaia.com), Apr 13 2001
   - Integral representation: a(n) = int(x^n*sqrt((4-x)/x), x=0..4)/(2*Pi). - _Karol A. Penson_, Apr 12 2001
   - E.g.f.: exp(2*x)*(I_0(2*x)-I_1(2*x)), where I_n is Bessel function. - _Karol A. Penson_, Oct 07 2001
   - Polygorial(n, 6)/Polygorial(n, 3). - Daniel Dockery (peritus(AT)gmail.com), Jun 24 2003
   - G.f. A(x) satisfies ((A(x) + A(-x)) / 2)^2 = A(4*x^2). - _Michael Somos_, Jun 27, 2003
   - G.f. A(x) satisfies Sum_{k>=1} k(A(x)-1)^k = Sum_{n >= 1} 4^{n-1}*x^n. - Shapiro, Woan, Getu
   - a(n+m) = Sum_{k} <a href="http://oeis.org/A039599">A039599</a>(n, k)*<a href="http://oeis.org/A039599">A039599</a>(m, k). - _Philippe Deléham_, Dec 22 2003
   - a(n+1) = (1/(n+1))*Sum_{k=0..n} a(n-k)*binomial(2k+1, k+1). - _Philippe Deléham_, Jan 24 2004
   - a(n) = Sum_{k>=0} <a href="http://oeis.org/A008313">A008313</a>(n, k)^2. - _Philippe Deléham_, Feb 14 2004
   - a(m+n+1) = Sum_{k>=0} <a href="http://oeis.org/A039598">A039598</a>(m, k)*<a href="http://oeis.org/A039598">A039598</a>(n, k). - _Philippe Deléham_, Feb 15 2004
   - C(n-1) = binomial(2*n-2,n-1)/n = (1/n!) * [ n^(n-1) + ( binomial(n-2,1) + binomial(n-2,2) )*n^(n-2) + ( 2*binomial(n-3,1) + 7*binomial(n-3,2) + 8*binomial(n-3,3) + 3*binomial(n-3,4) )*n^(n-3) + ( 6*binomial(n-4,1) + 38*binomial(n-4,2) + 93*binomial(n-4,3) + 111*binomial(n-4,4) + 65*binomial(n-4,5) + 15*binomial(n-4,6) )*n^(n-4) + ... ]. - _André F. Labossière_, Nov 10 2004, corrected by _M. F. Hasler_, Nov 10 2015
   - a(n) = Sum_{k=0..n} (-1)^k*2^(n-k)*binomial(n, k)*binomial(k, floor(k/2)). - Paul Barry, Jan 27 2005
   - Sum_{n>=0} 1/a(n) = 2 + 4*Pi/3^(5/2) = F(1,2;1/2;1/4) = 2.806133050770763... (see L'Univers de Pi link). - _Gerald McGarvey_ and _Benoit Cloitre_, Feb 13 2005
   - a(n) = Sum_{k=0..floor(n/2)} ((n-2*k+1)*binomial(n, n-k)/(n-k+1))^2, which is equivalent to: a(n) = Sum_{k=0..n} <a href="http://oeis.org/A053121">A053121</a>(n, k)^2, for n>=0. - _Paul D. Hanna_, Apr 23 2005
   - a((m+n)/2) = Sum_{k>=0} <a href="http://oeis.org/A053121">A053121</a>(m, k)*<a href="http://oeis.org/A053121">A053121</a>(n, k) if m+n is even. - _Philippe Deléham_, May 26 2005
   - E.g.f. Sum_{n>=0} a(n) * x^(2*n) / (2*n)! = BesselI(1, 2*x) / x. - _Michael Somos_, Jun 22 2005
   - Given g.f. A(x), then B(x) = x * A(x^3) satisfies 0 = f(x, B(X)) where f(u, v) = u - v + (u*v)^2 or B(x) = x + (x * B(x))^2 which implies B(-B(x)) = -x and also (1 + B^3) / B^2 = (1 - x^3) / x^2. - _Michael Somos_, Jun 27 2005
   - a(n) = a(n-1)*(4-6/(n+1)). a(n) = 2a(n-1)*(8a(n-2)+a(n-1))/(10a(n-2)-a(n-1)). - _Franklin T. Adams-Watters_, Feb 08 2006
   - Sum_{k>=1} a(k)/4^k = 1. - _Franklin T. Adams-Watters_, Jun 28 2006
   - a(n) = <a href="http://oeis.org/A047996">A047996</a>(2*n+1, n). - _Philippe Deléham_, Jul 25 2006
   - Binomial transform of <a href="http://oeis.org/A005043">A005043</a>. - _Philippe Deléham_, Oct 20 2006
   - a(n) = Sum_{k=0..n} (-1)^k*<a href="http://oeis.org/A116395">A116395</a>(n,k). - _Philippe Deléham_, Nov 07 2006
   - a(n) = (1/(s-n))*Sum_{k=0..n} (-1)^k (k+s-n)*binomial(s-n,k) * binomial(s+n-k,s) with s a nonnegative free integer [H. W. Gould].
   - a(k) = Sum_{i=1..k} |<a href="http://oeis.org/A008276">A008276</a>(i,k)| * (k-1)^(k-i) / k!. - _André F. Labossière_, May 29 2007
   - a(n) = Sum_{k=0..n} <a href="http://oeis.org/A129818">A129818</a>(n,k) * <a href="http://oeis.org/A007852">A007852</a>(k+1). - _Philippe Deléham_, Jun 20 2007
   - a(n) = Sum_{k=0..n} <a href="http://oeis.org/A109466">A109466</a>(n,k) * <a href="http://oeis.org/A127632">A127632</a>(k). - _Philippe Deléham_, Jun 20 2007
   - Row sums of triangle <a href="http://oeis.org/A124926">A124926</a>. - _Gary W. Adamson_, Oct 22 2007
   - Lim_{n->infinity}(1+Sum_{k=0..n}a(k)/<a href="http://oeis.org/A004171">A004171</a>(k)) = 4/Pi. - _Reinhard Zumkeller_, Aug 26 2008
   - a(n) = Sum_{k=0..n} <a href="http://oeis.org/A120730">A120730</a>(n,k)^2 and a(k+1) = Sum_{n>=k} <a href="http://oeis.org/A120730">A120730</a>(n,k). - _Philippe Deléham_, Oct 18 2008
   - Given an integer t >= 1 and initial values u = [a_0, a_1, ..., a_{t-1}], we may define an infinite sequence Phi(u) by setting a_n = a_{n-1} + a_0*a_{n-1} + a_1*a_{n-2} + ... + a_{n-2}*a_1 for n >= t. For example, the present sequence is Phi([1]) (also Phi([1,1])). - _Gary W. Adamson_, Oct 27 2008
   - a(n) = Sum_{l_1=0..n+1} Sum_{l_2=0..n}...Sum_{l_i=0..n-i}...Sum_{l_n=0..1} delta(l_1,l_2,...,l_i,...,l_n) where delta(l_1,l_2,...,l_i,...,l_n) = 0 if any l_i < l_(i+1) and l_(i+1) <> 0 for i=1..n-1 and delta(l_1,l_2,...,l_i,...,l_n) = 1 otherwise. - _Thomas Wieder_, Feb 25 2009
   - Let A(x) be the g.f., then B(x)=x*A(x) satisfies the differential equation B'(x)-2*B'(x)*B(x)-1=0. - _Vladimir Kruchinin_, Jan 18 2011
   - G.f.: 1/(1-x/(1-x/(1-x/(...)))) (continued fraction). - _Joerg Arndt_, Mar 18 2011
   - With F(x) = (1-2*x-sqrt(1-4*x))/(2*x) an o.g.f. in x for the Catalan series, G(x) = x/(1+x)^2 is the compositional inverse of F (nulling the n=0 term). - _Tom Copeland_, Sep 04 2011
   - With H(x) = 1/(dG(x)/dx) = (1+x)^3 / (1-x), the n-th Catalan number is given by (1/n!)*((H(x)*d/dx)^n)x evaluated at x=0, i.e., F(x) = exp(x*H(u)*d/du)u, evaluated at u = 0. Also, dF(x)/dx = H(F(x)), and H(x) is the o.g.f. for <a href="http://oeis.org/A115291">A115291</a>. - _Tom Copeland_, Sep 04 2011
   - With F(x) = {1-sqrt[1-4*x]}/2 an o.g.f. in x for the Catalan series, G(x)= x*(1-x) is the compositional inverse and this relates the Catalan numbers to the row sums of <a href="http://oeis.org/A125181">A125181</a>. - _Tom Copeland_, Sep 30 2011
   - With H(x)=1/(dG(x)/dx)= 1/(1-2x), the n-th Catalan number (offset 1) is given by (1/n!)*((H(x)*d/dx)^n)x evaluated at x=0, i.e., F(x) = exp(x*H(u)*d/du)u, evaluated at u = 0. Also, dF(x)/dx = H(F(x)). - _Tom Copeland_, Sep 30 2011
   - G.f.: (1-sqrt(1-4*x))/(2*x)=G(0) where G(k)=1+(4*k+1)*x/(k+1-2*x*(k+1)*(4*k+3)/(2*x*(4*k+3)+(2*k+3)/G(k+1))); (continued fraction). - _Sergei N. Gladkovskii_, Nov 30 2011
   - E.g.f.: exp(2*x)*(BesselI(0,2*x)-BesselI(1,2*x))=G(0) where G(k)=1+(4*k+1)*x/((k+1)*(2*k+1)-x*(k+1)*(2*k+1)*(4*k+3)/(x*(4*k+3)+(k+1)*(2*k+3)/G(k+1))); (continued fraction). - _Sergei N. Gladkovskii_, Nov 30 2011
   - E.g.f.: Hypergeometric([1/2],[2],4*x) which coincides with the e.g.f. given just above, and also by _Karol A. Penson_ further above. - _Wolfdieter Lang_, Jan 13 2012
   - a(n) = <a href="http://oeis.org/A208355">A208355</a>(2*n-1) = <a href="http://oeis.org/A208355">A208355</a>(2*n) for n > 0. - _Reinhard Zumkeller_, Mar 04 2012
   - G.f.: 1 + 2*x/(U(0)-2*x) where U(k)= k*(4*x+1) + 2*x + 2 - x*(2*k+3)*(2*k+4)/U(k+1); (continued fraction, Euler's 1st kind, 1-step). - _Sergei N. Gladkovskii_, Sep 20 2012
   - G.f.: hypergeom([1/2,1],[2],4*x). - _Joerg Arndt_, Apr 06 2013
   - Special values of Jacobi polynomials, in Maple notation: a(n) = 4^n*JacobiP(n,1,-1/2-n,-1)/(n+1). - _Karol A. Penson_, Jul 28 2013
   - For n > 0: a(n) = sum of row n in triangle <a href="http://oeis.org/A001263">A001263</a>. - _Reinhard Zumkeller_, Oct 10 2013
   - a(n) = binomial(2n,n-1)/n and a(n) mod n = binomial(2n,n) mod n = <a href="http://oeis.org/A059288">A059288</a>(n). - _Jonathan Sondow_, Dec 14 2013
   - a(n-1) = sum(t1+2*t2+...+n*tn=n, (-1)^(1+t1+t2+...+tn)*multinomial(t1+t2 +...+tn,t1,t2,...,tn)*a(1)^t1*a(2)^t2*...*a(n)^tn). - _Mircea Merca_, Feb 27 2014
   - a(n) = sum_{k=1..n} binomial(n+k-1,n)/n if n>0. _Alexander Adamchuk_, Mar 25 2014
   - a(n) = -2^(2*n+1) * binomial(n-1/2, -3/2). - _Peter Luschny_, May 06 2014
   - a(n) = (4*<a href="http://oeis.org/A000984">A000984</a>(n)-<a href="http://oeis.org/A000984">A000984</a>(n+1))/2. - _Stanislav Sykora_, Aug 09 2014
   - a(n) = <a href="http://oeis.org/A246458">A246458</a>(n) * <a href="http://oeis.org/A246466">A246466</a>(n). - _Tom Edgar_, Sep 02 2014
   - a(n) = (2*n)!*[x^(2*n)]hypergeom([],[2],x^2). - _Peter Luschny_, Jan 31 2015
   - a(n) = 4^(n-1)*hypergeom([3/2, 1-n], [3], 1). - _Peter Luschny_, Feb 03 2015
   - a(2n) = 2*<a href="http://oeis.org/A000150">A000150</a>(2n); a(2n+1) = 2*<a href="http://oeis.org/A000150">A000150</a>(2n+1) + a(n). - _John Bodeen_, Jun 24 2015
   - a(n) = Sum_{t=1, n+1} n^(t-1)*abs(stirling1(n+1, t)) / Sum_{t=1, n+1} abs(stirling1(n+1, t)), for n >0, see (10) in Cereceda link. - _Michel Marcus_, Oct 06 2015
   - a(n) ~ 4^(n-2)*(128+160/N^2+84/N^4+715/N^6-10180/N^8)/(N^(3/2)*Pi^(1/2)) where N=4*n+3. - _Peter Luschny_, Oct 14 2015
   - a(n) = sum_{k=1..floor((n+1)/2)} (-1)^(k-1)*binomial(n+1-k,k)*a(n-k) if n > 0; and a(0) = 1. - _David Pasino_, Jun 29 2016
   - Sum_{n>=0} (-1)^n/a(n) = 14/25 - 24*arccsch(2)/(25*sqrt(5)) = 14/25 - 24*<a href="http://oeis.org/A002390">A002390</a>/(25*sqrt(5)) = 0.353403708337278061333... - _Ilya Gutkovskiy_, Jun 30 2016

_Cross references_:
   - Cf. <a href="http://oeis.org/A000142">A000142</a>, <a href="http://oeis.org/A000245">A000245</a>, <a href="http://oeis.org/A000344">A000344</a>, <a href="http://oeis.org/A000588">A000588</a>, <a href="http://oeis.org/A000957">A000957</a>, <a href="http://oeis.org/A000984">A000984</a>, <a href="http://oeis.org/A001392">A001392</a>, <a href="http://oeis.org/A001453">A001453</a>, <a href="http://oeis.org/A001791">A001791</a>, <a href="http://oeis.org/A002057">A002057</a>, <a href="http://oeis.org/A002420">A002420</a>, <a href="http://oeis.org/A003046">A003046</a>, <a href="http://oeis.org/A003517">A003517</a>, <a href="http://oeis.org/A003518">A003518</a>, <a href="http://oeis.org/A003519">A003519</a>, <a href="http://oeis.org/A006480">A006480</a>, <a href="http://oeis.org/A008276">A008276</a>, <a href="http://oeis.org/A008549">A008549</a>, <a href="http://oeis.org/A014137">A014137</a>, <a href="http://oeis.org/A014138">A014138</a>, <a href="http://oeis.org/A014140">A014140</a>, <a href="http://oeis.org/A022553">A022553</a>, <a href="http://oeis.org/A024492">A024492</a>, <a href="http://oeis.org/A032357">A032357</a>, <a href="http://oeis.org/A032443">A032443</a>, <a href="http://oeis.org/A039599">A039599</a>, <a href="http://oeis.org/A048990">A048990</a>, <a href="http://oeis.org/A059288">A059288</a>, <a href="http://oeis.org/A068875">A068875</a>, <a href="http://oeis.org/A069640">A069640</a>, <a href="http://oeis.org/A086117">A086117</a>, <a href="http://oeis.org/A094216">A094216</a>, <a href="http://oeis.org/A094638">A094638</a>, <a href="http://oeis.org/A094639">A094639</a>, <a href="http://oeis.org/A098597">A098597</a>, <a href="http://oeis.org/A099731">A099731</a>, <a href="http://oeis.org/A119822">A119822</a>, <a href="http://oeis.org/A120304">A120304</a>, <a href="http://oeis.org/A124926">A124926</a>, <a href="http://oeis.org/A129763">A129763</a>, <a href="http://oeis.org/A137697">A137697</a>, <a href="http://oeis.org/A154559">A154559</a>, <a href="http://oeis.org/A161581">A161581</a>, <a href="http://oeis.org/A167892">A167892</a>, <a href="http://oeis.org/A167893">A167893</a>, <a href="http://oeis.org/A179277">A179277</a>, <a href="http://oeis.org/A211611">A211611</a>.
   - A row of <a href="http://oeis.org/A060854">A060854</a>.
   - See <a href="http://oeis.org/A001003">A001003</a>, <a href="http://oeis.org/A001190">A001190</a>, <a href="http://oeis.org/A001699">A001699</a>, <a href="http://oeis.org/A000081">A000081</a> for other ways to count parentheses.
   - Enumerates objects encoded by <a href="http://oeis.org/A014486">A014486</a>.
   - A diagonal of any of the essentially equivalent arrays <a href="http://oeis.org/A009766">A009766</a>, <a href="http://oeis.org/A030237">A030237</a>, <a href="http://oeis.org/A033184">A033184</a>, <a href="http://oeis.org/A059365">A059365</a>, <a href="http://oeis.org/A099039">A099039</a>, <a href="http://oeis.org/A106566">A106566</a>, <a href="http://oeis.org/A130020">A130020</a>, <a href="http://oeis.org/A047072">A047072</a>.
   - Cf. <a href="http://oeis.org/A051168">A051168</a> (diagonal of the square array described).
   - Cf. <a href="http://oeis.org/A033552">A033552</a>, <a href="http://oeis.org/A176137">A176137</a> (partitions into Catalan numbers).
   - Cf. <a href="http://oeis.org/A000753">A000753</a>, <a href="http://oeis.org/A000736">A000736</a> (Boustrophedon transforms).
   - Cf. <a href="http://oeis.org/A120303">A120303</a> (largest prime factor of Catalan number).
   - Cf. <a href="http://oeis.org/A121839">A121839</a> (reciprocal Catalan constant).
   - Cf. <a href="http://oeis.org/A038003">A038003</a>, <a href="http://oeis.org/A119861">A119861</a>, <a href="http://oeis.org/A119908">A119908</a>, <a href="http://oeis.org/A120274">A120274</a>, <a href="http://oeis.org/A120275">A120275</a> (odd Catalan number).
   - Cf. <a href="http://oeis.org/A002390">A002390</a> (decimal expansion of natural logarithm of golden ratio).
   - Coefficients of square root of the g.f. are <a href="http://oeis.org/A001795">A001795</a>/<a href="http://oeis.org/A046161">A046161</a>.
   - For a(n) mod 6 see <a href="http://oeis.org/A259667">A259667</a>.

_Links_:
   - Neil J. A. Sloane, K. D. Bajpai and Robert G. Wilson v, <a href="/<a href="http://oeis.org/A000108">A000108</a>/b000108.txt">Table of n, a(n) for n = 0..1000</a> (first 200 terms from Neil J. A. Sloane and the first 351 from K. D. Bajpai)
   - James Abello, <a href="http://dx.doi.org/10.1137/0404001">The weak Bruhat order of S consistent sets, and Catalan numbers</a>, SIAM J. Discrete Math. 4 (1991), 1-16.
   - P. C. Allaart and K. Kawamura, <a href="http://arxiv.org/abs/1110.1691">The Takagi function: a survey</a>, Real Analysis Exchange, 37 (2011/12), 1-54; arXiv:1110.1691 [math.CA]. See Section 3.2.
   - N. Alon, Y. Caro and I. Krasikov, <a href="http://www.math.tau.ac.il/~nogaa/PDFS/Publications/Bisection of trees and sequences.pdf">Bisection of trees and sequences</a>, Discrete Math., 114 (1993), 3-7. (See Lemma 2.1.)
   - G. Alvarez, J. E. Bergner, R. Lopez, <a href="http://arxiv.org/abs/1503.00044">Action graphs and Catalan numbers</a>, arXiv preprint arXiv:1503.00044, 2015
   - Joerg Arndt, <a href="http://www.jjj.de/fxt/#fxtbook">Matters Computational (The Fxtbook)</a>, p. 333 and p. 337.
   - Jean-Christophe Aval, <a href="http://arxiv.org/abs/0711.0906">Multivariate Fuss-Catalan numbers</a>, arXiv:0711.0906v1, Discrete Math., 308 (2008), 4660-4669.
   - M. Azaola and F. Santos, <a href="http://personales.unican.es/santosf/Articulos/">The number of triangulations of the cyclic polytope C(n,n-4)</a>, Discrete Comput. Geom., 27 (2002), 29-48. (C(n) = number of triangulations of cyclic polytope C(n,2).)
   - John Baez, <a href="http://math.ucr.edu/home/baez/week202.html">This week's finds in mathematical physics, Week 202</a>
   - C. Banderier, M. Bousquet-Mélou, A. Denise, P. Flajolet, D. Gardy and D. Gouyou-Beauchamps, <a href="http://algo.inria.fr/banderier/Papers/DiscMath99.ps">Generating Functions for Generating Trees</a>, Discrete Mathematics 246(1-3), March 2002, pp. 29-55.
   - E. Barcucci, A. Del Lungo, E. Pergola and R. Pinzani, <a href="http://www.dmtcs.org/volumes/abstracts/dm040103.abs.html">Permutations avoiding an increasing number of length-increasing forbidden subsequences</a>, Discrete Mathematics and Theoretical Computer Science 4, 2000, 31-44.
   - J.-L. Baril, <a href="http://www.combinatorics.org/Volume_18/PDF/v18i1p178.pdf">Classical sequences revisited with permutations avoiding dotted pattern</a>, Electronic Journal of Combinatorics, 18 (2011), #P178.
   - J.-L. Baril, T. Mansour, A. Petrossian, <a href="http://jl.baril.u-bourgogne.fr/equival.pdf">Equivalence classes of permutations modulo excedances</a>, 2014.
   - J.-L. Baril, J.-M. Pallo, <a href="http://jl.baril.u-bourgogne.fr/Motzkin.pdf">Motzkin subposet and Motzkin geodesics in Tamari lattices</a>, 2013.
   - Paul Barry, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL8/Barry/barry84.html">A Catalan Transform and Related Transformations on Integer Sequences</a>, Journal of Integer Sequences, Vol. 8 (2005), Article 05.4.5.
   - Paul Barry, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL9/Barry/barry91.html">On Integer-Sequence-Based Constructions of Generalized Pascal Triangles</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.2.4.
   - Paul Barry, <a href="http://arxiv.org/abs/1107.5490">Invariant number triangles, eigentriangles and Somos-4 sequences</a>, arXiv:1107.5490 [math.CO], 2011.
   - A. M. Baxter, L. K. Pudwell, <a href="http://faculty.valpo.edu/lpudwell/papers/AvoidingPairs.pdf">Ascent sequences avoiding pairs of patterns</a>, 2014.
   - L. W. Beineke and R. E. Pippert, Enumerating labeled k-dimensional trees and ball dissections, pp. 12-26 of Proceedings of Second Chapel Hill Conference on Combinatorial Mathematics and its Applications, University of North Carolina, Chapel Hill, 1970. Reprinted in <a href="http://dx.doi.org/10.1007/BF02330563">Math. Annalen 191 (1971), 87-98</a>.
   - Matthew Bennett, Vyjayanthi Chari, R. J. Dolbin and Nathan Manning, <a href="http://arxiv.org/abs/0912.4983">Square partitions and Catalan numbers</a>, arXiv:0912.4983 [math.RT], 2009.
   - E. E. Bernard and P. D. A. Mole, <a href="/<a href="http://oeis.org/A000108">A000108</a>/a000108_7.pdf">Generating strategies for continuous separation processes</a>, Computer J., 2 (1959), 87-89. [Annotated scanned copy]
   - M. Bernstein and N. J. A. Sloane, <a href="http://arXiv.org/abs/math.CO/0205301">Some canonical sequences of integers</a>, Linear Alg. Applications, 226-228 (1995), 57-72; erratum 320 (2000), 210.
   - D. Bill, <a href="http://www.durangobill.com/BinTrees.html">Durango Bill's Enumeration of Binary Trees</a>
   - Miklós Bóna, <a href="http://www.combinatorics.org/ojs/index.php/eljc/article/view/v19i1p62">Surprising Symmetries in Objects Counted by Catalan Numbers</a>, Electronic J. Combin., 19 (2012), P62.
   - H. Bottomley, <a href="/<a href="http://oeis.org/A000108">A000108</a>/a000108b.gif">Catalan Space Invaders</a>
   - H. Bottomley, <a href="/<a href="http://oeis.org/A002694">A002694</a>/a002694.gif">Illustration for <a href="http://oeis.org/A000108">A000108</a>, <a href="http://oeis.org/A001147">A001147</a>, <a href="http://oeis.org/A002694">A002694</a>, <a href="http://oeis.org/A067310">A067310</a> and <a href="http://oeis.org/A067311">A067311</a></a>
   - T. Bourgeron, <a href="http://www.dma.ens.fr/culturemath/maths/pdf/combi/montagnards.pdf">Montagnards et polygones</a>
   - Mireille Bousquet-Mélou, <a href="http://www.sciencedirect.com/science/article/pii/S0012365X00001461">Sorted and/or sortable permutations</a>, Discrete Mathematics, vol.225, no.1-3, pp.25-50, (2000).
   - M. Bousquet-Mélou and Gilles Schaeffer, <a href="http://www.labri.fr/Perso/~bousquet/Articles/Slitplane/PTRF/final.ps.gz">Walks on the slit plane</a>, Probability Theory and Related Fields, Vol. 124, no. 3 (2002), 305-344.
   - M Bouvel, V Guerrini, S Rinaldi, <a href="http://arxiv.org/abs/1511.04864">Slicings of parallelogram polyominoes, or how Baxter and Schroeder can be reconciled</a>, arXiv preprint arXiv:1511.04864, 2015
   - G. Bowlin and M. G. Brin, <a href="http://arxiv.org/abs/1301.3984">Coloring Planar Graphs via Colored Paths in the Associahedra</a>, arXiv preprint arXiv:1301.3984 [math.CO], 2013.
   - Douglas Bowman and Alon Regev, <a href="http://arxiv.org/abs/1209.6270">Counting symmetry classes of dissections of a convex regular polygon</a>, arXiv preprint arXiv:1209.6270 [math.CO], 2012.
   - W. G. Brown, <a href="/<a href="http://oeis.org/A000108">A000108</a>/a000108_5.pdf">Historical note on a recurrent combinatorial problem</a>,  Amer. Math. Monthly, 72 (1965), 973-977. [Annotated scanned copy]
   - B. Bukh, PlanetMath.org, <a href="http://planetmath.org/encyclopedia/CatalanNumbers.html">Catalan numbers</a>
   - Alexander Burstein, Sergi Elizalde and Toufik Mansour, <a href="http://arXiv.org/abs/math.CO/0610234">Restricted Dumont permutations, Dyck paths and noncrossing partitions</a>, arXiv:math/0610234 [math.CO], 2006.
   - W. Butler, A. Kalotay and N. J. A. Sloane, <a href="/<a href="http://oeis.org/A000108">A000108</a>/a000108_3.pdf">Correspondence, 1974</a>
   - W. Butler and N. J. A. Sloane, <a href="/<a href="http://oeis.org/A000108">A000108</a>/a000108_2.pdf">Correspondence, 1974</a>
   - David Callan, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL8/Callan/callan301.html">A Combinatorial Interpretation for a Super-Catalan Recurrence</a>, Journal of Integer Sequences, Vol. 8 (2005), Article 05.1.8.
   - David Callan, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL9/Callan/callan96.html">A Combinatorial Interpretation of the Eigensequence for Composition</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.1.4.
   - D. Callan, <a href="http://arxiv.org/abs/1204.5704">A variant of Touchard's Catalan number identity</a>, arXiv preprint arXiv:1204.5704 [math.CO], 2012.
   - David Callan and Emeric Deutsch, <a href="http://arxiv.org/abs/1112.3639">The Run Transform</a>,  Discrete Math. 312 (2012), no. 19, 2927-2937, arXiv:1112.3639 [math.CO], 2011.
   - N. T. Cameron, <a href="http://www.princeton.edu/~wmassey/NAM03/cameron.pdf">Random walks, trees and extensions of Riordan group techniques</a>
   - Naiomi T. Cameron and Asamoah Nkwanta, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL8/Cameron/cameron46.html">On Some (Pseudo) Involutions in the Riordan Group</a>, Journal of Integer Sequences, Vol. 8 (2005), Article 05.3.7.
   - P. J. Cameron, <a href="http://dx.doi.org/10.1093/qmath/38.2.155">Some treelike objects</a>, Quart. J. Math. Oxford, 38 (1987), 155-183. See p. 162
   - P. J. Cameron, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL3/groups.html">Sequences realized by oligomorphic permutation groups</a>, J. Integ. Seqs. Vol. 3 (2000), #00.1.5.
   - A. Cayley, <a href="http://dx.doi.org/10.1112/plms/s1-22.1.237">On the partitions of a polygon</a>, Proc. London Math. Soc., 22 (1891), 237-262 = Collected Mathematical Papers. Vols. 1-13, Cambridge Univ. Press, London, 1889-1897, Vol. 13, pp. 93ff.
   - F. Cazals, <a href="http://algo.inria.fr/libraries/autocomb/NCC-html/NCC.html">Combinatorics of Non-Crossing Configurations</a>, Studies in Automatic Combinatorics, Volume II (1997).
   - José Luis Cereceda, <a href="http://arxiv.org/abs/1510.00731">An alternative recursive formula for the sums of powers of integers</a>, arXiv:1510.00731 [math.CO], 2015.
   - G. Chatel, V. Pilaud, <a href="http://arxiv.org/abs/1411.3704">The Cambrian and Baxter-Cambrian Hopf Algebras</a>, arXiv preprint arXiv:1411.3704 [math.CO], 2014.
   - Julie Christophe, Jean-Paul Doignon and Samuel Fiorini, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL6/Doignon/doignon40.html">Counting Biorders</a>, J. Integer Seqs., Vol. 6, 2003.
   - J. Cigler, <a href="http://arxiv.org/abs/1109.1449">Some nice Hankel determinants</a>, arXiv:1109.1449 [math.CO], 2011.
   - J. Cigler, <a href="http://homepage.univie.ac.at/Johann.Cigler/preprints/chebyshev-survey.pdf">Some remarks about q-Chebyshev polynomials and q-Catalan numbers and related results</a>, 2013.
   - Alissa S. Crans, <a href="https://www.youtube.com/watch?v=eoofvKI_Okg">A surreptitious sequence: the Catalan numbers</a> (2014)
   - S. J. Cyvin, J. Brunvoll, E. Brendsdal, B. N. Cyvin and E. K. Lloyd, <a href="/<a href="http://oeis.org/A002057">A002057</a>/a002057.pdf">Enumeration of polyene hydrocarbons: a complete mathematical solution</a>, J. Chem. Inf. Comput. Sci., 35 (1995) 743-751. [Annotated scanned copy]
   - Dennis E. Davenport, Lara K. Pudwell, Louis W. Shapiro, Leon C. Woodson, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL18/Davenport/dav3.html">The Boundary of Ordered Trees</a>, Journal of Integer Sequences, Vol. 18 (2015), Article 15.5.8.
   - T. Davis, <a href="http://www.geometer.org/mathcircles/catalan.pdf">Catalan Numbers</a>
   - E. Deutsch and B. E. Sagan, <a href="http://arxiv.org/abs/math.CO/0407326">Congruences for Catalan and Motzkin numbers and related sequences</a>, J. Num. Theory 117 (2006), 191-215.
   - R. M. Dickau, <a href="http://mathforum.org/advanced/robertd/catalan.html">Catalan numbers</a> (<a href="http://www-groups.dcs.st-andrews.ac.uk/~history/Miscellaneous/CatalanNumbers/catalan.html">another copy</a>)
   - T. Dokos, I. Pak, <a href="http://arxiv.org/abs/1401.0770">The expected shape of random doubly alternating Baxter permutations</a>, arXiv:1401.0770 [math.CO], 2014.
   - Eric S. Egge, Kailee Rubin, <a href="http://arxiv.org/abs/1508.05310">Snow Leopard Permutations and Their Even and Odd Threads</a>, arXiv:1508.05310 [math.CO], 2015.
   - Shalosh B. Ekhad, Nathaniel Shar, and Doron Zeilberger, <a href="http://arxiv.org/abs/1504.02513">The number of 1...d-avoiding permutations of length d+r for SYMBOLIC d but numeric r</a>, arXiv:1504.02513 [math.CO], 2015.
   - I. M. H. Etherington, <a href="/<a href="http://oeis.org/A000108">A000108</a>/a000108_15.pdf">Non-associate powers and a functional equation</a>, Math. Gaz., 21 (1937), 36-39. [Annotated scanned copy]
   - I. M. H. Etherington, <a href="/<a href="http://oeis.org/A000108">A000108</a>/a000108_13.pdf">On non-associative combinations</a>, Proc. Royal Soc. Edinburgh, 59 (Part 2, 1938-39), 153-162. [Annotated scanned copy]
   - I. M. H. Etherington, <a href="/<a href="http://oeis.org/A000108">A000108</a>/a000108_14.pdf">Some problems of non-associative combinations (I)</a>, Edinburgh Math. Notes, 32 (1940), pp. i-vi. [Annotated scanned copy]. Part II [not scanned] is by A. Erdelyi and I. M. H. Etherington, and is on pages vii-xiv of the same issue.
   - Luca Ferrari and Emanuele Munarini, <a href="http://arxiv.org/abs/1203.6792">Enumeration of edges in some lattices of paths</a>, arXiv preprint arXiv:1203.6792 [math.CO], 2012. - From _N. J. A. Sloane_, Oct 03 2012
   - FindStat - Combinatorial Statistic Finder, <a href="http://www.findstat.org/StatisticsDatabase/St000028/">The number of stack-sorts needed to sort a permutation</a>
   - Philippe Flajolet, Éric Fusy, Xavier Gourdon, Daniel Panario and Nicolas Pouyanne, <a href="http://arxiv.org/abs/math/0606370">A hybrid of Darboux's method and singularity analysis in combinatorial asymptotics</a>, arXiv:math/0606370 [math.CO], 2006.
   - P. Flajolet and R. Sedgewick, <a href="http://algo.inria.fr/flajolet/Publications/books.html">Analytic Combinatorics</a>, 2009; see page 18, 35
   - D. Foata, G-N. Han, <a href="http://dx.doi.org/10.1007/s11139-009-9194-9">The doubloon polynomial triangle</a>, Ram. J. 23 (2010), 107-126
   - Dominique Foata and Guo-Niu Han, <a href="http://dx.doi.org/10.1093/qmath/hap043">Doubloons and new q-tangent numbers</a>, Quart. J. Math. 62 (2) (2011) 417-432
   - D. Foata and D. Zeilberger, <a href="http://www.math.rutgers.edu/~zeilberg/mamarim/mamarimPDF/classic.pdf">A classic proof of a recurrence for a very classical sequence</a>
   - S. Forcey, M. Kafashan, M. Maleki and M. Strayer, <a href="http://arxiv.org/abs/1212.1188">Recursive bijections for Catalan objects</a>, arXiv preprint arXiv:1212.1188 [math.CO], 2012.
   - H. G. Forder, <a href="/<a href="http://oeis.org/A000108">A000108</a>/a000108_4.pdf">Some problems in combinatorics</a>, Math. Gazette, vol. 45, 1961, 199-201. [Annotated scanned copy]
   - I. Galkin, <a href="http://ulcar.uml.edu/~iag/CS/Catalan.html">Enumeration of the Binary Trees(Catalan Numbers)</a>
   - Mohammad Ganjtabesh, Armin Morabbi and Jean-Marc Steyaert, <a href="http://www.lifl.fr/SEQUOIA/Arena/Presentations/Ganjtabesh.pdf">Enumerating the number of RNA structures</a>
   - E.-K. Ghang and D. Zeilberger, <a href="http://arxiv.org/abs/1303.0885">Zeroless Arithmetic: Representing Integers ONLY using ONE</a>, arXiv preprint arXiv:1303.0885 [math.CO], 2013.
   - A. Ghasemi, K. Sreenivas, L. K. Taylor, <a href="http://arxiv.org/abs/1309.4820">Numerical Stability and Catalan Numbers</a>, arXiv preprint arXiv:1309.4820 [math.NA], 2013.
   - Samuele Giraudo, <a href="http://arxiv.org/abs/1603.01394">Pluriassociative algebras II: The polydendriform operad and related operads</a>, arXiv:1603.01394 [math.CO], 2016.
   - Lisa R. Goldberg, <a href="http://www.sciencedirect.com/science/article/pii/0001870891900529">Catalan numbers and branched coverings by the Riemann sphere</a>, Adv. Math. 85 (1991), No. 2, 129-144.
   - K. Gorska and K. A. Penson, <a href="http://arxiv.org/abs/1304.6008">Multidimensional Catalan and related numbers as Hausdorff moments</a>, arXiv preprint arXiv:1304.6008 [math.CO], 2013.
   - H. W. Gould, <a href="http://www.ams.org/mathscinet-getitem?mr=2049119">Congr. Numer. 165 (2003) p 33-38</a>.
   - B. Gourevitch, <a href="http://www.pi314.net/">L'univers de Pi</a> (click Mathematiciens, Gosper)
   - Mats Granvik, <a href="http://pastebin.com/fsCtBUe1">Catalan numbers as convergents of power series</a>
   - Curtis Greene and Brady Haran, <a href="https://www.youtube.com/watch?v=JiwM5g_RC3c">Shapes and Hook Numbers (extra footage)</a> (2016)
   - H. G. Grundman and E. A. Teeple, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL10/Grundman/grundman5.html">Sequences of Generalized Happy Numbers with Small Bases</a>, Journal of Integer Sequences, Vol. 10 (2007), Article 07.1.8.
   - R. K. Guy, <a href="/<a href="http://oeis.org/A000108">A000108</a>/a000108_11.pdf">Dissecting a polygon into triangles</a>, Research Paper #9, Math. Dept., Univ. Calgary, 1967. [Annotated scanned copy]
   - R. K. Guy, Catwalks, Sandsteps and Pascal Pyramids, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL3/GUY/catwalks.html">J. Integer Seqs., Vol. 3 (2000), #00.1.6</a>
   - R. K. Guy and J. L. Selfridge, <a href="/<a href="http://oeis.org/A003018">A003018</a>/a003018.pdf">The nesting and roosting habits of the laddered parenthesis</a> (annotated cached copy)
   - Mark Haiman, with an Appendix by Ezra Miller, <a href="http://math.berkeley.edu/~mhaiman/ftp/msri-talks-2002/msri-comm-alg.pdf">Commutative algebra of n points in the plane</a>, Trends Commut. Algebra, MSRI Publ 51 (2004): 153-180. [See Theorem 1.2]
   - Guo-Niu Han, <a href="/<a href="http://oeis.org/A196265">A196265</a>/a196265.pdf">Enumeration of Standard Puzzles</a> [Cached copy]
   - Aoife Hennessy, <a href="http://repository.wit.ie/1693/1/AoifeThesis.pdf">A Study of Riordan Arrays with Applications to Continued Fractions, Orthogonal Polynomials and Lattice Paths</a>, Ph. D. Thesis, Waterford Institute of Technology, Oct. 2011
   - A. M. Hinz, S. Klavžar, U. Milutinović, C. Petr, <a href="http://dx.doi.org/10.1007/978-3-0348-0237-6">The Tower of Hanoi - Myths and Maths</a>, Birkhäuser 2013. See page 259. <a href="http://tohbook.info">Book's website</a>
   - V. E. Hoggatt, Jr., <a href="/<a href="http://oeis.org/A001628">A001628</a>/a001628.pdf">Letters to N. J. A. Sloane, 1974-1975</a>
   - V. E. Hoggatt, Jr. and M. Bicknell, <a href="http://www.fq.math.ca/Scanned/14-5/hoggatt1.pdf">Catalan and related sequences arising from inverses of Pascal's triangle matrices</a>, Fib. Quart., 14 (1976), 395-405.
   - V. E. Hoggatt, Jr., and Paul S. Bruckman, <a href="http://www.fq.math.ca/Scanned/13-4/hoggatt2.pdf">The H-convolution transform</a>, Fibonacci Quart., Vol. 13(4), 1975, p. 357.
   - C. Homberger, <a href="http://arxiv.org/abs/1410.2657">Patterns in Permutations and Involutions: A Structural and Enumerative Approach</a>, arXiv preprint arXiv:1410.2657 [math.CO], 2014.
   - Anders Hyllengren, <a href="/<a href="http://oeis.org/A001006">A001006</a>/a001006_5.pdf">Four integer sequences</a>, Oct 04 1985. Observes essentially that <a href="http://oeis.org/A000984">A000984</a> and <a href="http://oeis.org/A002426">A002426</a> are inverse binomial transforms of each other, as are <a href="http://oeis.org/A000108">A000108</a> and <a href="http://oeis.org/A001006">A001006</a>.
   - I. Jensen, <a href="http://www.ms.unimelb.edu.au/~iwan/polygons/Polygons_ser.html">Series exapansions for self-avoiding polygons</a>
   - S. Johnson, <a href="http://web.archive.org/web/20080129074833/http://www.saintanns.k12.ny.us/depart/math/Seth/catafrm.html">The Catalan Numbers</a>
   - A. Karttunen, <a href="/<a href="http://oeis.org/A014486">A014486</a>/a014486.pdf">Illustration of initial terms up to size n=7</a>
   - C. Kimberling, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL6/Kimberling/kimberling24.html">Matrix Transformations of Integer Sequences</a>, J. Integer Seqs., Vol. 6, 2003.
   - D. E. Knuth, <a href="http://arxiv.org/abs/math/9207221">Convolution polynomials</a>, The Mathematica J., 2 (1992), 67-78.
   - G. Kreweras, <a href="/<a href="http://oeis.org/A000108">A000108</a>/a000108_1.pdf">Sur les éventails de segments</a>, Cahiers du Bureau Universitaire de Recherche Opérationnelle, Institut de Statistique, Université de Paris, #15 (1970), 3-41. [Annotated scanned copy]
   - C. Krishnamachary and M. Bheemasena Rao, <a href="/<a href="http://oeis.org/A000108">A000108</a>/a000108_10.pdf">Determinants whose elements are Eulerian, prepared Bernoullian and other numbers</a>, J. Indian Math. Soc., 14 (1922), 55-62, 122-138 and 143-146. [Annotated scanned copy]
   - Nate Kube and Frank Ruskey, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL8/Ruskey/ruskey99.html">Sequences That Satisfy a(n-a(n))=0</a>, Journal of Integer Sequences, Vol. 8 (2005), Article 05.5.5.
   - W. Lang, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL3/LANG/lang.html">On generalizations of Stirling number triangles</a>, J. Integer Seqs., Vol. 3 (2000), #00.2.4.
   - J. W. Layman, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL4/LAYMAN/hankel.html">The Hankel Transform and Some of its Properties</a>, J. Integer Sequences, 4 (2001), #01.1.5.
   - Pierre Lescanne, <a href="http://arxiv.org/abs/1312.4917">An exercise on streams: convergence acceleration</a>, arXiv preprint arXiv:1312.4917 [cs.NA], 2013.
   - Hsueh-Yung Lin, <a href="http://arxiv.org/abs/1012.1756">The odd Catalan numbers modulo 2^k</a>, arXiv:1012.1756 [math.NT], 2010-2011.
   - R. P. Loh, A. G. Shannon, A. F. Horadam, <a href="/<a href="http://oeis.org/A000969">A000969</a>/a000969.pdf">Divisibility Criteria and Sequence Generators Associated with Fermat Coefficients</a>, Preprint, 1980.
   - Peter Luschny, <a href="http://oeis.org/wiki/User:Peter_Luschny/TheLostCatalanNumbers">The Lost Catalan Numbers And The Schröder Tableaux</a>
   - Sara Madariaga, <a href="http://arxiv.org/abs/1304.5184">Gröbner-Shirshov bases for the non-symmetric operads of dendriform algebras and quadri-algebras</a>, arXiv:1304.5184 [math.RA], 2013.
   - Colin L. Mallows and Lou Shapiro, <a href="http://www.cs.uwaterloo.ca/journals/JIS/MALLOWS/mallows.html">Balls on the Lawn</a>, J. Integer Sequences, Vol. 2, 1999, #5.
   - K Manes, A Sapounakis, I Tasoulas, P Tsikouras, <a href="http://arxiv.org/abs/1510.01952">Equivalence classes of ballot paths modulo strings of length 2 and 3</a>, arXiv preprint arXiv:1510.01952, 2015
   - Toufik Mansour, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL5/Mansour/mansour6.html">Counting Peaks at Height k in a Dyck Path</a>, Journal of Integer Sequences, Vol. 5 (2002), Article 02.1.1
   - Toufik Mansour, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL9/Mansour/mansour86.html">Statistics on Dyck Paths</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.1.5.
   - Toufik Mansour and Mark Shattuck, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL15/Shattuck/shattuck5.html">Counting Dyck Paths According to the Maximum Distance Between Peaks and Valleys</a>, Journal of Integer Sequences, Vol. 15 (2012), #12.1.1.
   - Toufik Mansour and Yidong Sun, <a href="http://arxiv.org/abs/0805.1274">Identities involving Narayana polynomials and Catalan numbers</a> (2008), arXiv:0805.1274 [math.CO]; Discrete Mathematics, Volume 309, Issue 12, Jun 28 2009, Pages 4079-4088
   - R. J. Marsh and P. P. Martin, <a href="http://arXiv.org/abs/math.CO/0612572">Pascal arrays: counting Catalan sets</a>, arXiv:math/0612572 [math.CO], 2006.
   - Math Forum Discussions, <a href="http://mathforum.org/kb/message.jspa?messageID=22219">The Meanings of Catalan Numbers</a>
   - Jon McCammond, <a href="http://arxiv.org/abs/math/0601687">Noncrossing partitions in surprising locations</a>, arXiv:math/0601687 [math.CO], (27-January-2006)
   - D. Merlini, R. Sprugnoli and M. C. Verri, <a href="http://www.dsi.unifi.it/~merlini/fun01.ps">Waiting patterns for a printer</a>, FUN with algorithm'01, Isola d'Elba, 2001.
   - Sam Miner and I. Pak, <a href="http://www.math.ucla.edu/~pak/papers/PermShape7.pdf">The shape of random pattern avoiding permutations</a>, 2013.
   - Marni Mishna and Lily Yen, <a href="http://arxiv.org/abs/1106.5036">Set partitions with no k-nesting</a>, arXiv:1106.5036 [math.CO], 2011.
   - T. Motzkin, <a href="http://dx.doi.org/10.1090/S0002-9904-1945-08486-9">The hypersurface cross ratio</a>, Bull. Amer. Math. Soc., 51 (1945), 976-984.
   - T. S. Motzkin, <a href="http://dx.doi.org/10.1090/S0002-9904-1948-09002-4 ">Relations between hypersurface cross ratios and a combinatorial formula for partitions of a polygon, for permanent preponderance and for non-associative products</a>, Bull. Amer. Math. Soc., 54 (1948), 352-360.
   - Torsten Muetze and Franziska Weber, <a href="http://arxiv.org/abs/1111.2413">Construction of 2-factors in the middle layer of the discrete cube</a>, arXiv preprint arXiv:1111.2413 [math.CO], 2011.
   - Liviu I. Nicolaescu, <a href="http://arxiv.org/abs/math/0512496">Counting Morse functions on the 2-sphere</a>, arXiv:math/0512496 [math.GT], 2005-2006.
   - J.-C. Novelli and J.-Y. Thibon, <a href="http://arXiv.org/abs/math.CO/0405597">Free quasi-symmetric functions of arbitrary level</a>, arXiv:math/0405597 [math.CO], 2004.
   - R. J. Nowakowski, G. Renault, E. Lamoureux, S. Mellon and T. Miller, <a href="http://www.labri.fr/perso/grenault/NRLMM.pdf">The Game of timber!</a>, 2013.
   - C. D. Olds (Proposer) and H. W. Becker (Discussion), <a href="/<a href="http://oeis.org/A000108">A000108</a>/a000108_6.pdf">Problem 4277</a>, Amer. Math. Monthly 56 (1949), 697-699. [Annotated scanned copy]
   - Igor Pak, <a href="http://www.math.ucla.edu/~pak/lectures/Cat/pakcat.htm">Catalan Numbers Page</a>
   - Igor Pak, <a href="http://igorpak.wordpress.com/2014/02/05/who-named-catalan-numbers/?">Who Named the Catalan Numbers?</a>
   - Igor Pak, <a href="http://arxiv.org/abs/1408.5711">History of Catalan numbers</a>, arXiv:1408.5711 [math.HO], 2014.
   - Hao Pan and Zhi-Wei Sun, <a href="http://arxiv.org/abs/math.CO/0509648">A combinatorial identity with application to Catalan numbers</a>, arXiv:math/0509648 [math.CO], 2005-2006.
   - A. Panayotopoulos and P. Tsikouras, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL7/Panayotopoulos/panayo4.html">Meanders and Motzkin Words</a>, J. Integer Seqs., Vol. 7, 2004.
   - A. Panholzer and H. Prodinger, <a href="http://doi.org/10.1016/S0012-365X(01)00282-5">Bijections for ternary trees and non-crossing trees</a>, Discrete Math., 250 (2002), 181-195 (see Eq. 4).
   - A. Papoulis, <a href="/<a href="http://oeis.org/A000108">A000108</a>/a000108_8.pdf">A new method of inversion of the Laplace transform</a>, Quart. Appl. Math 14 (1957), 405-414. [Annotated scan of selected pages]
   - Robert Parviainen, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL9/Parviainen/parviainen3.html">Lattice Path Enumeration of Permutations with k Occurrences of the Pattern 2-13</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.3.2.
   - P. Peart and W.-J. Woan, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL3/PEART/peart1.html">Generating Functions via Hankel and Stieltjes Matrices</a>, J. Integer Seqs., Vol. 3 (2000), #00.2.1.
   - P. Peart and W.-J. Woan, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL4/PEART/pwatjis2.html">Dyck Paths With No Peaks at Height k</a>, J. Integer Sequences, 4 (2001), #01.1.3.
   - Robin Pemantle and Mark C. Wilson, <a href="http://dx.doi.org/10.1137/050643866">Twenty Combinatorial Examples of Asymptotics Derived from Multivariate Generating Functions</a>, SIAM Rev., 50 (2) (2008), 199-272.
   - K. A. Penson and J.-M. Sixdeniers, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL4/SIXDENIERS/Catalan.html">Integral Representations of Catalan and Related Numbers</a>, J. Integer Sequences, 4 (2001), #01.2.5.
   - Karol A. Penson and Karol Zyczkowski, <a href="http://dx.doi.org/10.1103/PhysRevE.83.061118">Product of Ginibre matrices : Fuss-Catalan and Raney distribution</a>, <a href="http://arxiv.org/abs/1103.3453/">arXiv version</a>; Phys. Rev E. vol. 83, 061118 (2011).
   - T. K. Petersen and Bridget Eileen Tenner, <a href="http://arxiv.org/abs/1202.4765">The depth of a permutation</a>, arXiv:1202.4765 [math.CO], 2012-2014.
   - Ville H. Pettersson, <a href="http://www.combinatorics.org/ojs/index.php/eljc/article/view/v21i4p7">Enumerating Hamiltonian Cycles</a>, The Electronic Journal of Combinatorics, Volume 21, Issue 4, 2014.
   - Vincent Pilaud, <a href="http://arxiv.org/abs/1505.07665">Brick polytopes, lattice quotients, and Hopf algebras</a>, arXiv preprint arXiv:1505.07665 [math.CO], 2015.
   - Alexander Postnikov, <a href="http://arxiv.org/abs/math/0507163">Permutohedra, associahedra, and beyond</a>, 2005, arXiv:math/0507163  [math.CO], 2005.
   - J.-B. Priez, A. Virmaux, <a href="http://arxiv.org/abs/1411.4161">Non-commutative Frobenius characteristic of generalized parking functions: Application to enumeration</a>, arXiv preprint arXiv:1411.4161 [math.CO], 2014-2015.
   - L. Pudwell, A. Baxter, <a href="http://faculty.valpo.edu/lpudwell/slides/pp2014_pudwell.pdf">Ascent sequences avoiding pairs of patterns</a>, 2014.
   - Alon Regev, <a href="http://arxiv.org/abs/1208.3915">Enumerating Triangulations by Parallel Diagonals</a>, Journal of Integer Sequences, Vol. 15 (2012), #12.8.5; arXiv preprint arXiv:1208.3915, 2012.
   - Alon Regev, Amitai Regev, Doron Zeilberger, <a href="http://arxiv.org/abs/1507.03499">Identities in character tables of S_n</a>, arXiv preprint arXiv:1507.03499, 2015
   - C. M. Ringel, <a href="http://arxiv.org/abs/1502.06553">The Catalan combinatorics of the hereditary artin algebras</a>, arXiv preprint arXiv:1502.06553, 2015
   - J. Riordan, <a href="http://www.jstor.org/stable/2005477">The distribution of crossings of chords joining pairs of 2n points on a circle</a>, Math. Comp., 29 (1975), 215-222.
   - E. Rowland and R. Yassawi, <a href="http://arxiv.org/abs/1310.8635">Automatic congruences for diagonals of rational functions</a>, arXiv preprint arXiv:1310.8635 [math.NT], 2013-2014.
   - E. Rowland and D. Zeilberger, <a href="http://arxiv.org/abs/1311.4776">A Case Study in Meta-AUTOMATION: AUTOMATIC Generation of Congruence AUTOMATA For Combinatorial Sequences</a>, arXiv preprint arXiv:1311.4776 [math.CO], 2013.
   - Albert Sade, <a href="/<a href="http://oeis.org/A000108">A000108</a>/a000108_17.pdf">Sur les Chevauchements des Permutations</a>, published by the author, Marseille, 1949. [Annotated scanned copy]
   - A. Sapounakis, I. Tasoulas and P. Tsikouras, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL9/Tsikouras/tsikouras67.html">On the Dominance Partial Ordering of Dyck Paths</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.2.5.
   - A. Sapounakis and P. Tsikouras, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL7/Tsikouras/tsikouras43.html">On k-colored Motzkin words</a>, Journal of Integer Sequences, Vol. 7 (2004), Article 04.2.5.
   - E. Schröder, <a href="/<a href="http://oeis.org/A000108">A000108</a>/a000108_9.pdf">Vier combinatorische Probleme</a>, Z. f. Math. Phys., 15 (1870), 361-376. [Annotated scanned copy]
   - A. Schuetz and G. Whieldon, <a href="http://arxiv.org/abs/1401.7194">Polygonal Dissections and Reversions of Series</a>, arXiv preprint arXiv:1401.7194 [math.CO], 2014.
   - Sarah Shader, <a href="http://math.mit.edu/news/summer/RSIPapers/2013Shader.pdf">Weighted Catalan Numbers and Their Divisibility Properties</a>, Research Science Institute, MIT, 2014.
   - D. M. Silberger, <a href="/<a href="http://oeis.org/A000108">A000108</a>/a000108_12.pdf">Occurrences of the integer (2n-2)!/n!(n-1)!</a>, Roczniki Polskiego Towarzystwa Math. 13 (1969): 91-96. [Annotated scanned copy]
   - N. J. A. Sloane, <a href="/<a href="http://oeis.org/A000108">A000108</a>/a000108.html">Illustration of initial terms</a>
   - N. J. A. Sloane, <a href="/<a href="http://oeis.org/A000108">A000108</a>/a000108_16.pdf">Note on Sylvester's "On reducible cyclodes" paper</a> [Scanned copy]
   - N. Solomon and S. Solomon, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL11/Solomon/solomon8rev.html">A natural extension of Catalan Numbers</a>, JIS 11 (2008) 08.3.5
   - Frank Sottile, <a href="http://www.math.tamu.edu/~sottile/research/pages/ERAG/S4/1.html">The Schubert Calculus of Lines</a> (a section of Enumerative Real Algebraic Geometry)
   - Michael Z. Spivey and Laura L. Steil, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL9/Spivey/spivey7.html">The k-Binomial Transforms and the Hankel Transform</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.1.1.
   - R. P. Stanley, <a href="http://www-math.mit.edu/~rstan/papers.html">Hipparchus, Plutarch, Schröder and Hough</a>, Am. Math. Monthly, Vol. 104, No. 4, p. 344, 1997.
   - R. P. Stanley, <a href="http://www-math.mit.edu/~rstan/ec/catalan.pdf">Exercises on Catalan and Related Numbers</a>
   - R. P. Stanley, <a href="http://www-math.mit.edu/~rstan/ec#catadd">Catalan Addendum</a>
   - R. P. Stanley, <a href="/<a href="http://oeis.org/A000108">A000108</a>/a000108.pdf">Interpretations of Catalan Numbers (Notes)</a> [Annotated scanned copy]
   - Zhi-Wei Sun and Roberto Tauraso, <a href="http://arxiv.org/abs/0709.1665">On some new congruences for binomial coefficients</a>, arXiv:0709.1665 [math.NT], 2007-2011.
   - V. S. Sunder, <a href="http://www.imsc.res.in/~sunder/catalan.pdf">Catalan numbers</a>
   - P. Tarau, <a href="http://logic.cse.unt.edu/tarau/research/2013/catco.pdf">Computing with Catalan Families</a>, 2013.
   - P. Tarau, <a href="http://arxiv.org/abs/1406.1796">A Generic Numbering System based on Catalan Families of Combinatorial Objects</a>, arXiv preprint arXiv:1406.1796 [cs.MS], 2014.
   - P. Tarau, <a href="http://arxiv.org/abs/1507.06944">A Logic Programming Playground for Lambda Terms, Combinators, Types and Tree-based Arithmetic Computations</a>, arXiv preprint arXiv:1507.06944, 2015
   - D. Taylor, <a href="http://www.maths.usyd.edu.au/u/don/code/Catalan/Catalan.html">Catalan Structures(up to C(7))</a>
   - J.-D. Urbina, J. Kuipers, Q. Hummel, K. Richter, <a href="http://arxiv.org/abs/1409.1558">Multiparticle correlations in complex scattering and the mesoscopic Boson Sampling problem</a>, arXiv preprint arXiv:1409.1558 [quant-ph], 2014.
   - A. Vieru, <a href="http://arxiv.org/abs/1107.2938">Agoh's conjecture: its proof, its generalizations, its analogues</a>, arXiv:1107.2938 [math.NT], 2011.
   - Gérard Villemin, <a href="http://villemin.gerard.free.fr/aNombre/TYPDENOM/Catalan/Catalan.htm">Nombres De Catalan</a> (French)
   - D. W. Walkup, <a href="http://dx.doi.org/10.1112/S0025579300005659">The number of plane trees</a>, Mathematika, vol. 19, No. 2 (1972), 200-204.
   - Eric Weisstein's World of Mathematics, <a href="http://mathworld.wolfram.com/CatalanNumber.html">Catalan Number</a>, <a href="http://mathworld.wolfram.com/BinaryBracketing.html">Binary Bracketing</a>, <a href="http://mathworld.wolfram.com/BinaryTree.html">Binary Tree</a>, <a href="http://mathworld.wolfram.com/NonassociativeProduct.html">Nonassociative Product</a>, <a href="http://mathworld.wolfram.com/StaircaseWalk.html">Staircase Walk</a>, and <a href="http://mathworld.wolfram.com/DyckPath.html">Dyck Path</a>
   - J. Winter, M. M. Bonsangue and J. J. M. M. Rutten, <a href="http://oai.cwi.nl/oai/asset/21313/21313A.pdf ">Context-free coalgebras</a>, 2013
   - Roman Witula, Damian Slota and Edyta Hetmaniok, <a href="http://ami.ektf.hu/uploads/papers/finalpdf/AMI_41_from255to263.pdf">Bridges between different known integer sequences</a>, Annales Mathematicae et Informaticae, 41 (2013) pp. 255-263.
   - W.-J. Woan, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL4/WOAN/hankel2.html">Hankel Matrices and Lattice Paths</a>, J. Integer Sequences, 4 (2001), #01.1.2.
   - Wen-jin Woan, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL8/Woan/woan11.html">A Recursive Relation for Weighted Motzkin Sequences</a> Journal of Integer Sequences, Vol. 8 (2005), Article 05.1.6.
   - Wen-jin Woan, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL8/Woan2/woan35.html">Animals and 2-Motzkin Paths</a>, Journal of Integer Sequences, Vol. 8 (2005), Article 05.5.6.
   - Wen-jin Woan, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL9/Woan/woan206.html">A Relation Between Restricted and Unrestricted Weighted Motzkin Paths</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.1.7.
   - F. Yano and H. Yoshida, <a href="http://dx.doi.org/10.1016/j.disc.2007.03.050">Some set partition statistics in non-crossing partitions and generating functions</a>, Discr. Math., 307 (2007), 3147-3160.
   - Yan X Zhang, <a href="http://arxiv.org/abs/1508.00318">Four Variations on Graded Posets</a>, arXiv preprint arXiv:1508.00318, 2015
   - <a href="/index/Cor#core">Index entries for "core" sequences</a>
   - <a href="/index/Ne#necklaces">Index entries for sequences related to necklaces</a>
   - <a href="/index/Par#parens">Index entries for sequences related to parenthesizing</a>
   - <a href="/index/Ro#rooted">Index entries for sequences related to rooted trees</a>

_References_:
   - The large number of references and links demonstrates the ubiquity of the Catalan numbers.
   - Abrate, Marco; Barbero, Stefano; Cerruti, Umberto; Murru, Nadir. Colored compositions, Invert operator and elegant compositions with the "black tie". Discrete Math. 335 (2014), 1-7. MR3248794
   - M. Aigner, Enumeration via ballot numbers, Discrete Math., 308 (2008), 2544-2563.
   - R. Alter, Some remarks and results on Catalan numbers, pp. 109-132 in Proceedings of the Louisiana Conference on Combinatorics, Graph Theory and Computer Science. Vol. 2, edited R. C. Mullin et al., 1971.
   - R. Alter and K. K. Kubota, Prime and prime power divisibility of Catalan numbers, J. Combinatorial Theory, Ser. A, 15 (1973), 243-256.
   - Andrews, George E. Catalan numbers, q-Catalan numbers and hypergeometric series. J. Combin. Theory Ser. A 44 (1987), no. 2, 267-273. MR0879684 (88f:05015)
   - R. Bacher and C. Krattenthaler, Chromatic statistics for triangulations and FussCatalan complexes, Electronic Journal of Combinatorics, 18 (2011), #P152.
   - D. F. Bailey, Counting Arrangements of 1's and -1's, Mathematics Magazine 69(2) 128-131 1996.
   - I. Bajunaid et al., Function series, Catalan numbers and random walks on trees, Amer. Math. Monthly 112 (2005), 765-785.
   - Barcucci, E.; Del Lungo, A.; Pergola, E.; and Pinzani, R.; Some permutations with forbidden subsequences and their inversion number. Discrete Math. 234 (2001), no. 1-3, 1-15.
   - E. Barcucci, A. Frosini and S. Rinaldi, On directed-convex polyominoes in a rectangle, Discr. Math., 298 (2005). 62-78.
   - Baril, Jean-Luc; Petrossian, Armen. Equivalence classes of Dyck paths modulo some statistics. Discrete Math. 338 (2015), no. 4, 655-660. MR3300754
   - P Barry, Riordan arrays, generalized Narayana triangles, and series reversion, Linear Algebra and its Applications, 491 (2016) 343-385.
   - E. T. Bell, The iterated exponential numbers, Ann. Math., 39 (1938), 539-557.
   - F. Bergeron, G. Labelle and P. Leroux, Combinatorial Species and Tree-like Structures, EMA vol.67, Cambridge, 1998, p. 163, 167, 168, 252, 256, 291.
   - E. E. Bernard and P. D. A. Mole, Generating strategies for continuous separation processes, Computer J., 2 (1959), 87-89.
   - F. R. Bernhart, Catalan, Motzkin and Riordan numbers, Discr. Math., 204 (1999) 73-112.
   - A. Bernini, F. Disanto, R. Pinzani and S. Rinaldi, Permutations defining convex permutominoes, J. Int. Seq. 10 (2007) # 07.9.7
   - M. Bona and B. E. Sagan, On Divisibility of Narayana Numbers by Primes, Journal of Integer Sequences, Vol. 8 (2005), Article 05.2.4.
   - Michel Bousquet and Cedric Lamathe, On symmetric structures of order two. Discrete Math. Theor. Comput. Sci. 10 (2008), 153-176.
   - W. G. Brown, Historical note on a recurrent combinatorial problem, Amer. Math. Monthly, 72 (1965), 973-977.
   - D. Callan, A combinatorial interpretation of a Catalan numbers identity, Math. Mag., 72 (1999), 295-298.
   - D. Callan, The maximum associativeness of division, Problem 11091, Amer. Math. Monthly, 113 (#5, 2006), 462-463.
   - D. Callan, Pattern avoidance in "flattened" partitions, Discrete Math., 309 (2009), 4187-4191.
   - Peter J. Cameron, Some treelike objects. Quart. J. Math. Oxford Ser. (2) 38 (1987), no. 150, 155-183. MR0891613 (89a:05009). See p. 155.
   - Young-Ming Chen, The Chung-Feller theorem revisited. Discrete Math. 308 (2008), no. 7, 1328-1329. MR2382368 (2008j:05019)
   - Kai Lai Chung and W. Feller, On fluctuations in coin-tossing. Proc. Nat. Acad. Sci. U. S. A. 35, (1949). 605-608.
   - L. Comtet, Advanced Combinatorics, Reidel, 1974, p. 53.
   - J. H. Conway and R. K. Guy, The Book of Numbers, New York: Springer-Verlag, 1995, ch. 4, pp. 96-106.
   - Harry Crane, Left-right arrangements, set partitions, and pattern avoidance. Australasian Journal of Combinatorics, 61(1) (2105), 57-72.
   - S. J. Cyvin and I. Gutman, Kekulé structures in benzenoid hydrocarbons, Lecture Notes in Chemistry, No. 46, Springer, New York, 1988 (see pp. 183, 196, etc.).
   - Michael Dairyko, Samantha Tyner, Lara Pudwell, and Casey Wynn, Non-contiguous pattern avoidance in binary trees. Electron. J. Combin. 19 (2012), no. 3, Paper 22, 21 pp. MR2967227. - From _N. J. A. Sloane_, Feb 01 2013
   - E. Deutsch, Dyck path enumeration, Discrete Math., 204, 167-202, 1999.
   - E. Deutsch and L. Shapiro, A survey of the Fine numbers, Discrete Math., 241 (2001), 241-265.
   - E. Deutsch and L. Shapiro, Seventeen Catalan identities, Bulletin of the Institute of Combinatorics and its Applications, 31, 31-38, 2001.
   - Tomislav Doslic and Darko Veljan, Logarithmic behavior of some combinatorial sequences. Discrete Math. 308 (2008), no. 11, 2182-2212. MR2404544 (2009j:05019)
   - S. Dulucq and J.-G. Penaud, Cordes, arbres et permutations. Discrete Math. 117 (1993), no. 1-3, 89-105.
   - Roger B. Eggleton and Richard K. Guy, "Catalan strikes again! How likely is a function to be convex?" Mathematics Magazine, 61 (1988): 211-219.
   - A. Errera, Analysis situs - Un problème d'énumération, Mémoires Acad. Bruxelles, Classe des sciences, Série 2, Vol. XI, Fasc. 6, No. 1421 (1931), 26 pp.
   - Ehrenfeucht, Andrzej; Haemer, Jeffrey; Haussler, David. Quasimonotonic sequences: theory, algorithms and applications. SIAM J. Algebraic Discrete Methods 8 (1987), no. 3, 410-429. MR0897739 (88h:06026)
   - I. M. H. Etherington, Non-associate powers and a functional equation. The Mathematical Gazette, 21 (1937): 36-39; addendum 21 (1937), 153.
   - I. M. H. Etherington, On non-associative combinations, Proc. Royal Soc. Edinburgh, 59 (Part 2, 1938-39), 153-162.
   - I. M. H. Etherington, Some problems of non-associative combinations (I), Edinburgh Math. Notes, 32 (1940), pp. i-vi. Part II is by A. Erdelyi and I. M. H. Etherington, and is on pages vii-xiv of the same issue.
   - K. Fan, Structure of a Hecke algebra quotient, J. Amer. Math. Soc., 10 (1997), 139-167.
   - Susanna Fishel, Myrto Kallipoliti and Eleni Tzanaki, Facets of the Generalized Cluster Complex and Regions in the Extended Catalan Arrangement of Type A, The electronic Journal of Combinatorics 20(4) (2013), #P7.
   - Flajolet, Philippe; Gourdon, Xavier; and Dumas, Philippe; Mellin transforms and asymptotics: harmonic sums. Special volume on mathematical analysis of algorithms. Theoret. Comput. Sci. 144 (1995), no. 1-2, 3-58.
   - D. Foata and D. Zeilberger, A classic proof of a recurrence for a very classical sequence, J. Comb Thy A 80 380-384 1997.
   - H. G. Forder, Some problems in combinatorics, Math. Gazette, vol. 45, 1961, 199-201.
   - Fürlinger, J.; Hofbauer, J.,  q-Catalan numbers. J. Combin. Theory Ser. A 40 (1985), no. 2, 248-264. MR0814413 (87e:05017)
   - J. R. Gaggins, Constructing the Centroid of a Polygon, Math. Gaz., 61 (1988), 211-212.
   - M. Gardner, Time Travel and Other Mathematical Bewilderments, Chap. 20 pp. 253-266, W. H. Freeman NY 1988.
   - S. Gilliand, C. Johnson, S. Rush, D. Wood, The sock matching problem, Involve, a Journal of Mathematics, Vol. 7 (2014), No. 5, 691-697; DOI: 10.2140/involve.2014.7.691
   - James Gleick, Faster, Vintage Books, NY, 2000 (see pp. 259-261).
   - Lisa R. Goldberg, Catalan numbers and branched coverings by the Riemann sphere, Adv. Math. 85 (1991), No. 2, 129-144.
   - S Goodenough, C Lavault, Overview on Heisenberg—Weyl Algebra and Subsets of Riordan Subgroups, The Electronic Journal of Combinatorics, 22(4) (2015), #P4.16,
   - H. W. Gould, Research bibliography of two special number sequences, Mathematica Monongaliae, Vol. 12, 1971.
   - Alain Goupil and Gilles Schaeffer, Factoring N-Cycles and Counting Maps of Given Genus. Europ. J. Combinatorics (1998) 19 819-834.
   - D. Gouyou-Beauchamps, Chemins sous-diagonaux et tableau de Young, pp. 112-125 of "Combinatoire Enumerative (Montreal 1985)", Lect. Notes Math. 1234, 1986.
   - M. Griffiths, The Backbone of Pascal's Triangle, United Kingdom Mathematics Trust (2008), 53-63 and 85-93.
   - J. L. Gross and J. Yellen, eds., Handbook of Graph Theory, CRC Press, 2004; p. 530.
   - N. S. S. Gu, N. Y. Li and T. Mansour, 2-Binary trees: bijections and related issues, Discr. Math., 308 (2008), 1209-1221.
   - R. K. Guy, Dissecting a polygon into triangles, Research Paper #9, Math. Dept., Univ. Calgary, 1967.
   - R. K. Guy and J. L. Selfridge, The nesting and roosting habits of the laddered parenthesis. Amer. Math. Monthly 80 (1973), 868-876.
   - Peter Hajnal and Gabor V. Nagy, A bijective proof of Shapiro's Catalan convolution, Elect. J. Combin., 21 (2014), #P2.42.
   - F. Harary and E. M. Palmer, Graphical Enumeration, Academic Press, NY, 1973, p. 67, (3.3.23).
   - F. Harary, G. Prins, and W. T. Tutte, The number of plane trees. Indag. Math. 26, 319-327, 1964.
   - J. Harris, Algebraic Geometry: A First Course (GTM 133), Springer-Verlag, 1992, pages 245-247.
   - S. Heubach, N. Y. Li and T. Mansour, Staircase tilings and k-Catalan structures, Discrete Math., 308 (2008), 5954-5964.
   - Silvia Heubach and Toufik Mansour, Combinatorics of Compositions and Words, CRC Press, 2010.
   - Higgins, Peter M. Combinatorial results for semigroups of order-preserving mappings. Math. Proc. Camb. Phil. Soc. (1993), 113: 281-296.
   - B. D. Hughes, Random Walks and Random Environments, Oxford 1995, vol. 1, p. 513, Eq. (7.282).
   - F. Hurtado, M. Noy, Ears of triangulations and Catalan numbers, Discrete Mathematics, Volume 149, Issues 1-3, Feb 22 1996, Pages 319-324.
   - M. Janjic, Determinants and Recurrence Sequences, Journal of Integer Sequences, 2012, Article 12.3.5. - From _N. J. A. Sloane_, Sep 16 2012
   - R. H. Jeurissen, Raney and Catalan, Discrete Math., 308 (2008), 6298-6307.
   - Nicholas M. Katz, A NOTE ON RANDOM MATRIX INTEGRALS, MOMENT IDENTITIES, AND CATALAN NUMBERS, 2015; https://web.math.princeton.edu/~nmk/catalan11.pdf
   - M. Kauers and P. Paule, The Concrete Tetrahedron, Springer 2011, p. 36.
   - Kim, Ki Hang; Rogers, Douglas G.; Roush, Fred W. Similarity relations and semiorders. Proceedings of the Tenth Southeastern Conference on Combinatorics, Graph Theory and Computing (Florida Atlantic Univ., Boca Raton, Fla., 1979), pp. 577-594, Congress. Numer., XXIII-XXIV, Utilitas Math., Winnipeg, Man., 1979. MR0561081 (81i:05013)
   - Klarner, D. A. A Correspondence Between Sets of Trees. Indag. Math. 31, 292-296, 1969.
   - M. Klazar, On numbers of Davenport-Schinzel sequences, Discr. Math., 185 (1998), 77-87.
   - D. E. Knuth, The Art of Computer Programming, vol. 4A, Combinatorial Algorithms, Section 7.2.1.6 (p. 450).
   - Thomas Koshy and Mohammad Salmassi, "Parity and Primality of Catalan Numbers", College Mathematics Journal, Vol. 37, No. 1 (Jan 2006), pp. 52-53.
   - M. Kosters, A theory of hexaflexagons, Nieuw Archief Wisk., 17 (1999), 349-362.
   - E. Krasko, A. Omelchenko, Brown's Theorem and its Application for Enumeration of Dissections and Planar Trees, The Electronic Journal of Combinatorics, 22 (2105), #P1.17.
   - Kreweras, G. Sur les partitions non croisees d'un cycle. (French) Discrete Math. 1 (1972), no. 4, 333-350. MR0309747 (46 #8852)
   - C. Krishnamachary and M. Bheemasena Rao, Determinants whose elements are Eulerian, prepared Bernoullian and other numbers, J. Indian Math. Soc., 14 (1922), 55-62, 122-138 and 143-146.
   - P. Lafar and C. T. Long, A combinatorial problem, Amer. Math. Mnthly, 69 (1962), 876-883.
   - Laradji, A. and Umar, A. On certain finite semigroups of order-decreasing transformations I, Semigroup Forum 69 (2004), 184-200.
   - P. J. Larcombe, On pre-Catalan Catalan numbers: Kotelnikow (1766), Mathematics Today, 35 (1999), p. 25.
   - P. J. Larcombe, On the history of the Catalan numbers: a first record in China, Mathematics Today, 35 (1999), p. 89.
   - P. J. Larcombe, The 18th century Chinese discovery of the Catalan numbers, Math. Spectrum, 32 (1999/2000), 5-7.
   - P. J. Larcombe and P. D. C. Wilson, On the trail of the Catalan sequence, Mathematics Today, 34 (1998), 114-117.
   - P. J. Larcombe and P. D. C. Wilson, On the generating function of the Catalan sequence: a historical perspective, Congress. Numer., 149 (2001), 97-108.
   - P. J. Larcombe et al., On certain series expansions of the sine function: Catalan numbers and convergence, Fib. Q., 52 (2014), 236-242.
   - G. S. Lueker, Some techniques for solving recurrences, Computing Surveys, 12 (1980), 419-436.
   - J. J. Luo, Antu Ming, the first inventor of Catalan numbers in the world [in Chinese], Neimenggu Daxue Xuebao, 19 (1998), 239-245.
   - C. L. Mallows, R. J. Vanderbei, Which Young Tableaux Can Represent an Outer Sum?, Journal of Integer Sequences, Vol. 18, 2015, #15.9.1.
   - Toufik Mansour, Matthias Schork, and Mark Shattuck, Catalan numbers and pattern restricted set partitions. Discrete Math. 312(2012), no. 20, 2979-2991. MR2956089
   - Toufik Mansour and Simone Severini, Enumeration of (k,2)-noncrossing partitions, Discrete Math., 308 (2008), 4570-4577.
   - M. E. Mays and Jerzy Wojciechowski, A determinant property of Catalan numbers. Discrete Math. 211, No. 1-3, 125-133 (2000). Zbl 0945.05037
   - D. Merlini, R. Sprugnoli and M. C. Verri, The tennis ball problem, J. Combin. Theory, A 99 (2002), 307-344.
   - A. Milicevic and N. Trinajstic, "Combinatorial Enumeration in Chemistry", Chem. Modell., Vol. 4, (2006), pp. 405-469.
   - C. O. Oakley and R. J. Wisner, Flexagons, Amer. Math. Monthly, 64 (1957), 143-154.
   - A. Panholzer and H. Prodinger, Bijections for ternary trees and non-crossing trees, Discrete Math., 250 (2002), 181-195 (see Eq. 4).
   - Papoulis, Athanasios. "A new method of inversion of the Laplace transform."Quart. Appl. Math 14.405-414 (1957): 124.
   - S. G. Penrice, Stacks, bracketings and CG-arrangements, Math. Mag., 72 (1999), 321-324.
   - C. A. Pickover, Wonders of Numbers, Chap. 71, Oxford Univ. Press NY 2000.
   - Clifford A. Pickover, A Passion for Mathematics, Wiley, 2005; see p. 71.
   - G. Pólya, On the number of certain lattice polygons. J. Combinatorial Theory 6 1969 102-105. MR0236031 (38 #4329)
   - C. Pomerance, Divisors of the middle binomial coefficient, Amer. Math. Monthly, 112 (2015), 636-644.
   - Jocelyn Quaintance and Harris Kwong, A combinatorial interpretation of the Catalan and Bell number difference tables, Integers, 13 (2013), #A29.
   - R. Read, "Counting Binary Trees" in 'The Mathematical Gardner', D. A. Klarner Ed. pp. 331-334, Wadsworth CA 1989.
   - Amitai Regev, Nathaniel Shar, and Doron Zeilberger, A Very Short (Bijective!) Proof of Touchard's Catalan Identity, 2015; http://www.math.rutgers.edu/~zeilberg/mamarim/mamarimhtml/touchard.html
   - J.-L. Remy, Un procede iteratif de denombrement d'arbres binaires et son application a leur generation aleatoire, RAIRO Inform. Theor. 19 (1985), 179-195.
   - J. Riordan, Combinatorial Identities, Wiley, 1968, p. 101.
   - J. Riordan, The distribution of crossings of chords joining pairs of 2n points on a circle, Math. Comp., 29 (1975), 215-222.
   - T. Santiago Costa Oliveira, "Catalan traffic" and integrals on the Grassmannian of lines, Discr. Math., 308 (2007), 148-152.
   - A. Sapounakis, I. Tasoulas and P. Tsikouras, Counting strings in Dyck paths, Discrete Math., 307 (2007), 2909-2924.
   - E. Schröder, Vier combinatorische Probleme, Z. f. Math. Phys., 15 (1870), 361-376.
   - Shapiro, Louis W. Catalan numbers and "total information" numbers. Proceedings of the Sixth Southeastern Conference on Combinatorics, Graph Theory, and Computing (Florida Atlantic Univ., Boca Raton, Fla., 1975), pp. 531-539. Congressus Numerantium, No. XIV, Utilitas Math., Winnipeg, Man., 1975. MR0398853 (53 #2704).
   - L. W. Shapiro, A short proof of an identity of Touchard's concerning Catalan numbers, J. Combin. Theory, A 20 (1976), 375-376.
   - L. W. Shapiro and C. J. Wang, Generating identities via 2 X 2 matrices, Congressus Numerantium, 205 (2010), 33-46.
   - L. W. Shapiro, W.-J. Woan and S. Getu, The Catalan numbers via the World Series, Math. Mag., 66 (1993), 20-22.
   - D. M. Silberger, Occurrences of the integer (2n-2)!/n!(n-1)!, Roczniki Polskiego Towarzystwa Math. 13 (1969): 91-96.
   - N. J. A. Sloane, A Handbook of Integer Sequences, Academic Press, 1973 (includes this sequence).
   - N. J. A. Sloane and Simon Plouffe, The Encyclopedia of Integer Sequences, Academic Press, 1995 (includes this sequence).
   - S. Snover and S. Troyer, Multidimensional Catalan numbers, Abstracts 848-05-94 and 848-05-95, 848th Meeting, Amer. Math. Soc., Worcester Mass., March 15-16, 1989.
   - Solomon, A. Catalan monoids, monoids of local endomorphisms and their presentations. Semigroup Forum 53 (1996), 351-368.
   - R. P. Stanley, Enumerative Combinatorics, Wadsworth, Vol. 1, 1986, Vol. 2, 1999; see especially Chapter 6.
   - R. P. Stanley, Recent Progress in Algebraic Combinatorics, Bull. Amer. Math. Soc., 40 (2003), 55-68.
   - Richard P. Stanley, "Catalan Numbers",  Cambridge University Press, 2015.
   - T. Stojadinovic The Catalan numbers, Preprint 2015; https://www.researchgate.net/profile/Tanja_Stojadinovic2/publication/281062823_The_Catalan_numbers/links/55d3022008ae7fb244f56e70.pdf
   - J. J. Sylvester, On reducible cyclodes, Coll. Math. Papers, Vol. 2, see especially page 670, where Catalan numbers appear.
   - J. A. von Segner, Enumeratio modorum, quibus figurae planae rectilineae per diagonales dividuntur in triangula, Novi Comm. Acad. Scient. Imper. Petropolitanae, 7 (1758/1759), 203-209.
   - I. Vun and P. Belcher, Catalan numbers, Mathematical Spectrum, 30 (1997/1998), 3-5.
   - D. Wells, Penguin Dictionary of Curious and Interesting Numbers, Entry 42 p 121, Penguin Books, 1987.
   - J. Wuttke, The zig-zag walk with scattering and absorption on the real half line and in a lattice model, J. Phys. A 47 (2014), 215203, 1-9.
<hr><div align='center'><b><a href='http://oeis.org/A120588'>A120588</a></b>: <i>G.f. satisfies: 3*A(x) = 2 + x + A(x)^2, with a(0) = 1.</i><br></div>

by _Paul D. Hanna_, Jun 16 2006, Jan 24 2008

_Keywords_: `nonn,easy,changed`

_Data_:

$$
\begin{array}{c|ccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 \\
\hline
A120588(n) & 1 & 1 & 1 & 2 & 5 & 14 & 42 & 132 & 429 & 1430 & 4862 & 16796 & 58786 & 208012 & 742900
\end{array}
$$


_Comments_:
   - This is essentially a duplicate of entry <a href="http://oeis.org/A000108">A000108</a>, the Catalan numbers (a(n) = <a href="http://oeis.org/A000108">A000108</a>(n-1) for n>0).
   - In order for the g.f. of an integer sequence to satisfy a functional equation of the form: r*A(x) = c + b*x + A(x)^n, where n > 1, it is necessary that the sequence start with [1, d, m*n*(n-1)/2], where d divides m*n*(n-1)/2 (m>0) and that the coefficients are given by r = n + d^2/m, c = r-1 and b = d^3/m. The remaining terms may then be integer and still satisfy: a_n(k) = r*a(k), where a_n(k) is the k-th term of the n-th self-convolution of the sequence.

_Formulae_:
   - G.f.: A(x) = 1 + Series_Reversion(1+3*x - (1+x)^2).
   - Lagrange Inversion yields g.f.: A(x) = Sum_{n>=0} C(2*n,n)/(n+1)*(2+x)^(n+1)/3^(2*n+1).
   - G.f.: (3 - sqrt(1-4*x))/2. - Maksym Voznyy (voznyy(AT)mail.ru), Aug 11 2009
   - a(n) = Sum_{k=1..n-1} a(k) * a(n-k) if n>1. - _Michael Somos_, Jul 23 2011
   - G.f.: 2 - G(0), where G(k)= 2*x*(2*k+1) + k +1 - 2*x*(k+1)*(2*k+3)/G(k+1); (continued fraction). - _Sergei N. Gladkovskii_, Jul 14 2013
   - G.f.: 2 - G(0), where G(k)= 1 - x/G(k+1) ; (continued fraction). - _Sergei N. Gladkovskii_, Jul 19 2013
   - a(n) ~ 2^(2*n-2)/(sqrt(Pi)*n^(3/2)). - _Vaclav Kotesovec_, Aug 19 2013
   - Given g.f. A(x), <a href="http://oeis.org/A001850">A001850</a>(n-1) = coefficient of x^n in A(x)^n if n>0, the derivative of log(A(x)) is the g.f. for <a href="http://oeis.org/A026641">A026641</a>. - _Michael Somos_, May 18 2015
   - A(x) = (1 + 2*Sum_{n >= 1} Catalan(n)*x^n)/(1 + Sum_{n >= 1} Catalan(n)*x^n) = (1 + 3/2*Sum_{n >= 1} binomial(2*n,n)*x^n )/(1 + Sum_{n >= 1} binomial(2*n,n)*x^n). - _Peter Bala_, Sep 01 2016

_Cross references_:
   - Cf. <a href="http://oeis.org/A120589">A120589</a> (A(x)^2); <a href="http://oeis.org/A120590">A120590</a> - <a href="http://oeis.org/A120607">A120607</a>.
   - Cf. <a href="http://oeis.org/A001850">A001850</a>, <a href="http://oeis.org/A026641">A026641</a>.

_Links_:
   - G. C. Greubel, <a href="/<a href="http://oeis.org/A120588">A120588</a>/b120588.txt">Table of n, a(n) for n = 0..1000</a>

<hr><div align='center'><b><a href='http://oeis.org/A211216'>A211216</a></b>: <i>Expansion of (1-8*x+21*x^2-20*x^3+5*x^4)/(1-9*x+28*x^2-35*x^3+15*x^4-x^5).</i><br></div>

by _Bruno Berselli_, May 11 2012

_Keywords_: `nonn,easy`

_Data_:

$$
\begin{array}{c|ccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 \\
\hline
A211216(n) & 1 & 1 & 2 & 5 & 14 & 42 & 132 & 429 & 1430 & 4862 & 16795 & 58766 & 207783 & 740924 & 2660139
\end{array}
$$


_Comments_:
   - In the paper of Kitaev, Remmel and Tiefenbruck (see the Links section), Q_(132)^(k,0,0,0)(x,0) represents a generating function depending on k and x.
   - For successive values ​​of k we have:
   - k=1, the g.f. of <a href="http://oeis.org/A000012">A000012</a>: 1/(1-x);
   - k=2,      "      <a href="http://oeis.org/A011782">A011782</a>: (1-x)/(1-2*x);
   - k=3,      "      <a href="http://oeis.org/A001519">A001519</a>: (1-2*x)/(1-3*x+x^2);
   - k=4,      "      <a href="http://oeis.org/A124302">A124302</a>: (1-3*x+x^2)/(1-4*x+3*x^2);
   - k=5,      "      <a href="http://oeis.org/A080937">A080937</a>: (1-4*x+3*x^2)/(1-5*x+6*x^2-x^3);
   - k=6,      "      <a href="http://oeis.org/A024175">A024175</a>: (1-5*x+6*x^2-x^3)/(1-6*x+10*x^2-4*x^3);
   - k=7,      "      <a href="http://oeis.org/A080938">A080938</a>: (1-6*x+10*x^2-4*x^3)/(1-7*x+15*x^2-10*x^3+x^4);
   - k=8,      "      <a href="http://oeis.org/A033191">A033191</a>: (1-7*x+15*x^2-10*x^3+x^4)/(1-8*x+21*x^2

_Formulae_:
   - G.f.: (1-3*x+x^2)*(1-5*x+5*x^2)/(1-9*x+28*x^2-35*x^3+15*x^4-x^5).
   - G.f.: 1/(1-x/(1-x/(1-x/(1-x/(1-x/(1-x/(1-x/(1-x/(1-x))))))))). - _Philippe Deléham_, Mar 14 2013

_Cross references_:
   - Cf. <a href="http://oeis.org/A001519">A001519</a>, <a href="http://oeis.org/A011782">A011782</a>, <a href="http://oeis.org/A024175">A024175</a>, <a href="http://oeis.org/A033191">A033191</a>, <a href="http://oeis.org/A080937">A080937</a>, <a href="http://oeis.org/A080938">A080938</a>, <a href="http://oeis.org/A124302">A124302</a>.
   - Cf. square array in <a href="http://oeis.org/A080934">A080934</a>.

_Links_:
   - Bruno Berselli, <a href="/<a href="http://oeis.org/A211216">A211216</a>/b211216.txt">Table of n, a(n) for n = 0..1000</a>
   - Sergey Kitaev, Jeffrey Remmel and Mark Tiefenbruck, <a href="http://arxiv.org/abs/1201.6243">Marked mesh patterns in 132-avoiding permutations I,</a> arXiv:1201.6243v1 [math.CO], 2012 (page 10, Corollary 3).
   - <a href="/index/Rec">Index entries for linear recurrences with constant coefficients</a>, signature (9,-28,35,-15,1).

_References_:
   - K Mészáros, AH Morales, J Striker, On flow polytopes, order polytopes, and certain faces of the alternating sign matrix polytope, arXiv preprint arXiv:1510.03357, 2015
<hr><div align='center'><b><a href='http://oeis.org/A033191'>A033191</a></b>: <i>Binomial transform of [ 1, 0, 1, 1, 3, 6, 15, 36, 91, 231, 595,... ], which is essentially binomial(fibonacci(k) + 1, 2).</i><br></div>

by Simon Norton (simon(AT)dpmms.cam.ac.uk)

_Keywords_: `nonn,easy`

_Data_:

$$
\begin{array}{c|ccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 \\
\hline
A033191(n) & 1 & 1 & 2 & 5 & 14 & 42 & 132 & 429 & 1430 & 4861 & 16778 & 58598 & 206516 & 732825 & 2613834
\end{array}
$$


_Comments_:
   - Number of (s(0), s(1), ..., s(2n)) such that 0 < s(i) < 10 and |s(i) - s(i-1)| = 1 for i = 1,2,....,2n, s(0) = 1, s(2n) = 1. - _Herbert Kociemba_, Jun 14 2004
   - The sequence 1,2,5,14,... has g.f. 1/(1-2x-x^2/(1-2x-x^2/(1-2x-x^2/(1-2x)))) = (1-6x+10x^2-4x^3)/(1-8x+21x^2-20x^3+5x^4), and is the second binomial transform <a href="http://oeis.org/A001519">A001519</a> aerated. - _Paul Barry_, Dec 17 2009
   - Counts all paths of length (2*n), n>=0, starting and ending at the initial node on the path graph P_9, see the Maple program. - _Johannes W. Meijer_, May 29 2010

_Formulae_:
   - G.f.: (1-7x+15x^2-10x^3+x^4)/(1-8x+21x^2-20x^3+5x^4). - _Ralf Stephan_, May 13 2003
   - a(n) = (1/5)*sum(r, 1, 9, sin(r*Pi/10)^2(2*cos(r*Pi/10))^(2n)), n>=1; a(n) = 8*a(n-1)-21*a(n-2)+20*a(n-3)-5*a(n-4), n>=5. - _Herbert Kociemba_, Jun 14 2004
   - G.f.: 1 / (1 - x / (1 - x / (1 - x / (1 - x / (1 - x / (1 - x / (1 - x / (1 - x )))))))). - _Michael Somos_, May 12 2012

_Cross references_:
   - Cf. <a href="http://oeis.org/A033192">A033192</a>.
   - Cf. <a href="http://oeis.org/A081567">A081567</a>, <a href="http://oeis.org/A147748">A147748</a> and <a href="http://oeis.org/A178381">A178381</a>.
   - Cf. <a href="http://oeis.org/A211216">A211216</a>.

_Links_:
   - Sergey Kitaev, Jeffrey Remmel and Mark Tiefenbruck, <a href="http://arxiv.org/abs/1201.6243">Marked mesh patterns in 132-avoiding permutations I</a>, arXiv:1201.6243v1 [math.CO], 2012 (Corollary 3, case k=8, pages 10-11). [_Bruno Berselli_, May 12 2012]
   - <a href="/index/Rec">Index entries for linear recurrences with constant coefficients</a>, signature (8,-21,20,-5).

<hr><div align='center'><b><a href='http://oeis.org/A058094'>A058094</a></b>: <i>Number of 321-hexagon-avoiding permutations in S_n, i.e. permutations of 1...n with no submatrix equivalent to 321, 56781234, 46781235, 56718234 or 46718235.</i><br></div>

by _Sara Billey_, Dec 03 2000

_Keywords_: `nice,nonn,easy`

_Data_:

$$
\begin{array}{c|ccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 \\
\hline
A058094(n) & 1 & 1 & 2 & 5 & 14 & 42 & 132 & 429 & 1426 & 4806 & 16329 & 55740 & 190787 & 654044 & 2244153
\end{array}
$$


_Comments_:
   - If y is 321-hexagon avoiding, there are simple explicit formulas for all the Kazhdan-Lusztig polynomials P_{x,y} and the Kazhdan-Lusztig basis element C_y is the product of C_{s_i}'s corresponding to any reduced word for y.

_Formulae_:
   - a(n+1) = 6a(n) - 11a(n-1) + 9a(n-2) - 4a(n-3) - 4a(n-4) + a(n-5) for n >= 5.
   - O.g.f.: 1 -x*(1-4*x+4*x^2-3*x^3-x^4+x^5)/(-1+6*x-11*x^2+9*x^3-4*x^4 -4*x^5 +x^6). - _R. J. Mathar_, Dec 02 2007

_Cross references_:
   - Cf. <a href="http://oeis.org/A000108">A000108</a>, <a href="http://oeis.org/A092489">A092489</a>-<a href="http://oeis.org/A092492">A092492</a>.

_Links_:
   - Alois P. Heinz, <a href="/<a href="http://oeis.org/A058094">A058094</a>/b058094.txt">Table of n, a(n) for n = 0..1000</a>
   - S. C. Billey and G. S. Warrington, <a href="http://www-math.mit.edu/~sara/papers/hexagon.ps">Kazhdan-Lusztig Polynomials for 321-hexagon-avoiding permutations</a>, J. Alg. Combinatorics, 13, no. 2 (March 2001), 111-136.
   - <a href="/index/Rec#order_06">Index entries for linear recurrences with constant coefficients</a>, signature (6,-11,9,-4,-4,1).

_References_:
   - Z. Stankova and J. West, Explicit enumeration of 321, hexagon-avoiding permutations, Discrete Math., 280 (2004), 165-189.
<hr><div align='center'><b><a href='http://oeis.org/A080938'>A080938</a></b>: <i>Number of Catalan paths (nonnegative, starting and ending at 0, step +/-1) of 2n steps with all values less than or equal to 7.</i><br></div>

by _Henry Bottomley_, Feb 25 2003

_Keywords_: `nonn,easy`

_Data_:

$$
\begin{array}{c|ccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 \\
\hline
A080938(n) & 1 & 1 & 2 & 5 & 14 & 42 & 132 & 429 & 1429 & 4846 & 16645 & 57686 & 201158 & 704420 & 2473785
\end{array}
$$



_Formulae_:
   - a(n) = <a href="http://oeis.org/A080934">A080934</a>(n,7)
   - G.f.: -(2*x-1)*(2*x^2-4*x+1) / ( (x-1)*(x^3-9*x^2+6*x-1) ). - _Ralf Stephan_, May 13 2003
   - a(n) = 7*a(n-1)-15*a(n-2)+10*a(n-3)-a(n-4) - _Herbert Kociemba_, Jun 13 2004
   - G.f.: 1 / (1 - x / (1 - x / (1 - x / (1 - x / (1 - x / (1 - x / (1 - x))))))). - _Michael Somos_, May 12 2012

_Cross references_:
   - Cf. <a href="http://oeis.org/A000007">A000007</a>, <a href="http://oeis.org/A000012">A000012</a>, <a href="http://oeis.org/A011782">A011782</a>, <a href="http://oeis.org/A001519">A001519</a>, <a href="http://oeis.org/A007051">A007051</a>, <a href="http://oeis.org/A080937">A080937</a>, <a href="http://oeis.org/A024175">A024175</a>, <a href="http://oeis.org/A080938">A080938</a>, <a href="http://oeis.org/A033191">A033191</a> which essentially provide the same sequence for different limits and tend to <a href="http://oeis.org/A000108">A000108</a>.
   - Cf. <a href="http://oeis.org/A211216">A211216</a>, <a href="http://oeis.org/A094826">A094826</a> (first differences).

_Links_:
   - Sergey Kitaev, Jeffrey Remmel and Mark Tiefenbruck, <a href="http://arxiv.org/abs/1201.6243">Marked mesh patterns in 132-avoiding permutations I,</a> arXiv:1201.6243v1 [math.CO], 2012 (Corollary 3, case k=7, pages 10-11). - From _N. J. A. Sloane_, May 09 2012
   - <a href="/index/Rec#order_04">Index entries for linear recurrences with constant coefficients</a>, signature (7,-15,10,-1).

_References_:
   - Wei Chen, Enumeration of Set Partitions Refined by Crossing and Nesting Numbers, MS Thesis, Department of Mathematics. SIMON FRASER UNIVERSITY, Fall 2014. Table 4.1, k=4.
   - Dairyko, Michael; Tyner, Samantha; Pudwell, Lara; Wynn, Casey. Non-contiguous pattern avoidance in binary trees. Electron. J. Combin. 19 (2012), no. 3, Paper 22, 21 pp. MR2967227.
   - L. Pudwell, Pattern avoidance in trees (slides from a talk, mentions many sequences), http://faculty.valpo.edu/lpudwell/slides/notredame.pdf, 2012.
<hr><div align='center'><b><a href='http://oeis.org/A261590'>A261590</a></b>: <i>7-Modular Catalan Numbers C_{n,7}.</i><br></div>

by _Nickolas Hein_, Aug 25 2015

_Keywords_: `nonn`

_Data_:

$$
\begin{array}{c|ccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 \\
\hline
A261590(n) & 1 & 1 & 2 & 5 & 14 & 42 & 132 & 429 & 1429 & 4851 & 16718 & 58331 & 205632 & 731272 & 2620176
\end{array}
$$


_Comments_:
   - Definition: Given a primitive k-th root of unity w, a binary operation a*b=a+wb, and sufficiently general fixed complex numbers x_0, ..., x_n, the k-modular Catalan numbers C_{n,k} enumerate parenthesizations of x_0*x_1*...*x_n that give distinct values.
   - Theorem: C_{n,k} enumerates the following objects:
   - (1) binary trees with n internal nodes avoiding a certain subtree (i.e., comb_k^{+1}),
   - (2) plane trees with n+1 nodes whose non-root nodes have degree less than k,
   - (3) Dyck paths of length 2n avoiding a down-step followed immediately by k consecutive up-steps,
   - (4) partitions with n nonnegative parts bounded by the staircase partition (n-1,n-2,...,1,0) such that each positive number appears fewer than k times,
   - (5) standard 2-by-n Young tableaux whose top row avoids contiguous labels of the form i,j+1,j+2,...,j+k for all i<j, and
   - (6) permutations of {1,2,...,n} avoiding 1-3-2 and 23...(k+1)1.

_Formulae_:
   - sum( 1<=l<=n, (l/n)sum( m_1+...+m_k=n and m_2+2m_3+...+(k-1)m_k=n-l , MC(n;m_1,...,m_k) ) ), where MC(n;m_1,...,m_k) is the multinomial coefficient associated to the multiset (m_1,...,m_k).

_Cross references_:
   - C_{n,1} is the all 1's sequence <a href="http://oeis.org/A000012">A000012</a>. For C_{k,k} with k=2,3,4, see <a href="http://oeis.org/A011782">A011782</a>, <a href="http://oeis.org/A005773">A005773</a>, <a href="http://oeis.org/A159772">A159772</a>. For k=5,6,8,9, see <a href="http://oeis.org/A261588">A261588</a>, <a href="http://oeis.org/A261589">A261589</a>, <a href="http://oeis.org/A261591">A261591</a>, <a href="http://oeis.org/A261592">A261592</a>.

_Links_:
   - Nickolas Hein, Jia Huang, <a href="http://arxiv.org/abs/1508.01688">Modular Catalan Numbers</a>, arXiv:1508.01688 [math.CO], (24-August-2015)

<hr><div align='center'><b><a href='http://oeis.org/A261591'>A261591</a></b>: <i>8-Modular Catalan Numbers C_{n,8}.</i><br></div>

by _Nickolas Hein_, Aug 25 2015

_Keywords_: `nonn`

_Data_:

$$
\begin{array}{c|ccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 \\
\hline
A261591(n) & 1 & 1 & 2 & 5 & 14 & 42 & 132 & 429 & 1430 & 4861 & 16784 & 58695 & 207452 & 739840 & 2658936
\end{array}
$$


_Comments_:
   - Definition: Given a primitive k-th root of unity w, a binary operation a*b=a+wb, and sufficiently general fixed complex numbers x_0, ..., x_n, the k-modular Catalan numbers C_{n,k} enumerate parenthesizations of x_0*x_1*...*x_n that give distinct values.
   - Theorem: C_{n,k} enumerates the following objects:
   - (1) binary trees with n internal nodes avoiding a certain subtree (i.e., comb_k^{+1}),
   - (2) plane trees with n+1 nodes whose non-root nodes have degree less than k,
   - (3) Dyck paths of length 2n avoiding a down-step followed immediately by k consecutive up-steps,
   - (4) partitions with n nonnegative parts bounded by the staircase partition (n-1,n-2,...,1,0) such that each positive number appears fewer than k times,
   - (5) standard 2-by-n Young tableaux whose top row avoids contiguous labels of the form i,j+1,j+2,...,j+k for all i<j, and
   - (6) permutations of {1,2,...,n} avoiding 1-3-2 and 23...(k+1)1.

_Formulae_:
   - sum( 1<=l<=n, (l/n)sum( m_1+...+m_k=n and m_2+2m_3+...+(k-1)m_k=n-l , MC(n;m_1,...,m_k) ) ), where MC(n;m_1,...,m_k) is the multinomial coefficient associated to the multiset (m_1,...,m_k).

_Cross references_:
   - C_{n,1} is the all 1's sequence <a href="http://oeis.org/A000012">A000012</a>. For C_{k,k} with k=2,3,4, see <a href="http://oeis.org/A011782">A011782</a>, <a href="http://oeis.org/A005773">A005773</a>, <a href="http://oeis.org/A159772">A159772</a>. For k=5,6,7,9, see <a href="http://oeis.org/A261588">A261588</a>, <a href="http://oeis.org/A261589">A261589</a>, <a href="http://oeis.org/A261590">A261590</a>, <a href="http://oeis.org/A261592">A261592</a>.

_Links_:
   - Nickolas Hein, Jia Huang, <a href="http://arxiv.org/abs/1508.01688">Modular Catalan Numbers</a>, arXiv:1508.01688 [math.CO], (24-August-2015)

<hr><div align='center'><b><a href='http://oeis.org/A261592'>A261592</a></b>: <i>9-Modular Catalan Numbers C_{n,9}.</i><br></div>

by _Nickolas Hein_, Aug 25 2015

_Keywords_: `nonn`

_Data_:

$$
\begin{array}{c|ccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 \\
\hline
A261592(n) & 1 & 1 & 2 & 5 & 14 & 42 & 132 & 429 & 1430 & 4862 & 16795 & 58773 & 207907 & 742220 & 2670564
\end{array}
$$


_Comments_:
   - Definition: Given a primitive k-th root of unity w, a binary operation a*b=a+wb, and sufficiently general fixed complex numbers x_0, ..., x_n, the k-modular Catalan numbers C_{n,k} enumerate parenthesizations of x_0*x_1*...*x_n that give distinct values.
   - Theorem: C_{n,k} enumerates the following objects:
   - (1) binary trees with n internal nodes avoiding a certain subtree (i.e., comb_k^{+1}),
   - (2) plane trees with n+1 nodes whose non-root nodes have degree less than k,
   - (3) Dyck paths of length 2n avoiding a down-step followed immediately by k consecutive up-steps,
   - (4) partitions with n nonnegative parts bounded by the staircase partition (n-1,n-2,...,1,0) such that each positive number appears fewer than k times,
   - (5) standard 2-by-n Young tableaux whose top row avoids contiguous labels of the form i,j+1,j+2,...,j+k for all i<j, and
   - (6) permutations of {1,2,...,n} avoiding 1-3-2 and 23...(k+1)1.

_Formulae_:
   - sum( 1<=l<=n, (l/n)sum( m_1+...+m_k=n and m_2+2m_3+...+(k-1)m_k=n-l , MC(n;m_1,...,m_k) ) ), where MC(n;m_1,...,m_k) is the multinomial coefficient associated to the multiset (m_1,...,m_k).

_Cross references_:
   - C_{n,1} is the all 1's sequence <a href="http://oeis.org/A000012">A000012</a>. For C_{k,k} with k=2,3,4, see <a href="http://oeis.org/A011782">A011782</a>, <a href="http://oeis.org/A005773">A005773</a>, <a href="http://oeis.org/A159772">A159772</a>. For k=5,6,7,8, see <a href="http://oeis.org/A261588">A261588</a>, <a href="http://oeis.org/A261589">A261589</a>, <a href="http://oeis.org/A261590">A261590</a>, <a href="http://oeis.org/A261591">A261591</a>.

_Links_:
   - Nickolas Hein, Jia Huang, <a href="http://arxiv.org/abs/1508.01688">Modular Catalan Numbers</a>, arXiv:1508.01688 [math.CO], (24-August-2015)

<hr><div align='center'><b><a href='http://oeis.org/A242450'>A242450</a></b>: <i>Number T(n,k) of Dyck paths of semilength n having exactly k (possibly overlapping) occurrences of the consecutive steps UUDDUDUUUUDUDDDDUUDD (with U=(1,1), D=(1,-1)); triangle T(n,k), n>=0, k<=0<=max(0,floor((n-2)/8)), read by rows.</i><br></div>

by _Alois P. Heinz_, Jun 12 2014

_Keywords_: `nonn,tabf`

_Data_:

$$
\begin{array}{c|ccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 \\
\hline
A242450(n) & 1 & 1 & 2 & 5 & 14 & 42 & 132 & 429 & 1430 & 4862 & 16795 & 1 & 58783 & 3 & 208002
\end{array}
$$


_Comments_:
   - UUDDUDUUUUDUDDDDUUDD is a Dyck path that contains all sixteen consecutive step patterns of length 4.


_Cross references_:
   - Row sums give <a href="http://oeis.org/A000108">A000108</a>.
   - T(834828,k) = <a href="http://oeis.org/A243752">A243752</a>(834828,k).
   - T(n,0) = <a href="http://oeis.org/A243753">A243753</a>(n,834828).
   - Cf. <a href="http://oeis.org/A243820">A243820</a>.

_Links_:
   - Alois P. Heinz, <a href="/<a href="http://oeis.org/A242450">A242450</a>/b242450.txt">Rows n = 0..500, flattened</a>


### Search by open content

In [4]:
searchable = oeis_search(query="pascal", table=True)

In [5]:
searchable(data_upper_limit=(10,10), comment=lambda i,c: i in range(10))

_Results for query: <a href='https://oeis.org/search?fmt=json&start=0&q=pascal+keyword%3Atabl'>https://oeis.org/search?fmt=json&start=0&q=pascal+keyword%3Atabl</a>_<br><hr><div align='center'><b><a href='http://oeis.org/A007318'>A007318</a></b>: <i>Pascal's triangle read by rows: C(n,k) = binomial(n,k) = n!/(k!*(n-k)!), 0 <= k <= n.</i><br></div>

by _N. J. A. Sloane_ and _Mira Bernstein_, Apr 28 1994

_Keywords_: `nonn,tabl,nice,easy,core,look,hear,changed`

_Data_:

$$
\begin{array}{c|ccccccccccc}
n, k & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\hline
0 & 1 &  &  &  &  &  &  &  &  & \\1 & 1 & 1 &  &  &  &  &  &  &  & \\2 & 1 & 2 & 1 &  &  &  &  &  &  & \\3 & 1 & 3 & 3 & 1 &  &  &  &  &  & \\4 & 1 & 4 & 6 & 4 & 1 &  &  &  &  & \\5 & 1 & 5 & 10 & 10 & 5 & 1 &  &  &  & \\6 & 1 & 6 & 15 & 20 & 15 & 6 & 1 &  &  & \\7 & 1 & 7 & 21 & 35 & 35 & 21 & 7 & 1 &  & \\8 & 1 & 8 & 28 & 56 & 70 & 56 & 28 & 8 & 1 & \\9 & 1 & 9 & 36 & 84 & 126 & 126 & 84 & 36 & 9 & 1
\end{array}
$$


_Comments_:
   - C(n,k) = number of k-element subsets of an n-element set.
   - Row n gives coefficients in expansion of (1+x)^n.
   - Binomial(n+k-1,n-1) is the number of ways of placing k indistinguishable balls into n boxes (the "bars and stars" argument - see Feller).
   - Binomial(n-1,k-1) is the number of compositions (ordered partitions) of n with k summands.
   - Binomial(n+k-1,k-1) is the number of weak compositions (ordered weak partitions) of n into exactly k summands.- _Juergen Will_, Jan 23 2016
   - Binomial(n,k) is the number of lattice paths from (0,0) to (n,k) using steps (1,0) and (1,1). - _Joerg Arndt_, Jul 01 2011
   - If thought of as an infinite lower triangular matrix, inverse begins:
   -   +1
   -   -1 +1
   -   +1 -2 +1

_Formulae_:
   - a(n, k) = C(n,k) = binomial(n, k).
   - C(n, k) = C(n-1, k) + C(n-1, k-1).
   - a(n+1, m) = a(n, m) + a(n, m-1), a(n, -1) := 0, a(n, m) := 0, n<m; a(0, 0)=1.
   - C(n, k)=n!/(k!(n-k)!) if 0<=k<=n, otherwise 0.
   - G.f.: 1/(1-y-x*y) = Sum(C(n, k)*x^k*y^n, n, k>=0)
   - G.f.: 1/(1-x-y) = Sum(C(n+k, k)*x^k*y^n, n, k>=0).
   - G.f. for row n: (1+x)^n = Sum_{k=0..n} C(n, k)x^k.
   - G.f. for column n: x^n/(1-x)^n.
   - E.g.f.: A(x, y) = exp(x+x*y).
   - E.g.f. for column n: x^n*exp(x)/n!.
   - In general the m-th power of <a href="http://oeis.org/A007318">A007318</a> is given by: T(0, 0) = 1, T(n, k) = T(n-1, k-1) + m*T(n-1, k), where n is the row-index and k is the column; also T(n, k) = m^(n-k) C(n, k).
   - Triangle T(n, k) read by rows; given by <a href="http://oeis.org/A000007">A000007</a> DELTA <a href="http://oeis.org/A000007">A000007</a>, where DELTA is Deléham's operator defined in <a href="http://oeis.org/A084938">A084938</a>.
   - Let P(n+1) = the number of integer partitions of (n+1); let p(i) = the number of parts of the i-th partition of (n+1); let d(i) = the number of different parts of the i-th partition of (n+1); let m(i, j) = multiplicity of the j-th part of the i-th partition of (n+1). Define the operator Sum_{i=1..P(n+1), p(i)=k+1} as the sum running from i=1 to i=P(n+1) but taking only partitions with p(i)=(k+1) parts into account. Define the operator Product_{j=1..d(i)} = product running from j=1 to j=d(i). Then C(n, k) = Sum_{p(i)=(k+1), i=1..P(n+1)} p(i)! / [Product_{j=1..d(i)} m(i, j)!]. E.g., C(5, 3) = 10 because n=6 has the following partitions with m=3 parts: (114), (123), (222). For their multiplicities one has: (114): 3!/(2!*1!) = 3; (123): 3!/(1!*1!*1!) = 6; (222): 3!/3! = 1. The sum is 3 + 6 + 1 = 10 = C(5, 3). - _Thomas Wieder_, Jun 03 2005
   - C(n, k) = Sum_{j=0..k} = (-1)^j*C(n+1+j, k-j)*<a href="http://oeis.org/A000108">A000108</a>(j). - _Philippe Deléham_, Oct 10 2005
   - G.f.: 1 + x(1 + x) + x^3(1 + x)^2 + x^6(1 + x)^3 + ... . - _Michael Somos_, Sep 16 2006
   - Sum_{k=0..floor(n/2)} x^(n-k)*T(n-k,k) = <a href="http://oeis.org/A000007">A000007</a>(n), <a href="http://oeis.org/A000045">A000045</a>(n+1), <a href="http://oeis.org/A002605">A002605</a>(n), <a href="http://oeis.org/A030195">A030195</a>(n+1), <a href="http://oeis.org/A057087">A057087</a>(n), <a href="http://oeis.org/A057088">A057088</a>(n), <a href="http://oeis.org/A057089">A057089</a>(n), <a href="http://oeis.org/A057090">A057090</a>(n), <a href="http://oeis.org/A057091">A057091</a>(n), <a href="http://oeis.org/A057092">A057092</a>(n), <a href="http://oeis.org/A057093">A057093</a>(n) for x = 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, respectively. Sum_{k=0..floor(n/2)} (-1)^k*x^(n-k)*T(n-k,k) = <a href="http://oeis.org/A000007">A000007</a>(n), <a href="http://oeis.org/A010892">A010892</a>(n), <a href="http://oeis.org/A009545">A009545</a>(n+1), <a href="http://oeis.org/A057083">A057083</a>(n), <a href="http://oeis.org/A001787">A001787</a>(n+1), <a href="http://oeis.org/A030191">A030191</a>(n), <a href="http://oeis.org/A030192">A030192</a>(n), <a href="http://oeis.org/A030240">A030240</a>(n), <a href="http://oeis.org/A057084">A057084</a>(n), <a href="http://oeis.org/A057085">A057085</a>(n+1), <a href="http://oeis.org/A057086">A057086</a>(n), <a href="http://oeis.org/A084329">A084329</a>(n+1) for x = 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, respectively. - _Philippe Deléham_, Sep 16 2006
   - C(n,k) <= <a href="http://oeis.org/A062758">A062758</a>(n) for n > 1. - _Reinhard Zumkeller_, Mar 04 2008
   - C(t+p-1, t) = Sum_{i=0..t} C(i+p-2, i) = Sum_{i=1..p} C(i+t-2, t-1). A binomial number is the sum of its left parent and all its right ancestors, which equals the sum of its right parent and all its left ancestors. - Lee Naish (lee(AT)cs.mu.oz.au), Mar 07 2008
   - From _Paul D. Hanna_, Mar 24 2011: 
       - Let A(x) = Sum_{n>=0} x^(n(n+1)/2)*(1+x)^n be the g.f. of the flattened triangle:
       - A(x) = 1 + (x + x^2) + (x^3 + 2*x^4 + x^5) + (x^6 + 3*x^7 + 3*x^8 + x^9) +...
       - then A(x) equals the series Sum_{n>=0} (1+x)^n*x^n*Product_{k=1..n} (1-(1+x)*x^(2k-1))/(1-(1+x)*x^(2k));
       - also, A(x) equals the continued fraction 1/(1- x*(1+x)/(1+ x*(1-x)*(1+x)/(1- x^3*(1+x)/(1+ x^2*(1-x^2)*(1+x)/(1- x^5*(1+x)/(1+ x^3*(1-x^3)*(1+x)/(1- x^7*(1+x)/(1+ x^4*(1-x^4)*(1+x)/(1- ...))))))))).
       - These formulas are due to (1) a q-series identity and (2) a partial elliptic theta function expression. 
   - Row n of the triangle is the result of applying the ConvOffs transform to the first n terms of the natural numbers (1, 2, 3, ..., n). See <a href="http://oeis.org/A001263">A001263</a> or <a href="http://oeis.org/A214281">A214281</a> for a definition of this transformation. - _Gary W. Adamson_, Jul 12 2012
   - From _L. Edson Jeffery_, Aug 02 2012: 
       - Row n (n >= 0) of the triangle is given by the n-th antidiagonal of the infinite matrix P^n, where P = (p_{i,j}), i,j >= 0, is the production matrix
       - 0, 1,
       - 1, 0, 1,
       - 0, 1, 0, 1,
       - 0, 0, 1, 0, 1,
       - 0, 0, 0, 1, 0, 1,
       - 0, 0, 0, 0, 1, 0, 1,
       - 0, 0, 0, 0, 0, 1, 0, 1,
       - 0, 0, 0, 0, 0, 0, 1, 0, 1,
       - ... 
   - Row n of the triangle is also given by the n+1 coefficients of the polynomial P_n(x) defined by the recurrence P_0(x) = 1, P_1(x) = x + 1, P_n(x) = x*P_{n-1}(x) + P_{n-2}(x), n > 1. - _L. Edson Jeffery_, Aug 12 2013
   - For a closed-form formula for arbitrary left and right borders of Pascal-like triangles see <a href="http://oeis.org/A228196">A228196</a>. - _Boris Putievskiy_, Aug 18 2013
   - For a closed-form formula for generalized Pascal's triangle see <a href="http://oeis.org/A228576">A228576</a>. - _Boris Putievskiy_, Sep 04 2013
   - (1+x)^n = Sum_{k=0..n} (-1)^(n-k)*binomial(n,k)*Sum_{i=0..k} k^(n-i)*binomial(k,i)*x^(n-i)/(n-i)!. - _Vladimir Kruchinin_, Oct 21 2013
   - E.g.f.: A(x,y) = exp(x+x*y) = 1 + (x+y*x)/( E(0)-(x+y*x)), where E(k) = 1 + (x+y*x)/(1 + (k+1)/E(k+1) ); (continued fraction). - _Sergei N. Gladkovskii_, Nov 08 2013
   - E.g.f.:  E(0) -1, where E(k) = 2 + x*(1+y)/(2*k+1 - x*(1+y)/E(k+1) ); (continued fraction). - _Sergei N. Gladkovskii_, Dec 24 2013
   - G.f.: 1 + x*(1+x)*(1+x^2*(1+x)/(W(0)-x^2-x^3)), where W(k) = 1 + (1+x)*x^(k+2) - (1+x)*x^(k+3)/W(k+1); (continued fraction). - _Sergei N. Gladkovskii_, Dec 24 2013
   - Sum_{n>=0} C(n,k)/n! = e/k!, where e = exp(1), while allowing n < k where C(n,k) = 0. Also Sum_{n>=0} C(n+k-1,k)/n! = e * <a href="http://oeis.org/A000262">A000262</a>(k)/k!, and for k>=1 equals e * <a href="http://oeis.org/A067764">A067764</a>(k)/<a href="http://oeis.org/A067653">A067653</a>(k). - _Richard R. Forberg_, Jan 01 2014
   - Sum_{n>=k} 1/C(n,k) = k/(k-1) for k>=1. - _Richard R. Forberg_, Feb 10 2014
   - From _Tom Copeland_, Apr 17 and 26 2014: 
       - Multiply each n-th diagonal of the Pascal lower triangular matrix by x^n and designate the result by <a href="http://oeis.org/A007318">A007318</a>(x) = P(x). Then with :xD:^n = x^n*(d/dx)^n and B(n,x), the Bell polynomials (<a href="http://oeis.org/A008277">A008277</a>),
       - A) P(x)= exp(x*dP) = exp[x*(e^M-I)] = exp[M*B(.,x)] = (I+dP)^B(.,x)
       -    with dP = <a href="http://oeis.org/A132440">A132440</a>, M = <a href="http://oeis.org/A238385">A238385</a>-I, and I = identity matrix, and
       - B) P(:xD:) = exp(dP:xD:) = exp[(e^M-I):xD:] = exp[M*B(.,:xD:)] = exp[M*xD] = (I+dP)^(xD) with action P(:xD:)g(x) = exp(dP:xD:)g(x) = g[(I+dP)*x] (cf. also <a href="http://oeis.org/A238363">A238363</a>).
       - C) P(x)^y = P(y*x). P(2x) = <a href="http://oeis.org/A038207">A038207</a>(x) = exp[M*B(.,2x)], the face vectors of the n-dim hypercubes.
       - D) P(x) = [St2]*exp(x*M)*[St1] = [St2]*(I+dP)^x*[St1]
       - E) = [St1]^(-1)*(I+dP)^x*[St1] = [St2]*(I+dP)^x*[St2]^(-1)
       - where [St1]=padded <a href="http://oeis.org/A008275">A008275</a> just as [St2]=<a href="http://oeis.org/A048993">A048993</a>=padded <a href="http://oeis.org/A008277">A008277</a> and exp(x*M) = (I+dP)^x = sum(k=0,..,infinity, C(x,k) dP^k). 
   - From _Peter Bala_, Dec 21 2014: 
       - Recurrence equation: T(n,k) = T(n-1,k)*(n + k)/(n - k) - T(n-1,k-1) for n >= 2 and 1 <= k < n, with boundary conditions T(n,0) = T(n,n) = 1. Note, changing the minus sign in the recurrence to a plus sign gives a recurrence for the square of the binomial coefficients - see <a href="http://oeis.org/A008459">A008459</a>.
       - There is a relation between the e.g.f.'s of the rows and the diagonals of the triangle, namely, exp(x) * e.g.f. for row n = e.g.f. for diagonal n. For example, for n = 3 we have exp(x)*(1 + 3*x + 3*x^2/2! + x^3/3!) = 1 + 4*x + 10*x^2/2! + 20*x^3/3! + 35*x^4/4! + .... This property holds more generally for the Riordan arrays of the form ( f(x), x/(1 - x) ), where f(x) is an o.g.f. of the form 1 + f_1*x + f_2*x^2 + .... See, for example, <a href="http://oeis.org/A055248">A055248</a> and <a href="http://oeis.org/A106516">A106516</a>.
       - Let P denote the present triangle. For k = 0,1,2,... define P(k) to be the lower unit triangular block array
       - /I_k 0\
       - \ 0  P/ having the k X k identity matrix I_k as the upper left block; in particular, P(0) = P. The infinite product P(0)*P(1)*P(2)*..., which is clearly well-defined, is equal to the triangle of Stirling numbers of the second kind <a href="http://oeis.org/A008277">A008277</a>. The infinite product in the reverse order, that is, ...*P(2)*P(1)*P(0), is equal to the triangle of Stirling cycle numbers <a href="http://oeis.org/A130534">A130534</a>. 
   - C(a+b,c) = Sum_{k=0..a} C(a,k)*C(b,b-c+k). This is a generalization of equation 1 from section 4.2.5 of the Prudnikov et al. reference, for a=b=c=n: C(2n,n) = Sum_{k=0..n} C(n,k)^2. See Links section for animation of new formula. - _Hermann Stamm-Wilbrandt_, Aug 26 2015
   - The row polynomials of the Pascal matrix P(n,x) = (1+x)^n are related to the Bernoulli polynomials Br(n,x) and their umbral compositional inverses Bv(n,x) by the umbral relation P(n,x) = (-Br(.,-Bv(.,x)))^n = (-1)^n Br(n,-Bv(.,x)), which translates into the matrix relation P = M * Br * M * Bv, where P is the Pascal matrix, M is the diagonal matrix diag(1,-1,1,-1,...), Br is the matrix for the coefficients of the Bernoulli polynomials, and Bv that for the umbral inverse polynomials defined umbrally by Br(n,Bv(.,x)) = x^n = Bv(n,Br(.,x)). Note M = M^(-1). - _Tom Copeland_, Sep 05 2015

_Cross references_:
   - Equals differences between consecutive terms of <a href="http://oeis.org/A102363">A102363</a>. - David G. Williams (davidwilliams(AT)Paxway.com), Jan 23 2006
   - Row sums give <a href="http://oeis.org/A000079">A000079</a> (powers of 2).
   - Cf. <a href="http://oeis.org/A083093">A083093</a> (triangle read mod 3), <a href="http://oeis.org/A214292">A214292</a> (first differences of rows).
   - Partial sums of rows give triangle <a href="http://oeis.org/A008949">A008949</a>.
   - Infinite matrix squared: <a href="http://oeis.org/A038207">A038207</a>, cubed: <a href="http://oeis.org/A027465">A027465</a>.
   - Cf. <a href="http://oeis.org/A101164">A101164</a>. If rows are sorted we get <a href="http://oeis.org/A061554">A061554</a> or <a href="http://oeis.org/A107430">A107430</a>.
   - Another version: <a href="http://oeis.org/A108044">A108044</a>.
   - Cf. <a href="http://oeis.org/A008277">A008277</a>, <a href="http://oeis.org/A132311">A132311</a>, <a href="http://oeis.org/A132312">A132312</a>, <a href="http://oeis.org/A052216">A052216</a>, <a href="http://oeis.org/A052217">A052217</a>, <a href="http://oeis.org/A052218">A052218</a>, <a href="http://oeis.org/A052219">A052219</a>, <a href="http://oeis.org/A052220">A052220</a>, <a href="http://oeis.org/A052221">A052221</a>, <a href="http://oeis.org/A052222">A052222</a>, <a href="http://oeis.org/A052223">A052223</a>, <a href="http://oeis.org/A144225">A144225</a>, <a href="http://oeis.org/A202750">A202750</a>, <a href="http://oeis.org/A211226">A211226</a>, <a href="http://oeis.org/A047999">A047999</a>, <a href="http://oeis.org/A026729">A026729</a>, <a href="http://oeis.org/A052553">A052553</a>, <a href="http://oeis.org/A051920">A051920</a>, <a href="http://oeis.org/A193242">A193242</a>.
   - Triangle sums (see the comments): <a href="http://oeis.org/A000079">A000079</a> (Row1); <a href="http://oeis.org/A000007">A000007</a> (Row2); <a href="http://oeis.org/A000045">A000045</a> (Kn11 & Kn21); <a href="http://oeis.org/A000071">A000071</a> (Kn12 & Kn22); <a href="http://oeis.org/A001924">A001924</a> (Kn13 & Kn23); <a href="http://oeis.org/A014162">A014162</a> (Kn14 & Kn24); <a href="http://oeis.org/A014166">A014166</a> (Kn15 & Kn25); <a href="http://oeis.org/A053739">A053739</a> (Kn16 & Kn26); <a href="http://oeis.org/A053295">A053295</a> (Kn17 & Kn27); <a href="http://oeis.org/A053296">A053296</a> (Kn18 & Kn28); <a href="http://oeis.org/A053308">A053308</a> (Kn19 & Kn29); <a href="http://oeis.org/A053309">A053309</a> (Kn110 & Kn210); <a href="http://oeis.org/A001519">A001519</a> (Kn3 & Kn4); <a href="http://oeis.org/A011782">A011782</a> (Fi1 & Fi2); <a href="http://oeis.org/A000930">A000930</a> (Ca1 & Ca2); <a href="http://oeis.org/A052544">A052544</a> (Ca3 & Ca4); <a href="http://oeis.org/A003269">A003269</a> (Gi1 & Gi2); <a href="http://oeis.org/A055988">A055988</a> (Gi3 & Gi4); <a href="http://oeis.org/A034943">A034943</a> (Ze1 & Ze2); <a href="http://oeis.org/A005251">A005251</a> (Ze3 & Ze4). - _Johannes W. Meijer_, Sep 22 2010
   - Fibonacci-Pascal triangles: <a href="http://oeis.org/A027926">A027926</a>, <a href="http://oeis.org/A036355">A036355</a>, <a href="http://oeis.org/A037027">A037027</a>, <a href="http://oeis.org/A074829">A074829</a>, <a href="http://oeis.org/A105809">A105809</a>, <a href="http://oeis.org/A109906">A109906</a>, <a href="http://oeis.org/A111006">A111006</a>, <a href="http://oeis.org/A114197">A114197</a>, <a href="http://oeis.org/A162741">A162741</a>, <a href="http://oeis.org/A228074">A228074</a>, <a href="http://oeis.org/A228196">A228196</a>, <a href="http://oeis.org/A228576">A228576</a>.
   - Cf. <a href="http://oeis.org/A137948">A137948</a>, <a href="http://oeis.org/A245334">A245334</a>.
   - Cf. <a href="http://oeis.org/A085478">A085478</a>, <a href="http://oeis.org/A258993">A258993</a>.

_Links_:
   - N. J. A. Sloane, <a href="/<a href="http://oeis.org/A007318">A007318</a>/b007318.txt">First 141 rows of Pascal's triangle, formatted as a simple linear sequence n, a(n)</a>
   - M. Abramowitz and I. A. Stegun, eds., <a href="http://www.nrbook.com/abramowitz_and_stegun/">Handbook of Mathematical Functions</a>, National Bureau of Standards, Applied Math. Series 55, Tenth Printing, 1972 [alternative scanned copy].
   - Tewodros Amdeberhan, Moa Apagodu, Doron Zeilberger, <a href="http://arxiv.org/abs/1507.07660">Wilf's "Snake Oil" Method Proves an Identity in The Motzkin Triangle</a>, arXiv:1507.07660 [math.CO], 2015.
   - V. Asundi, <a href="http://www.stanford.edu/class/gene211/examples/class8/yanghui.html">Generate a Yanghui Triangle</a>
   - S. V. Ault and C. Kicey, <a href="http://dx.doi.org/10.1016/j.disc.2014.05.020">Counting paths in corridors using circular Pascal arrays</a>, Discrete Mathematics (2014).
   - Mohammad K. Azarian, <a href="http://www.m-hikari.com/ijcms/ijcms-2012/37-40-2012/azarianIJCMS37-40-2012.pdf">Fibonacci Identities as Binomial Sums</a>, International Journal of Contemporary Mathematical Sciences, Vol. 7, No. 38, 2012, pp. 1871-1876.
   - Mohammad K. Azarian, <a href="http://www.m-hikari.com/ijcms/ijcms-2012/41-44-2012/azarianIJCMS41-44-2012.pdf">Fibonacci Identities as Binomial Sums II</a>, International Journal of Contemporary Mathematical Sciences, Vol. 7, No. 42, 2012, pp. 2053-2059.
   - P. Bala, <a href="/<a href="http://oeis.org/A007318">A007318</a>/a007318.pdf">A combinatorial interpretation for the binomial coefficients</a>
   - C. Banderier and D. Merlini, <a href="http://algo.inria.fr/banderier/Papers/infjumps.ps">Lattice paths with an infinite set of jumps</a>
   - J. Fernando Barbero G., Jesús Salas, Eduardo J. S. Villaseñor, <a href="http://arxiv.org/abs/1307.2010">Bivariate Generating Functions for a Class of Linear Recurrences. I. General Structure</a>, arXiv:1307.2010 [math.CO], 2013.
   - Paul Barry, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL9/Barry/barry91.html">On Integer-Sequence-Based Constructions of Generalized Pascal Triangles</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.2.4.
   - P. Barry, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL12/Barry4/barry64.html">Symmetric Third-Order Recurring Sequences, Chebyshev Polynomials, and Riordan Arrays </a>, JIS 12 (2009) 09.8.6
   - Paul Barry, <a href="http://arxiv.org/abs/1105.3043">Eulerian polynomials as moments, via exponential Riordan arrays</a>, arXiv:1105.3043 [math.CO], 2011.
   - Paul Barry, <a href="http://arxiv.org/abs/1105.3044">Combinatorial polynomials as moments, Hankel transforms and exponential Riordan arrays</a>, arXiv:1105.3044 [math.CO], 2011.
   - Paul Barry, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL15/Barry4/bern2.html">Riordan-Bernstein Polynomials, Hankel Transforms and Somos Sequences</a>, Journal of Integer Sequences, Vol. 15 2012, #12.8.2
   - Paul Barry, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL16/Barry1/barry242.html">On the Central Coefficients of Riordan Matrices</a>, Journal of Integer Sequences, 16 (2013), #13.5.1.
   - Paul Barry, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL16/Barry2/barry231.html">A Note on a Family of Generalized Pascal Matrices Defined by Riordan Arrays</a>, Journal of Integer Sequences, 16 (2013), #13.5.4.
   - Paul Barry, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL16/Barry3/barry252.html">On the Inverses of a Family of Pascal-Like Matrices Defined by Riordan Arrays</a>, Journal of Integer Sequences, 16 (2013), #13.5.6.
   - Paul Barry, <a href="http://dx.doi.org/10.1155/2013/657806">On the Connection Coefficients of the Chebyshev-Boubaker polynomials</a>, The Scientific World Journal, Volume 2013 (2013), Article ID 657806, 10 pages.
   - Paul Barry, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL16/Barry4/barry271.html">General Eulerian Polynomials as Moments Using Exponential Riordan Arrays</a>, Journal of Integer Sequences, 16 (2013), #13.9.6.
   - P. Barry and A. Hennessy, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL15/Barry1/barry202.html">Four-term Recurrences, Orthogonal Polynomials and Riordan Arrays</a>, Journal of Integer Sequences, 2012, article 12.4.2. - From _N. J. A. Sloane_, Sep 21 2012
   - J. W. Bober, <a href="http://arxiv.org/abs/0709.1977">Factorial ratios, hypergeometric series, and a family of step functions</a>, arXiv:0709.1977v1 [math.NT], J. London Math. Soc. (2) 79 (2009), 422-444.
   - B. A. Bondarenko, <a href="http://www.fq.math.ca/pascal.html">Generalized Pascal Triangles and Pyramids</a>, English translation published by Fibonacci Association, Santa Clara Univ., Santa Clara, CA, 1993; see p. 4.
   - D. Butler, <a href="http://www.tsm-resources.com/alists/pasc.html">Pascal's Triangle</a>
   - Naiomi T. Cameron and Asamoah Nkwanta, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL8/Cameron/cameron46.html">On Some (Pseudo) Involutions in the Riordan Group</a>, Journal of Integer Sequences, Vol. 8 (2005), Article 05.3.7.
   - C. Cobeli and A. Zaharescu, <a href="http://rms.unibuc.ro/bulletin/pdf/56-1/PromenadePascalPart1.pdf">Promenade around Pascal Triangle-Number Motives</a>, Bull. Math. Soc. Sci. Math. Roumanie, Tome 56(104) No. 1, 2013, 73-98. - From _N. J. A. Sloane_, Feb 16 2013
   - J. H. Conway and N. J. A. Sloane, <a href="http://citeseerx.ist.psu.edu/viewdoc/download;jsessionid=E556ADF68595139EB07F425FD5A79F11?doi=10.1.1.39.9899&amp;rep=rep1&amp;type=pdf">Low-dimensional lattices. VII Coordination sequences</a>, Proc. R. Soc. Lond. A (1997) 453, 2369-2389.
   - T. Copeland, <a href="http://tcjpn.wordpress.com/2012/11/29/infinigens-the-pascal-pyramid-and-the-witt-and-virasoro-algebras/">Infinitesimal Generators, the Pascal Pyramid, and the Witt and Virasoro Algebras</a>
   - Tomislav Došlic, Darko Veljan, <a href="http://dx.doi.org/10.1016/j.disc.2007.04.066">Logarithmic behavior of some combinatorial sequences</a>, Discrete Math. 308 (2008), no. 11, 2182--2212. MR2404544 (2009j:05019) - From _N. J. A. Sloane_, May 01 2012
   - L. Euler, <a href="http://arXiv.org/abs/math.HO/0505425">On the expansion of the power of any polynomial (1+x+x^2+x^3+x^4+etc.)^n</a>, arXiv:math/0505425 [math.HO], 2005.
   - L. Euler, <a href="http://www.eulerarchive.org">De evolutione potestatis polynomialis cuiuscunque (1+x+x^2+x^3+x^4+etc.)^n</a> E709
   - A. Farina, et al., <a href="http://dx.doi.org/10.1007/s11760-011-0228-6">Tartaglia-Pascal's triangle: a historical perspective with applications</a>, Signal, Image and Video Processing, January 2013, Volume 7, Issue 1, pp 173-188.
   - S. R. Finch, P. Sebah and Z.-Q. Bai, <a href="http://arXiv.org/abs/0802.2654">Odd Entries in Pascal's Trinomial Triangle</a>, arXiv:0802.2654 [math.NT], 2008.
   - D. Fowler, <a href="http://www.jstor.org/stable/2975209">The binomial coefficient function</a>, Amer. Math. Monthly, 103 (1996), 1-17.
   - He, Tian-Xiao, and Sprugnoli, Renzo; <a href="http://dx.doi.org/10.1016/j.disc.2008.11.021">Sequence characterization of Riordan arrays</a>, Discrete Math. 309 (2009), no. 12, 3962-3974.
   - Nick Hobson, <a href="/<a href="http://oeis.org/A007318">A007318</a>/a007318.py.txt">Python program</a>
   - V. E. Hoggatt, Jr. and M. Bicknell, <a href="http://www.fq.math.ca/Scanned/14-5/hoggatt1.pdf">Catalan and related sequences arising from inverses of Pascal's triangle matrices</a>, Fib. Quart., 14 (1976), 395-405.
   - Matthew Hubbard and Tom Roby, <a href="http://web.archive.org/web/20060912171955/http://binomial.csuhayward.edu/">Pascal's Triangle From Top to Bottom</a> [archived page]
   - Charles Jordan, <a href="http://books.google.co.jp/books/about/Calculus_of_Finite_Differences.html?id=3RfZOsDAyQsC&amp;redir_esc=y">Calculus of Finite Differences</a> (p. 65) [From _Tom Copeland_, Apr 16 2014]
   - S. Kak, <a href="http://arxiv.org/abs/physics/0411195">The Golden Mean and the Physics of Aesthetics</a>, arXiv:physics/0411195 [physics.hist-ph], 2004.
   - W. Knight, <a href="http://www.math.unb.ca/~knight/utility/binomial.html">Short Table of Binomial Coefficients</a>
   - W. Lang, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL3/LANG/lang.html">On generalizations of Stirling number triangles</a>, J. Integer Seqs., Vol. 3 (2000), #00.2.4.
   - P. A. MacMahon, <a href="http://www.jstor.org/stable/90632?seq=1#page_scan_tab_contents">Memoir on the Theory of the Compositions of Numbers</a>, Phil. Trans. Royal Soc. London A, 184 (1893), 835-901.
   - Mathforum, <a href="http://mathforum.org/dr.math/faq/faq.pascal.triangle.html">Pascal's Triangle</a>
   - Mathforum, <a href="http://mam2000.mathforum.org/workshops/usi/pascal/pascal.links.html">Links for Pascal's triangle</a>
   - C. McDermottroe, <a href="http://www.netsoc.tcd.ie/~jgilbert/maths_site/applets/sequences_and_series/pascal_s_triangle_and_the_binomial_theorem.html">n-th row generator of Pascal's triangle</a>
   - D. Merlini, R. Sprugnoli and M. C. Verri, <a href="http://www.dsi.unifi.it/~merlini/Lucidi.ps">An algebra for proper generating trees</a>
   - D. Merlini, F. Uncini and M. C. Verri, <a href="http://www.emis.de/journals/INTEGERS/papers/e23/e23.Abstract.html">A unified approach to the study of general and palindromic compositions</a>, Integers 4 (2004), A23, 26 pp.
   - Y. Moshe, <a href="http://dx.doi.org/10.1016/S0022-314X(03)00103-3">The density of 0's in recurrence double sequences</a>, J. Number Theory, 103 (2003), 109-121.
   - Lee Naish <a href="http://www.cs.mu.oz.au/~lee/papers/pascal/">Pascal's Triangle and debugging software</a>
   - A. Necer, <a href="http://www.emis.de/journals/JTNB/1997-2/jtnb9-2_english.html#jourelec">Series formelles et produit de Hadamard</a>
   - Asamoah Nkwanta and Earl R. Barnes, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL15/Nkwanta/nkwanta2.html">Two Catalan-type Riordan Arrays and their Connections to the Chebyshev Polynomials of the First Kind</a>, Journal of Integer Sequences, Article 12.3.3, 2012. - From _N. J. A. Sloane_, Sep 16 2012
   - A. Nkwanta, A. Tefera, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL16/Nkwanta/nkwanta4.html">Curious Relations and Identities Involving the Catalan Generating Function and Numbers</a>, Journal of Integer Sequences, 16 (2013), #13.9.5.
   - M. A. A. Obaid, S. K. Nauman, W. M. Fakieh, C. M. Ringel, <a href="http://www.math.uni-bielefeld.de/~ringel/opus/jeddah.pdf">The numbers of support-tilting modules for a Dynkin algebra</a>, 2014.
   - OEIS Wiki, <a href="/wiki/Binomial_coefficients">Binomial coefficients</a>
   - Ed Pegg, Jr., <a href="http://www.mathpuzzle.com/MAA/07-Sequence%20Pictures/mathgames_12_08_03.html">Sequence Pictures</a>, Math Games column, Dec 08 2003.
   - Ed Pegg, Jr., <a href="/<a href="http://oeis.org/A000043">A000043</a>/a000043_2.pdf">Sequence Pictures</a>, Math Games column, Dec 08 2003 [Cached copy, with permission (pdf only)]
   - T. M. Richardson, <a href="http://arxiv.org/abs/1405.6315">The Reciprocal Pascal Matrix</a>, arXiv preprint arXiv:1405.6315, 2014
   - L. W. Shapiro, S. Getu, W.-J. Woan and L. C. Woodson, <a href="http://dx.doi.org/10.1016/0166-218X(91)90088-E">The Riordan group</a>, Discrete Applied Math., 34 (1991), 229-239.
   - G. Sivek et al., ThinkQuest, <a href="http://library.thinkquest.org/C006087/english/interactive.shtml">Pascal's Triangle Row Generator</a>
   - N. J. A. Sloane, <a href="http://neilsloane.com/doc/sg.txt">My favorite integer sequences</a>, in Sequences and their Applications (Proceedings of SETA '98).
   - N. J. A. Sloane, <a href="/<a href="http://oeis.org/A222314">A222314</a>/a222314_1.pdf">Triangle showing silhouette of first 30 rows of Pascal's triangle (after Cobeli and Zaharescu)</a>
   - Hermann Stamm-Wilbrandt, <a href="/<a href="http://oeis.org/A007318">A007318</a>/a007318_2.gif">Compute C(n+m,...) based on C(n,...) and C(m,...) values animation</a>
   - Ch. Stover and E. W. Weisstein, <a href="http://mathworld.wolfram.com/Composition.html">Composition</a>. From MathWorld - A Wolfram Web Resource.
   - H. Verrill, <a href="http://hverrill.net/pages~helena/pascal">Pascal's Triangle and related triangles</a>
   - G. Villemin's Almanach of Numbers, <a href="http://villemin.gerard.free.fr/Wwwgvmm/Iteration/TrgPasca.htm">Triangle de Pascal</a>
   - Eric Weisstein's World of Mathematics, <a href="http://mathworld.wolfram.com/PascalsTriangle.html">Pascal's Triangle</a>
   - Wikipedia, <a href="http://en.wikipedia.org/wiki/Pascal&#39;s_triangle">Pascal's triangle</a>
   - H. S. Wilf, <a href="http://www.math.upenn.edu/~wilf/DownldGF.html">Generatingfunctionology</a>, 2nd edn., Academic Press, NY, 1994, pp. 12ff.
   - K. Williams, Mathforum, <a href="http://mathforum.org/dr.cgi/pascal.cgi">Interactive Pascal's Triangle</a>
   - K. Williams, MathForum, <a href="http://mathforum.org/~ken/pascal.html">Pascal's Triangle to Row 19</a>
   - D. Zeilberger, <a href="http://arXiv.org/abs/math.CO/9809136">The Combinatorial Astrology of Rabbi Abraham Ibn Ezra</a>, arXiv:math/9809136 [math.CO], 1998.
   - <a href="/index/Pas#Pascal">Index entries for triangles and arrays related to Pascal's triangle</a>
   - <a href="/index/Cor#core">Index entries for "core" sequences</a>

_References_:
   - M. Abramowitz and I. A. Stegun, eds., Handbook of Mathematical Functions, National Bureau of Standards Applied Math. Series 55, 1964 (and various reprintings), p. 828.
   - P Barry, Riordan arrays, generalized Narayana triangles, and series reversion, Linear Algebra and its Applications, 491 (2016) 343-385.
   - A. T. Benjamin and J. J. Quinn, Proofs that really count: the art of combinatorial proof, M.A.A. 2003, p. 63ff.
   - B. A. Bondarenko, Generalized Pascal Triangles and Pyramids (in Russian), FAN, Tashkent, 1990, ISBN 5-648-00738-8.
   - L. Comtet, Advanced Combinatorics, Reidel, 1974, p. 306.
   - P. Curtz, Intégration numérique des systèmes différentiels à conditions initiales, Centre de Calcul Scientifique de l'Armement, Arcueil, 1969.
   - W. Feller, An Introduction to Probability Theory and Its Application, Vol. 1, 2nd ed. New York: Wiley, p. 36, 1968.
   - R. L. Graham, D. E. Knuth and O. Patashnik, Concrete Mathematics. Addison-Wesley, Reading, MA, 2nd. ed., 1994, p. 155.
   - D. Hök, Parvisa mönster i permutationer [Swedish], 2007.
   - D. E. Knuth, The Art of Computer Programming, Vol. 1, 2nd ed., p. 52.
   - S. K. Lando, Lecture on Generating Functions, Amer. Math. Soc., Providence, R.I., 2003, pp. 60-61.
   - Clifford A. Pickover, A Passion for Mathematics, Wiley, 2005; see p. 71.
   - A. P. Prudnikov, Yu. A. Brychkov and O. I. Marichev, "Integrals and Series", Volume 1: "Elementary Functions", Chapter 4: "Finite Sums", New York, Gordon and Breach Science Publishers, 1986-1992.
   - J. Riordan, An Introduction to Combinatorial Analysis, Wiley, 1958, p. 6.
   - J. Riordan, Combinatorial Identities, Wiley, 1968, p. 2.
   - R. Sedgewick and P. Flajolet, An Introduction to the Analysis of Algorithms, Addison-Wesley, Reading, MA, 1996, p. 143.
   - N. J. A. Sloane and Simon Plouffe, The Encyclopedia of Integer Sequences, Academic Press, 1995 (includes this sequence).
   - D. Wells, The Penguin Dictionary of Curious and Interesting Numbers, pp. 115-8, Penguin Books 1987.
<hr><div align='center'><b><a href='http://oeis.org/A047999'>A047999</a></b>: <i>Sierpiński's [Sierpinski's] triangle (or gasket): triangle, read by rows, formed by reading Pascal's triangle mod 2.</i><br></div>

by _N. J. A. Sloane_

_Keywords_: `nonn,tabl,easy,nice`

_Data_:

$$
\begin{array}{c|ccccccccccc}
n, k & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\hline
0 & 1 &  &  &  &  &  &  &  &  & \\1 & 1 & 1 &  &  &  &  &  &  &  & \\2 & 1 & 0 & 1 &  &  &  &  &  &  & \\3 & 1 & 1 & 1 & 1 &  &  &  &  &  & \\4 & 1 & 0 & 0 & 0 & 1 &  &  &  &  & \\5 & 1 & 1 & 0 & 0 & 1 & 1 &  &  &  & \\6 & 1 & 0 & 1 & 0 & 1 & 0 & 1 &  &  & \\7 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 &  & \\8 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & \\9 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1
\end{array}
$$


_Comments_:
   - Restored the alternative spelling of Sierpinski to facilitate searching for this triangle using regular-expression matching commands in ASCII. - _N. J. A. Sloane_, Jan 18 2016
   - Also triangle giving successive states of cellular automaton generated by "Rule 60" and "Rule 102". - _Hans Havermann_, May 26 2002
   - Also triangle formed by reading triangle of Eulerian numbers (A08292) mod 2. - _Philippe Deléham_, Oct 02 2003
   - Self-inverse when regarded as an infinite lower triangular matrix over GF(2).
   - Start with [1], repeatedly apply the map 0 -> [00/00], 1 -> [10/11] [Allouche and Berthe]
   - Also triangle formed by reading triangles <a href="http://oeis.org/A011117">A011117</a>, <a href="http://oeis.org/A028338">A028338</a>, <a href="http://oeis.org/A039757">A039757</a>, <a href="http://oeis.org/A059438">A059438</a>, <a href="http://oeis.org/A085881">A085881</a>, <a href="http://oeis.org/A086646">A086646</a>, <a href="http://oeis.org/A086872">A086872</a>, <a href="http://oeis.org/A087903">A087903</a>, <a href="http://oeis.org/A010421">A010421</a>9 mod 2 . - _Philippe Deléham_, Jun 18 2005
   - J. H. Conway writes (in Math Forum): at least the first 31 rows give odd-sided constructible polygons (sides 1, 3, 5, 15,17 ... see <a href="http://oeis.org/A001317">A001317</a>). The 1's form a Sierpiński sieve. - M. Dauchez (mdzzdm(AT)yahoo.fr), Sep 19 2005
   - When regarded as an infinite lower triangular matrix, its inverse is a (-1,0,1)-matrix with zeros undisturbed and the nonzero entries in every column form the Prouhet-Thue-Morse sequence (1,-1,-1,1,-1,1,1,-1,...) <a href="http://oeis.org/A010060">A010060</a> (up to relabeling). - _David Callan_, Oct 27 2006
   - Triangle read by rows: antidiagonals of an array formed by successive iterates of running sums mod 2, beginning with (1, 1, 1,...). - _Gary W. Adamson_, Jul 10 2008
   - T(n,k) = <a href="http://oeis.org/A057427">A057427</a>(<a href="http://oeis.org/A143333">A143333</a>(n,k)). [From _Reinhard Zumkeller_, Oct 24 2010]

_Formulae_:
   - Lucas's Theorem is that T(n,k) = 1 if and only if the 1's in the binary expansion of k are a subset of the 1's in the binary expansion of n; or equivalently, k AND NOT n is zero, where AND and NOT are bitwise operators. - _Chai Wah Wu_, Feb 09 2016 and _N. J. A. Sloane_, Feb 10 2016
   - Sum_{k>=0} T(n, k) = <a href="http://oeis.org/A001316">A001316</a>(n) = 2^<a href="http://oeis.org/A000120">A000120</a>(n).
   - T(n,k) = T(n-1,k-1) XOR T(n-1,k), 0<k<n; T(n,0) = T(n,n) = 1. [From _Reinhard Zumkeller_, Dec 13 2009]
   - From _Vladimir Shevelev_, Dec 31 2013 
       - For polynomial {s_n(x)} we have
       - s_0(x)=1; for n>=1, s_n(x) = prod{i=1,...,<a href="http://oeis.org/A000120">A000120</a>(n)}(x^(2^k_i) + 1),
       - if the binary expansion of n is n = sum{i=1,...,<a href="http://oeis.org/A000120">A000120</a>(n)}2^k_i;
       - G.f. sum {n>=0} s_n(x)*z^n = prod{k>=0}(1 + (x^(2^k)+1)z^(2^k)) (0<z<1/x).
       - Let x>1, t>0 be real numbers. Then
       - sum{n>=0}1/s_n(x)^t = prod{k>=0}(1 + 1/(x^(2^k)+1)^t);
       - sum{n>=0}(-1)^<a href="http://oeis.org/A000120">A000120</a>(n)/s_n(x)^t = prod{k>=0}(1 - 1/(x^(2^k)+1)^t).
       - In particular, for t=1, x>1, we have
       - sum{n>=0}(-1)^<a href="http://oeis.org/A000120">A000120</a>(n)/s_n(x) = 1 - 1/x. 

_Cross references_:
   - Cf. <a href="http://oeis.org/A007318">A007318</a>, <a href="http://oeis.org/A054431">A054431</a>, <a href="http://oeis.org/A001317">A001317</a>, <a href="http://oeis.org/A008292">A008292</a>, <a href="http://oeis.org/A083093">A083093</a>, <a href="http://oeis.org/A034931">A034931</a>, <a href="http://oeis.org/A034930">A034930</a>, <a href="http://oeis.org/A008975">A008975</a>, <a href="http://oeis.org/A034932">A034932</a>.
   - From _Johannes W. Meijer_, Jun 05 2011: 
       - <a href="http://oeis.org/A106344">A106344</a> is a skew version of this triangle.
       - Triangle sums (see the comments): <a href="http://oeis.org/A001316">A001316</a> (Row1; Related to Row2), <a href="http://oeis.org/A002487">A002487</a> (Related to Kn11, Kn12, Kn13, Kn21, Kn22, Kn23), <a href="http://oeis.org/A007306">A007306</a> (Kn3, Kn4), <a href="http://oeis.org/A060632">A060632</a> (Fi1, Fi2), <a href="http://oeis.org/A120562">A120562</a> (Ca1, Ca2), <a href="http://oeis.org/A112970">A112970</a> (Gi1, Gi2), <a href="http://oeis.org/A127830">A127830</a> (Ze3, Ze4). 
   - Cf. <a href="http://oeis.org/A166360">A166360</a>, <a href="http://oeis.org/A249133">A249133</a>, <a href="http://oeis.org/A064194">A064194</a>, <a href="http://oeis.org/A227133">A227133</a>.
   - See <a href="http://oeis.org/A038183">A038183</a> for another version.

_Links_:
   - N. J. A. Sloane, <a href="/<a href="http://oeis.org/A047999">A047999</a>/b047999.txt">Table of n, a(n) for n = 0..10584</a> [First 144 rows, flattened; first 50 rows from T. D. Noe].
   - J.-P. Allouche and V. Berthe, <a href="http://www.math.jussieu.fr/~allouche/">Triangle de Pascal, complexite et automates</a>, Bulletin of the Belgian Mathematical Society Simon Stevin 4.1 (1997): 1-24.
   - David Applegate, Omar E. Pol and N. J. A. Sloane, <a href="http://neilsloane.com/doc/tooth.pdf">The Toothpick Sequence and Other Sequences from Cellular Automata</a>, Congressus Numerantium, Vol. 206 (2010), 157-191.
   - J. Baer, <a href="http://ccins.camosun.bc.ca/~jbritton/blaise/bigblaise.html">Explore patterns in Pascal's Triangle</a>
   - A. Bogomolny, <a href="http://www.cut-the-knot.org/ctk/Sierpinski.shtml">Dot Patterns and Sierpinski Gasket</a>
   - Paul Bradley and Peter Rowley, <a href="http://eprints.ma.man.ac.uk/2167/01/covered/MIMS_ep2014_42.pdf">Orbits on k-subsets of 2-transitive Simple Lie-type Groups</a>, 2014.
   - S. Butkevich, <a href="http://www.math.ohio-state.edu/~btk/Pascal">Pascal Triangle Applet</a>
   - David Callan, <a href="http://arxiv.org/abs/math/0610932">Sierpinski's triangle and the Prouhet-Thue-Morse word</a>, arXiv:math/0610932 [math.CO], 2006.
   - B. Cherowitzo, <a href="http://www-math.cudenver.edu/~wcherowi/jcorn5.html">Pascal's Triangle using Clock Arithmetic, Part I</a>
   - B. Cherowitzo, <a href="http://www-math.cudenver.edu/~wcherowi/jcorn6.html">Pascal's Triangle using Clock Arithmetic, Part II</a>
   - C. Cobeli, A. Zaharescu, A game with divisors and absolute differences of exponents, Journal of Difference Equations and Applications, Vol. 20, #11, 2014. Also <a href="http://arxiv.org/abs/1411.1334">arXiv 1411.1334</a>.
   - A. Granville, <a href="http://www.cecm.sfu.ca/organics/papers/granville/support/pascalform.html">Pascal's Triangle Interface</a>
   - R. K. Guy, <a href="http://www.jstor.org/stable/2322249">The strong law of small numbers</a>. Amer. Math. Monthly 95 (1988), no. 8, 697-712.
   - I. Kobayashi et al., <a href="http://www.ies.co.jp/math/java/misc/PascalTriangle/PascalTriangle.html">Pascal's Triangle</a>
   - Y. Moshe, <a href="http://dx.doi.org/10.1016/j.disc.2005.03.022">The distribution of elements in automatic double sequences</a>, Discr. Math., 297 (2005), 91-103.
   - National Curve Bank, <a href="http://curvebank.calstatela.edu/sierpinski/sierpinski.htm">Sierpinski Triangles</a>
   - Hieu D. Nguyen, <a href="http://arxiv.org/abs/1412.3181">A Digital Binomial Theorem</a>, arXiv:1412.3181 [math.NT], 2014.
   - S. Northshield, <a href="http://faculty.plattsburgh.edu/sam.northshield/PasTriMod2v3F.pdf">Sums across Pascal's triangle modulo 2</a>, Congressus Numerantium, 200, pp. 35-52, 2010.
   - F. Richman, <a href="http://math.fau.edu/richman/jscripts.htm">Javascript for computing Pascal's triangle modulo n</a>. Go to this page, then under "Modern Algebra and Other Things", click "Pascal's triangle modulo n".
   - V. Shevelev, <a href="http://arxiv.org/abs/1011.6083">On Stephan's conjectures concerning Pascal triangle modulo 2 and their polynomial generalization</a>, J. of Algebra Number Theory: Advances and Appl., 7 (2012), no.1, 11-29. Also arXiv:1011.6083, 2010.
   - N. J. A. Sloane, <a href="/wiki/Catalog_of_Toothpick_and_CA_Sequences_in_OEIS">Catalog of Toothpick and Cellular Automata Sequences in the OEIS</a>
   - Eric Weisstein's World of Mathematics, <a href="http://mathworld.wolfram.com/SierpinskiSieve.html">Sierpiński Sieve</a>, <a href="http://mathworld.wolfram.com/Rule60.html">Rule 60</a>, <a href="http://mathworld.wolfram.com/Rule102.html">Rule 102</a>
   - <a href="/index/Ce#cell">Index entries for sequences related to cellular automata</a>
   - <a href="/index/Pas#Pascal">Index entries for triangles and arrays related to Pascal's triangle</a>
   - <a href="/index/Si#sieve">Index entries for sequences generated by sieves</a>

_References_:
   - J.-P. Allouche, F. von Haeseler, H.-O. Peitgen, G. Skordev, Linear cellular automata, finite automata and Pascal's triangle, Discrete Applied Mathematics, Volume 66, Issue 1, 22 April 1996, Pages 1-22.
   - Brand, Neal; Das, Sajal; Jacob, Tom. The number of nonzero entries in recursively defined tables modulo primes. Proceedings of the Twenty-first Southeastern Conference on Combinatorics, Graph Theory, and Computing (Boca Raton, FL, 1990). Congr. Numer. 78 (1990), 47--59. MR1140469 (92h:05004).
   - John W. Milnor and James D. Stasheff, Characteristic Classes, Princeton University Press, 1974, pp. 43-49 (sequence appears on p. 46).
   - H.-O. Peitgen, H. Juergens and D. Saupe: Chaos and Fractals (Springer-Verlag 1992), p. 408.
   - Michel Rigo, Formal Languages, Automata and Numeration Systems, 2 vols., Wiley, 2014. Mentions this sequence - see "List of Sequences" in Vol. 2.
   - S. Wolfram, A New Kind of Science, Wolfram Media, 2002; Chapter 3.
<hr><div align='center'><b><a href='http://oeis.org/A029635'>A029635</a></b>: <i>The (1,2)-Pascal triangle (or Lucas triangle) read by rows.</i><br></div>

by _Mohammad K. Azarian_

_Keywords_: `nonn,tabl,nice,easy`

_Data_:

$$
\begin{array}{c|ccccccccccc}
n, k & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\hline
0 & 2 &  &  &  &  &  &  &  &  & \\1 & 1 & 2 &  &  &  &  &  &  &  & \\2 & 1 & 3 & 2 &  &  &  &  &  &  & \\3 & 1 & 4 & 5 & 2 &  &  &  &  &  & \\4 & 1 & 5 & 9 & 7 & 2 &  &  &  &  & \\5 & 1 & 6 & 14 & 16 & 9 & 2 &  &  &  & \\6 & 1 & 7 & 20 & 30 & 25 & 11 & 2 &  &  & \\7 & 1 & 8 & 27 & 50 & 55 & 36 & 13 & 2 &  & \\8 & 1 & 9 & 35 & 77 & 105 & 91 & 49 & 15 & 2 & \\9 & 1 & 10 & 44 & 112 & 182 & 196 & 140 & 64 & 17 & 2
\end{array}
$$


_Comments_:
   - Dropping the first term and changing the boundary conditions to T(n,1)=n, T(n,n-1)=2 (n>=2), T(n,n)=1 yields the number of nonterminal symbols (which generate strings of length k) in a certain context-free grammar in Chomsky normal form that generates all permutations of n symbols. Summation over k (1<=k<=n) results in <a href="http://oeis.org/A003945">A003945</a>. For the number of productions of this grammar: see <a href="http://oeis.org/A090327">A090327</a>. Example: 1; 2, 1; 3, 2, 1; 4, 5, 2, 1; 5, 9, 7, 2, 1; 6, 14, 16, 9, 2, 1; In addition to the example of <a href="http://oeis.org/A090327">A090327</a> we have T(3,3)=#{S}=1, T(3,2)=#{D,E}=2 and T(3,1)=#{A,B,C}=3. - _Peter R. J. Asveld_, Jan 29 2004
   - With T(0,0)=1 : Triangle T(n,k), read by rows, given by (1,0,0,0,0,0,0,0,) DELTA (2,-1,0,0,0,0,0,0,0,...) where DELTA is the operator defined in <a href="http://oeis.org/A084938">A084938</a>. - _Philippe Deléham_, Oct 10 2011
   - For n > 0: T(n,k) = <a href="http://oeis.org/A097207">A097207</a>(n-1,k), 0 <= k < n. - _Reinhard Zumkeller_, Mar 12 2012
   - Much as the original Pascal triangle gives the Fibonacci numbers as sums of its diagonals, this triangle gives the Lucas numbers (<a href="http://oeis.org/A000032">A000032</a>) as sums of its diagonals; see Posamentier & Lehmann (2007). - _Alonso del Arte_, Apr 09 2012
   - For n > 0: T(n,k) = <a href="http://oeis.org/A029600">A029600</a>(n,k) - <a href="http://oeis.org/A007318">A007318</a>(n,k), 0 <= k <= n. - _Reinhard Zumkeller_, Apr 16 2012
   - Riordan array ((2-x)/(1-x), x/(1-x)). - _Philippe Deléham_, Mar 15 2013
   - For a closed-form formula for generalized Pascal's triangle see <a href="http://oeis.org/A228576">A228576</a>.  - _Boris Putievskiy_, Sep 04 2013
   - It appears that for the infinite set of (1,N) Pascal's triangles, the binomial transform of the n-th row (n>0), followed by zeros, is equal to the n-th partial sum of (1, N, N, N, ...). Example: for the (1,2) Pascal's triangle, the binomial transform of the second row followed by zeros, i.e., of (1, 3, 2, 0, 0, 0, ...), is equal to the second partial sum of (1, 2, 2, 2, ...) = (1, 4, 9, 16, ...). - _Gary W. Adamson_, Aug 11 2015
   - Given any (1,N) Pascal triangle, let the binomial transform of the n-th row (n>1) followed by zeros be Q(x). It appears that the binomial transform of the (n-1)-th row prefaced by a zero is Q(n-1). Example: In the (1,2) Pascal triangle the binomial transform of row 3: (1, 4, 5, 2, 0, 0, 0, ...) is <a href="http://oeis.org/A000330">A000330</a> starting with 1: (1, 5, 14, 30, 55, 91, ...). The binomial transform of row 2 prefaced by a zero and followed by zeros, i.e., of (0, 1, 3, 2, 0, 0, 0, ...) is (0, 1, 5, 14, 30, 55, ...). - _Gary W. Adamson_, Sep 28 2015
   - It appears that in the array accompanying each (1,N) Pascal triangle (diagonals of the triangle), the binomial transform of (..., 1, N, 0, 0, 0, ...) preceded by (n-1) zeros generates the n-th row of the array (n>0). Then delete the zeros in the result. Example: in the (1,2) Pascal triangle, row 3 (1, 5, 14, 30, ...) is the binomial transform of (0, 0, 1, 2, 0, 0, 0, ...) with the resulting zeros deleted. - _Gary W. Adamson_, Oct 11 2015

_Formulae_:
   - T(n,k) = T(n-1, k-1) + T(n-1, k) = C(n, k) + C(n-1, k-1) = C(n, k)*(n+k)/n = <a href="http://oeis.org/A007318">A007318</a>(n, k) + <a href="http://oeis.org/A007318">A007318</a>(n-1, k-1) = <a href="http://oeis.org/A061896">A061896</a>(n+k, k) but with T(0, 0)=1 and T(1, 1)=2. Row sum is floor[3^2(n-1)] i.e., <a href="http://oeis.org/A003945">A003945</a>. - _Henry Bottomley_, Apr 26 2002
   - G.f.: 1 + (1 + x*y) / (1 - x - x*y). - _Michael Somos_, Jul 15 2003
   - G.f. for n-th row: (x+2*y)*(x+y)^(n-1).
   - O.g.f. for row n: (1+x)/(1-x)^(n+1). The entries in row n are the nonzero entries in column n of <a href="http://oeis.org/A053120">A053120</a> divided by 2^(n-1). - _Peter Bala_, Aug 14 2008
   - T(2n,n) - T(2n,n+1)= Catalan(n)= <a href="http://oeis.org/A000108">A000108</a>(n). - _Philippe Deléham_, Mar 19 2009
   - With T(0,0) = 1, as in the Example section below, this is known as Vieta's array. The LU factorization of the square array is given by Yang and Leida, equation 20. - _Peter Bala_, Feb 11 2012
   - exp(x) * e.g.f. for row n = e.g.f. for diagonal n. For example, for n = 3 we have exp(x)*(1 + 4*x + 5*x^2/2! + 2*x^3/3!) = 1 + 5*x + 14*x^2/2! + 30*x^3/3! + 55*x^4/4! + .... The same property holds more generally for Riordan arrays of the form ( f(x), x/(1 - x) ). - _Peter Bala_, Dec 22 2014
   - For n>=1: T(n,0) + T(n,1) + T(n,2) = <a href="http://oeis.org/A000217">A000217</a>(n+1). T(n,n-2) = (n-1)^2. - _Bob Selcoe_, Mar 29 2016:

_Cross references_:
   - Cf. <a href="http://oeis.org/A007318">A007318</a>, <a href="http://oeis.org/A034807">A034807</a>, <a href="http://oeis.org/A061896">A061896</a>, <a href="http://oeis.org/A029653">A029653</a> (row-reversed), <a href="http://oeis.org/A157000">A157000</a>.
   - Sums along ascending diagonals give Lucas numbers, n>0.
   - Cf. <a href="http://oeis.org/A090327">A090327</a>, <a href="http://oeis.org/A003945">A003945</a>, <a href="http://oeis.org/A029638">A029638</a>, <a href="http://oeis.org/A228196">A228196</a>, <a href="http://oeis.org/A228576">A228576</a>.
   - Cf. <a href="http://oeis.org/A000330">A000330</a>.
   - Cf. <a href="http://oeis.org/A000217">A000217</a>.

_Links_:
   - Vincenzo Librandi, <a href="/<a href="http://oeis.org/A029635">A029635</a>/b029635.txt">Rows n = 0..102, flattened</a>
   - P. R. J. Asveld, <a href="http://dx.doi.org/10.1016/j.tcs.2005.11.010">Generating all permutations by context-free grammars in Chomsky normal form</a>, Theoretical Computer Science 354 (2006) 118-130.
   - Mohammad K. Azarian, <a href="http://www.m-hikari.com/ijcms/ijcms-2012/45-48-2012/azarianIJCMS45-48-2012.pdf">Identities Involving Lucas or Fibonacci and Lucas Numbers as Binomial Sums</a>, International Journal of Contemporary Mathematical Sciences, Vol. 7, No. 45, 2012, pp. 2221-2227.
   - Arthur T. Benjamin, <a href="http://www.math.hmc.edu/~benjamin/papers/LucasTriangle.pdf">The Lucas triangle recounted</a>
   - Junesang Choi, <a href="http://dx.doi.org/10.1186/1687-1847-2013-236">Determinants of the Laplacians on the n-dimensional unit sphere S^n </a>, Advances in Difference Equations, 236, 2013
   - N. Durov, S. Meljanac, A. Samsarov, Z. Skoda, <a href="http://arxiv.org/abs/math.RT/0604096">A universal formula for representing Lie algebra generators as formal power series with coefficients in the Weyl algebra</a>, arXiv preprint arXiv:math/0604096 [math.RT], 2006.
   - S. Falcon, <a href="http://scik.org/index.php/jmcs/article/view/102">On the Lucas triangle and its relationship with the k-Lucas numbers</a>, Journal of Mathematical and Computational Science, 2 (2012), No. 3, 425-434. - _N. J. A. Sloane_, Sep 20 2012
   - Hans-Christian Herbig, Daniel Herden, Christopher Seaton, <a href="http://arxiv.org/abs/1605.01572">An algebra generated by x-2</a>, arXiv:1605.01572 [math.CO], 2016.
   - M. A. A. Obaid, S. K. Nauman, W. M. Fakieh, C. M. Ringel, <a href="http://www.math.uni-bielefeld.de/~ringel/opus/jeddah.pdf">The numbers of support-tilting modules for a Dynkin algebra</a>, 2014.
   - C. M. Ringel, <a href="http://arxiv.org/abs/1502.06553">The Catalan combinatorics of the hereditary artin algebras</a>, arXiv preprint arXiv:1502.06553 [math.RT], 2015.
   - Neville Robbins, <a href="http://www.fq.math.ca/Papers1/43-2/paper43-2-9.pdf">The Lucas triangle revisited</a>, Fibonacci Quarterly 43, May 2005, pp. 142-148.
   - Lee-Peng Teo, <a href="http://arxiv.org/abs/1412.0758">Zeta function of spheres and real projective spaces</a>, arXiv:1412.0758v1 [math.NT], 2014
   - Y. Yang and J. Leida, <a href="http://www.fq.math.ca/Papers1/42-3/quartyang03_2004.pdf">Pascal decompositions of geometric arrays in matrices</a>, The Fibonacci Quarterly 42.3 (2004)

_References_:
   - B. A. Bondarenko, Generalized Pascal Triangles and Pyramids (in Russian), FAN, Tashkent, 1990, ISBN 5-648-00738-8. English translation published by Fibonacci Association, Santa Clara Univ., Santa Clara, CA, 1993; see p. 25.
   - Alfred S. Posamentier & Ingmar Lehmann, The (Fabulous) Fibonacci Numbers. New York: Prometheus Books (2007): 97 - 105.
   - M. Shubin and S. Andersson, Pseudodifferential Operators and Spectral Theory, Springer Series in Soviet Mathematics, 1987.
<hr><div align='center'><b><a href='http://oeis.org/A029653'>A029653</a></b>: <i>Numbers in (2,1)-Pascal triangle (by row).</i><br></div>

by _Mohammad K. Azarian_

_Keywords_: `nonn,tabl`

_Data_:

$$
\begin{array}{c|ccccccccccc}
n, k & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\hline
0 & 1 &  &  &  &  &  &  &  &  & \\1 & 2 & 1 &  &  &  &  &  &  &  & \\2 & 2 & 3 & 1 &  &  &  &  &  &  & \\3 & 2 & 5 & 4 & 1 &  &  &  &  &  & \\4 & 2 & 7 & 9 & 5 & 1 &  &  &  &  & \\5 & 2 & 9 & 16 & 14 & 6 & 1 &  &  &  & \\6 & 2 & 11 & 25 & 30 & 20 & 7 & 1 &  &  & \\7 & 2 & 13 & 36 & 55 & 50 & 27 & 8 & 1 &  & \\8 & 2 & 15 & 49 & 91 & 105 & 77 & 35 & 9 & 1 & \\9 & 2 & 17 & 64 & 140 & 196 & 182 & 112 & 44 & 10 & 1
\end{array}
$$


_Comments_:
   - Reverse of <a href="http://oeis.org/A029635">A029635</a>. Row sums are <a href="http://oeis.org/A003945">A003945</a>. Diagonal sums are Fib(n+2) = sum_{k=0..floor(n/2)} (2n-3k)C(n-k,n-2k)/(n-k). - _Paul Barry_, Jan 30 2005
   - Riordan array ((1+x)/(1-x), x/(1-x)). The signed triangle (-1)^(n-k)T(n,k) or ((1-x)/(1+x), x/(1+x)) is the inverse of <a href="http://oeis.org/A055248">A055248</a>. Row sums are <a href="http://oeis.org/A003945">A003945</a>. Diagonal sums are F(n+2). - _Paul Barry_, Feb 03 2005
   - Row sums = <a href="http://oeis.org/A003945">A003945</a>: (1, 3, 6, 12, 24, 48, 96...) = (1, 3, 7, 15, 31, 63, 127...) - (0, 0, 1, 3, 7, 15, 31,...); where (1, 3, 7, 15,...) = <a href="http://oeis.org/A000225">A000225</a>. - _Gary W. Adamson_, Apr 22 2007
   - Triangle T(n,k), read by rows, given by (2,-1,0,0,0,0,0,0,0,...) DELTA (1,0,0,0,0,0,0,0,0,...) where DELTA is the operator defined in <a href="http://oeis.org/A084938">A084938</a>. - _Philippe Deléham_, Nov 17 2011
   - <a href="http://oeis.org/A029653">A029653</a> is jointly generated with <a href="http://oeis.org/A208510">A208510</a> as an array of coefficients of polynomials v(n,x): initially, u(1,x)=v(1,x)=1; for n>1, u(n,x)=u(n-1,x)+x*v(n-1)x and v(n,x)=u(n-1,x)+x*v(n-1,x)+1. See the Mathematica section. - _Clark Kimberling_, Feb 28 2012
   - For a closed-form formula for arbitrary left and right borders of Pascal like triangle, see <a href="http://oeis.org/A228196">A228196</a>. - _Boris Putievskiy_, Aug 18 2013
   - For a closed-form formula for generalized Pascal's triangle, see <a href="http://oeis.org/A228576">A228576</a>. - _Boris Putievskiy_, Sep 04 2013

_Formulae_:
   - T(n, k) = C(n-2, k-1) + C(n-2, k) + C(n-1, k-1) + C(n-1, k).
   - G.f.: (1 + x + y + xy)/(1 - y - xy). - _Ralf Stephan_, May 17 2004
   - T(n, k) = (2n-k)*binomial(n, n-k)/n, n, k>0. - _Paul Barry_, Jan 30 2005
   - Sum_{k=0..n} T(n, k)*x^k gives <a href="http://oeis.org/A003945">A003945</a>-<a href="http://oeis.org/A003954">A003954</a> for x = 1, 2, 3, 4, 5, 6, 7, 8, 9, 10. - _Philippe Deléham_, Jul 10 2005
   - T(n, k) = C(n-1, k) + C(n, k) . - _Philippe Deléham_, Jul 10 2005
   - Equals <a href="http://oeis.org/A097806">A097806</a> * <a href="http://oeis.org/A007318">A007318</a>, i.e., the pairwise operator * Pascal's Triangle as infinite lower triangular matrices. - _Gary W. Adamson_, Apr 22 2007
   - From _Peter Bala_, Dec 27 2014: 
       - exp(x) * e.g.f. for row n = e.g.f. for diagonal n. For example, for n = 3 we have exp(x)*(2 + 5*x + 4*x^2/2! + x^3/3!) = 2 + 7*x + 16*x^2/2! + 30*x^3/3! + 50*x^4/4! + .... The same property holds more generally for Riordan arrays of the form ( f(x), x/(1 - x) ).
       - Let M denote the lower unit triangular array with 1's on the main diagonal and 1's everywhere else below the main diagonal except for the first column which consists of the sequence [1,2,2,2,....]. For k = 0,1,2,... define M(k) to be the lower unit triangular block array
       - /I_k 0\
       - \ 0  M/ having the k X k identity matrix I_k as the upper left block; in particular, M(0) = M. Then the present triangle equals the infinite product M(0)*M(1)*M(2)*... (which is clearly well-defined). See the Example section. 

_Cross references_:
   - (d, 1) Pascal triangles for d=3..10: <a href="http://oeis.org/A093560">A093560</a>-5, <a href="http://oeis.org/A093644">A093644</a>-5.
   - Cf. <a href="http://oeis.org/A007318">A007318</a>, <a href="http://oeis.org/A003945">A003945</a>, <a href="http://oeis.org/A208510">A208510</a>, <a href="http://oeis.org/A228196">A228196</a>, <a href="http://oeis.org/A228576">A228576</a>.
   - Cf. <a href="http://oeis.org/A078812">A078812</a>, <a href="http://oeis.org/A106195">A106195</a>.

_Links_:
   - Reinhard Zumkeller, <a href="/<a href="http://oeis.org/A029653">A029653</a>/b029653.txt">Rows n = 0..125 of triangle, flattened</a>
   - Mohammad K. Azarian, <a href="http://www.m-hikari.com/ijcms/ijcms-2012/45-48-2012/azarianIJCMS45-48-2012.pdf">Identities Involving Lucas or Fibonacci and Lucas Numbers as Binomial Sums</a>, International Journal of Contemporary Mathematical Sciences, Vol. 7, No. 45, 2012, pp. 2221-2227.
   - P. Barry, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL16/Barry2/barry231.html">A Note on a Family of Generalized Pascal Matrices Defined by Riordan Arrays</a>, Journal of Integer Sequences, 16 (2013), #13.5.4.
   - Hacene Belbachir and Athmane Benmezai, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL15/Belbachir/bel22.html">Expansion of Fibonacci and Lucas Polynomials: An Answer to Prodinger's Question</a>, Journal of Integer Sequences, Vol. 15 (2012), #12.7.6.
   - B. A. Bondarenko, <a href="http://www.fq.math.ca/pascal.html">Generalized Pascal Triangles and Pyramids</a> (in Russian), FAN, Tashkent, 1990, ISBN 5-648-00738-8. English translation published by Fibonacci Association, Santa Clara Univ., Santa Clara, CA, 1993; see p. 39.
   - H. Hosoya, <a href="http://dx.doi.org/10.1023/A:1019192302062">Pascal's triangle, non-adjacent numbers and D-dimensional atomic orbitals</a>, J. Math. Chemistry, vol. 23, 1998, 169-178.
   - M. Janjic and B. Petkovic, <a href="http://arxiv.org/abs/1301.4550">A Counting Function</a>, arXiv preprint arXiv:1301.4550, 2013. - From N. J. A. Sloane, Feb 13 2013
   - Mark C. Wilson, <a href="http://www.emis.ams.org/journals/DMTCS/pdfpapers/dmAD0129.pdf">Asymptotics for generalized Riordan arrays.</a> International Conference on Analysis of Algorithms DMTCS proc. AD. Vol. 323. 2005.

<hr><div align='center'><b><a href='http://oeis.org/A008459'>A008459</a></b>: <i>Square the entries of Pascal's triangle.</i><br></div>

by _N. J. A. Sloane_

_Keywords_: `nonn,tabl,easy`

_Data_:

$$
\begin{array}{c|ccccccccccc}
n, k & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\hline
0 & 1 &  &  &  &  &  &  &  &  & \\1 & 1 & 1 &  &  &  &  &  &  &  & \\2 & 1 & 4 & 1 &  &  &  &  &  &  & \\3 & 1 & 9 & 9 & 1 &  &  &  &  &  & \\4 & 1 & 16 & 36 & 16 & 1 &  &  &  &  & \\5 & 1 & 25 & 100 & 100 & 25 & 1 &  &  &  & \\6 & 1 & 36 & 225 & 400 & 225 & 36 & 1 &  &  & \\7 & 1 & 49 & 441 & 1225 & 1225 & 441 & 49 & 1 &  & \\8 & 1 & 64 & 784 & 3136 & 4900 & 3136 & 784 & 64 & 1 & \\9 & 1 & 81 & 1296 & 7056 & 15876 & 15876 & 7056 & 1296 & 81 & 1
\end{array}
$$


_Comments_:
   - Number of lattice paths from (0, 0) to (n, n) with steps (1, 0) and (0, 1), having k right turns. - _Emeric Deutsch_, Nov 23 2003
   - Product of <a href="http://oeis.org/A007318">A007318</a> and <a href="http://oeis.org/A105868">A105868</a>. - _Paul Barry_, Nov 15 2005
   - Number of partitions that fit in an n X n box with Durfee square k. - _Franklin T. Adams-Watters_, Feb 20 2006
   - From _Peter Bala_, Oct 23 2008: 
       - Narayana numbers of type B. Row n of this triangle is the h-vector of the simplicial complex dual to an associahedron of type B_n (a cyclohedron) [Fomin & Reading, p. 60]. See <a href="http://oeis.org/A063007">A063007</a> for the corresponding f-vectors for associahedra of type B_n. See <a href="http://oeis.org/A001263">A001263</a> for the h-vectors for associahedra of type A_n. The Hilbert transform of this triangular array is <a href="http://oeis.org/A108625">A108625</a> (see <a href="http://oeis.org/A145905">A145905</a> for the definition of this term).
       - Let A_n be the root lattice generated as a monoid by {e_i - e_j: 0 <= i, j <= n + 1}. Let P(A_n) be the polytope formed by the convex hull of this generating set. Then the rows of this array are the h-vectors of a unimodular triangulation of P(A_n) [Ardila et al.]. <a href="http://oeis.org/A063007">A063007</a> is the corresponding array of f-vectors for these type A_n polytopes. See <a href="http://oeis.org/A086645">A086645</a> for the array of h-vectors for type C_n polytopes and <a href="http://oeis.org/A108558">A108558</a> for the array of h-vectors associated with type D_n polytopes.
   - The n-th row consists of the coefficients of the polynomial P_n(t) = int((1 + t^2 - 2*t*cos(s))^n, s = 0..2*Pi)/Pi/2. For example, when n = 3, we get P_3(t) = t^6 + 9*t^4 + 9*t^2 + 1; the coefficients are 1, 9, 9, 1. - _Theodore Kolokolnikov_, Oct 26 2010
   - Let E(y) = sum {n >= 0} y^n/n!^2 = BesselJ(0, 2*sqrt(-y)). Then this triangle is the generalized Riordan array (E(y), y) with respect to the sequence n!^2 as defined in Wang and Wang. - _Peter Bala_, Jul 24 2013

_Formulae_:
   - E.g.f.: exp((1+y)*x)*BesselI(0, 2*sqrt(y)*x). - _Vladeta Jovovic_, Nov 17 2003
   - G.f.: 1/sqrt(1-2*y-2*x*y+y^2-2*x*y^2+x^2*y^2); g.f. for row n: (1-t)^n P_n[(1+t)/(1-t)] where the P_n's are the Legendre polynomials. - _Emeric Deutsch_, Nov 23 2003
   - G.f. for column k is sum(C(k, j)^2*x^(k+j), j, 0, k)/(1-x)^(2k+1). - _Paul Barry_, Nov 15 2005
   - Column k has g.f. x^k*Legendre_P(k, (1+x)/(1-x))/(1-x)^(k+1) = x^k*sum{j = 0..k, C(k, j)^2*x^j}/(1-x)^(2k+1). - _Paul Barry_, Nov 19 2005
   - Let E be the operator D*x*D, where D denotes the derivative operator d/dx. Then 1/n!^2 * E^n(1/(1-x)) = (row n generating polynomial)/(1-x)^(2n+1) = sum_{k = 0..inf} binomial(n + k, k)^2*x^k. For example, when n = 3 we have 1/3!^2*E^3(1/(1-x)) = (1 + 9*x + 9*x^2 + x^3)/(1-x)^7 = 1/3!^2 * sum_{k = 0..inf} [(k+1)*(k+2)*(k+3)]^2*x^k. - _Peter Bala_, Oct 23 2008
   - G.f.: A(x, y) = Sum_{n >= 0} (2n)!/n!^2 * x^(2n)*y^n/(1-x-x*y)^(2n+1). - _Paul D. Hanna_, Oct 31 2010
   - From _Peter Bala_, Jul 24 2013: 
       - Let E(y) = sum_{n >= 0} y^n/n!^2 = BesselJ(0, 2*sqrt(-y)). Generating function: E(y)*E(x*y) = 1 + (1 + x)*y + (1 + 4*x + x^2)*y^2/2!^2 + (1 + 9*x + 9*x^2 + x^3)*y^3/3!^2 + .... Cf. the unsigned version of <a href="http://oeis.org/A021009">A021009</a> with generating function exp(y)*E(x*y).
       - The n-th power of this array has the generating function E(y)^n*E(x*y). In particular, the matrix inverse <a href="http://oeis.org/A055133">A055133</a> has the generating function E(x*y)/E(y). 
   - T(n,k) = T(n-1,k)*(n+k)/(n-k)+T(n-1,k-1), T(n,0)=T(n,n)=1. - _Vladimir Kruchinin_, Oct 18 2014
   - Observe that the recurrence T(n,k) = T(n-1,k)*(n+k)/(n-k) - T(n-1,k-1), for n >= 2 and 1 <= k < n, with boundary conditions T(n,0) = T(n,n) = 1 gives Pascal's triangle <a href="http://oeis.org/A007318">A007318</a>. - _Peter Bala_, Dec 21 2014
   - n-th row polynomial R(n,x) = [z^n] (1 + (1 + x)*z + x*z^2)^n. Note that 1/n*[z^(n-1)] (1 + (1 + x)*z + x*z^2)^n gives the row polynomials of <a href="http://oeis.org/A001263">A001263</a>. - _Peter Bala_, Jun 24 2015
   - Binomial transform of <a href="http://oeis.org/A105868">A105868</a>. If G(x,t) = 1/sqrt(1 - 2*(1 + t)*x + (1 - t)^2*x^2) denotes the o.g.f. of this array then 1 + x*d/dx(log(G(x,t)) = 1 + (1 + t)*x + (1 + 6*t + t^2)*x^2 + ... is the o.g.f. for <a href="http://oeis.org/A086645">A086645</a>. - _Peter Bala_, Sep 06 2015

_Cross references_:
   - Row sums are in <a href="http://oeis.org/A000984">A000984</a>. Columns 0-3 are <a href="http://oeis.org/A000012">A000012</a>, <a href="http://oeis.org/A000290">A000290</a>, <a href="http://oeis.org/A000537">A000537</a>, <a href="http://oeis.org/A001249">A001249</a>.
   - Cf. <a href="http://oeis.org/A007318">A007318</a>, <a href="http://oeis.org/A055133">A055133</a>, <a href="http://oeis.org/A116647">A116647</a>, <a href="http://oeis.org/A001263">A001263</a>, <a href="http://oeis.org/A086645">A086645</a>, <a href="http://oeis.org/A063007">A063007</a>, <a href="http://oeis.org/A108558">A108558</a>, <a href="http://oeis.org/A108625">A108625</a>(Hilbert transform), <a href="http://oeis.org/A145903">A145903</a>, <a href="http://oeis.org/A181543">A181543</a>, <a href="http://oeis.org/A086645">A086645</a> (logarithmic derivative), <a href="http://oeis.org/A105868">A105868</a> (inverse binomial transform).

_Links_:
   - N. Alexeev, A. Tikhomirov, <a href="http://arxiv.org/abs/1501.04615">Singular Values Distribution of Squares of Elliptic Random Matrices and type-B Narayana Polynomials</a>, arXiv preprint arXiv:1501.04615 [math.PR], 2015.
   - F. Ardila, M. Beck, S. Hosten, J. Pfeifle and K. Seashore, <a href="http://arxiv.org/abs/0809.5123">Root polytopes and growth series of root lattices</a>, arXiv:0809.5123 [math.CO], 2008.
   - P. Bala, <a href="/<a href="http://oeis.org/A008459">A008459</a>/a008459.pdf">A commutative diagram of triangular arrays</a>
   - Paul Barry and Aoife Hennessy, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL15/Barry2/barry190r.html">Generalized Narayana Polynomials, Riordan Arrays, and Lattice Paths</a>, Journal of Integer Sequences, Vol. 15, 2012, #12.4.8.
   - Carl M. Bender and Gerald V. Dunne, <a href="http://dx.doi.org/10.1063/1.527869 ">Polynomials and operator orderings</a>, J. Math. Phys. 29 (1988), 1727-1731.
   - John H. Conway and N. J. A. Sloane, <a href="http://citeseerx.ist.psu.edu/viewdoc/download;jsessionid=E556ADF68595139EB07F425FD5A79F11?doi=10.1.1.39.9899&amp;rep=rep1&amp;type=pdf">Low-dimensional lattices. VII Coordination sequences</a>, Proc. R. Soc. Lond. A (1997) 453, 2369-2389.
   - R. Cori, G. Hetyei, <a href="http://arxiv.org/abs/1306.4628">Counting genus one partitions and permutations</a>, arXiv preprint arXiv:1306.4628 [math.CO], 2013.
   - R. Cori, G. Hetyei, <a href="http://www.dmtcs.org/dmtcs-ojs/index.php/proceedings/article/viewFile/dmAT0130/4488">How to count genus one partitions</a>, FPSAC 2014, Chicago, Discrete Mathematics and Theoretical Computer Science (DMTCS), Nancy, France, 2014, 333-344.
   - Sergey Fomin and Nathan Reading, <a href="http://arxiv.org/abs/math.CO/0505518">Root systems and generalized associahedra</a>, Lecture notes for IAS/Park-City 2004, arXiv:math/0505518 [math.CO], 2005, 2008. [From _Peter Bala_, Oct 23 2008]
   - Abdelkader Necer, <a href="http://jtnb.cedram.org/jtnb-bin/item?id=JTNB_1997__9_2_319_0">Series formelles et produit de Hadamard</a>, Journal de théorie des nombres de Bordeaux, 9:2 (1997), pp. 319-335.
   - Weiping Wang and Tianming Wang, <a href="http://dx.doi.org/10.1016/j.disc.2007.12.037">Generalized Riordan array</a>, Discrete Mathematics, Vol. 308, No. 24, 6466-6500.

_References_:
   - T. K. Petersen, Eulerian Numbers, Birkhauser, 2015, Chapter 12.
   - J. Riordan, An introduction to combinatorial analysis, Dover Publications, Mineola, NY, 2002, page 191, Problem 15. MR1949650
   - P. G. Tait, On the Linear Differential Equation of the Second Order, Proceedings of the Royal Society of Edinburgh, 9 (1876), 93-98 (see p. 97) [From _Tom Copeland_, Sep 09 2010, vol number corrected Sep 10 2010]
<hr><div align='center'><b><a href='http://oeis.org/A037027'>A037027</a></b>: <i>Skew Fibonacci-Pascal triangle read by rows.</i><br></div>

by _Floor van Lamoen_, Jan 01 1999

_Keywords_: `easy,nonn,tabl`

_Data_:

$$
\begin{array}{c|ccccccccccc}
n, k & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\hline
0 & 1 &  &  &  &  &  &  &  &  & \\1 & 1 & 1 &  &  &  &  &  &  &  & \\2 & 2 & 2 & 1 &  &  &  &  &  &  & \\3 & 3 & 5 & 3 & 1 &  &  &  &  &  & \\4 & 5 & 10 & 9 & 4 & 1 &  &  &  &  & \\5 & 8 & 20 & 22 & 14 & 5 & 1 &  &  &  & \\6 & 13 & 38 & 51 & 40 & 20 & 6 & 1 &  &  & \\7 & 21 & 71 & 111 & 105 & 65 & 27 & 7 & 1 &  & \\8 & 34 & 130 & 233 & 256 & 190 & 98 & 35 & 8 & 1 & \\9 & 55 & 235 & 474 & 594 & 511 & 315 & 140 & 44 & 9 & 1
\end{array}
$$


_Comments_:
   - T(n,k) is the number of lattice paths from (0,0) to (n,k) using steps (0,1), (1,0), (2,0). - _Joerg Arndt_, Jun 30 2011
   - T(n,k) is the number of lattice paths of length n, starting from the origin and ending at (n,k), using horizontal steps H=(1,0), up steps U=(1,1) and down steps D=(1,-1), never containing UUU, DD, HD. For instance, for n=4 and k=2, we have the paths; HHUU, HUHU, HUUH, UHHU, UHUH, UUHH, UUDU, UDUU, UUUD. - _Emanuele Munarini_, Mar 15 2011
   - Row sums form Pell numbers <a href="http://oeis.org/A000129">A000129</a>, T(n,0) forms Fibonacci numbers <a href="http://oeis.org/A000045">A000045</a>, T(n,1) forms <a href="http://oeis.org/A001629">A001629</a>. T(n+k,n-k) is polynomial sequence of degree k.
   - T(n,k) gives a convolved Fibonacci sequence (<a href="http://oeis.org/A001629">A001629</a>, <a href="http://oeis.org/A001872">A001872</a>, etc.).
   - As a Riordan array, this is (1/(1-x-x^2),x/(1-x-x^2)). An interesting factorization is (1/(1-x^2),x/(1-x^2))*(1/(1-x),x/(1-x)) [abs(<a href="http://oeis.org/A049310">A049310</a>) times <a href="http://oeis.org/A007318">A007318</a>]. Diagonal sums are the Jacobsthal numbers <a href="http://oeis.org/A001045">A001045</a>(n+1). - _Paul Barry_, Jul 28 2005
   - T(n,k) = T'(n+1,k+1), T' given by [0, 1, 1, -1, 0, 0, 0, 0, 0, 0, 0, ...] DELTA [1, 0, 0, 0, 0, 0, 0, 0, 0, ...] where DELTA is the operator defined in <a href="http://oeis.org/A084938">A084938</a>. - _Philippe Deléham_, Nov 19 2005
   - Equals <a href="http://oeis.org/A049310">A049310</a> * <a href="http://oeis.org/A007318">A007318</a> as infinite lower triangular matrices. - _Gary W. Adamson_, Oct 28 2007
   - This triangle may also be obtained from the coefficients of the Morgan-Voyce polynomials defined by: Mv(x, n) = (x + 1)*Mv(x, n - 1) + Mv(x, n - 2). - _Roger L. Bagula_, Apr 09 2008
   - Row sums are <a href="http://oeis.org/A000129">A000129</a>. - _Roger L. Bagula_, Apr 09 2008
   - Absolute value of coefficients of the characteristic polynomial of tridiagonal matrices with 1's along the main diagonal, and i's along the superdiagonal and the subdiagonal (where i=sqrt(-1), see Mathematica program). - _John M. Campbell_, Aug 23 2011

_Formulae_:
   - T(n, m) = T'(n-1, m)+T'(n-2, m)+T'(n-1, m-1), where T'(n, m) = T(n, m) for n >= 0 and 0< = m< = n and T'(n, m) = 0 otherwise.
   - G.f.: 1/(1 - y - y*z - y^2).
   - G.f. for k-th column: x/(1-x-x^2)^k.
   - T(n, m) = sum(binomial(m+k, m)*binomial(k, n-k-m), k=0..n-m), n>=m>=0, else 0. - _Wolfdieter Lang_, Jun 17 2002
   - T(n, m) = ((n-m+1)*T(n, m-1) + 2*(n+m)*T(n-1, m-1))/(5*m), n >= m >= 1; T(n, 0)= <a href="http://oeis.org/A000045">A000045</a>(n+1); T(n, m)= 0 if n<m. - _Wolfdieter Lang_, Apr 12 2000
   - Chebyshev coefficient triangle (abs(<a href="http://oeis.org/A049310">A049310</a>)) times Pascal's triangle (<a href="http://oeis.org/A007318">A007318</a>) as product of lower triangular matrices. T(n, k)=sum{k=0..n, C((n+j)/2, j)(1+(-1)^(n+j))C(j, k)/2}. - _Paul Barry_, Dec 22 2004
   - Let R(n) = n-th row polynomial in x, with R(0)=1, then R(n+1)/R(n) equals the continued fraction [1+x;1+x, ...(1+x) occurring (n+1) times..., 1+x] for n>=0. - _Paul D. Hanna_, Feb 27 2004
   - T(n,k) = sum{j=0..n, C(n-j,j)*C(n-2*j,k)}; in Egorychev notation, T(n,k)=res_w(1-w-w^2)^(-k-1)*w^(-n+k+1). - _Paul Barry_, Sep 13 2006
   - Sum_{k=0..n} T(n,k)*x^k = <a href="http://oeis.org/A000045">A000045</a>(n+1), <a href="http://oeis.org/A000129">A000129</a>(n+1), <a href="http://oeis.org/A006190">A006190</a>(n+1), <a href="http://oeis.org/A001076">A001076</a>(n+1), <a href="http://oeis.org/A052918">A052918</a>(n), <a href="http://oeis.org/A005668">A005668</a>(n+1), <a href="http://oeis.org/A054413">A054413</a>(n), <a href="http://oeis.org/A041025">A041025</a>(n), <a href="http://oeis.org/A099371">A099371</a>(n+1), <a href="http://oeis.org/A041041">A041041</a>(n), <a href="http://oeis.org/A049666">A049666</a>(n+1), <a href="http://oeis.org/A041061">A041061</a>(n), <a href="http://oeis.org/A140455">A140455</a>(n+1), <a href="http://oeis.org/A041085">A041085</a>(n), <a href="http://oeis.org/A154597">A154597</a>(n+1), <a href="http://oeis.org/A041113">A041113</a>(n) for n = 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15 respectively. - _Philippe Deléham_, Nov 29 2009
   - T((m+1)*n+r-1,m*n+r-1)*r/(m*n+r) = sum(k=1..n, k/n*T((m+1)*n-k-1,m*n-1)*(r+k,r)), n>=m>1.
   - T(n-1,m-1) = m/n*sum(k=1..n-m+1,k*<a href="http://oeis.org/A000045">A000045</a>(k)*T(n-k-1,m-2),k,1,n-m+1), n>=m>1. - _Vladimir Kruchinin_, Mar 17 2011
   - T(n,k) = binomial(n,k)*hypergeom([(k-n)/2, (k-n+1)/2], [-n], -4) for n>=1. - _Peter Luschny_, Apr 25 2016

_Cross references_:
   - <a href="http://oeis.org/A038112">A038112</a>(n)=T(2n, n). <a href="http://oeis.org/A038137">A038137</a> is reflected version. Maximal row entries: <a href="http://oeis.org/A038149">A038149</a>.
   - Diagonal differences are in <a href="http://oeis.org/A055830">A055830</a>. Vertical sums are in <a href="http://oeis.org/A091186">A091186</a>.
   - Cf. <a href="http://oeis.org/A007318">A007318</a>, <a href="http://oeis.org/A049310">A049310</a>, <a href="http://oeis.org/A000129">A000129</a>, <a href="http://oeis.org/A155161">A155161</a>, <a href="http://oeis.org/A122542">A122542</a>, <a href="http://oeis.org/A059283">A059283</a>, <a href="http://oeis.org/A228196">A228196</a>, <a href="http://oeis.org/A228576">A228576</a>.
   - Some other Fibonacci-Pascal triangles: <a href="http://oeis.org/A027926">A027926</a>, <a href="http://oeis.org/A036355">A036355</a>, <a href="http://oeis.org/A074829">A074829</a>, <a href="http://oeis.org/A105809">A105809</a>, <a href="http://oeis.org/A109906">A109906</a>, <a href="http://oeis.org/A111006">A111006</a>, <a href="http://oeis.org/A114197">A114197</a>, <a href="http://oeis.org/A162741">A162741</a>, <a href="http://oeis.org/A228074">A228074</a>.

_Links_:
   - Reinhard Zumkeller, <a href="/<a href="http://oeis.org/A037027">A037027</a>/b037027.txt">Rows n = 0..150 of triangle, flattened</a>
   - H. J. Brothers, Pascal's Prism: Supplementary Material, <a href="http://www.brotherstechnology.com/docs/Pascal&#39;s_Prism_(supplement).pdf">PDF version.</a>
   - T. Mansour, <a href="http://arXiv.org/abs/math.CO/0301157">Generalization of some identities involving the Fibonacci numbers</a>, arXiv:math/0301157 [math.CO], 2003.
   - P. Moree, <a href="http://arXiv.org/abs/math.CO/0311205">Convoluted convolved Fibonacci numbers</a>, arXiv:math/0311205 [math.CO], 2003.
   - Yidong Sun, <a href="http://www.fq.math.ca/Papers1/43-4/paper43-4-10b.pdf">Numerical Triangles and Several Classical Sequences</a>, Fib. Quart. 43, no. 4, Nov. 2005, pp. 359-370.
   - Eric Weisstein's World of Mathematics, <a href="http://mathworld.wolfram.com/Morgan-VoycePolynomials.html">Morgan-Voyce Polynomials.</a>

<hr><div align='center'><b><a href='http://oeis.org/A074909'>A074909</a></b>: <i>Running sum of Pascal's triangle (A007318), or beheaded Pascal's triangle read by beheaded rows.</i><br></div>

by _Wouter Meeussen_, Oct 01 2002

_Keywords_: `easy,nonn,tabl`

_Data_:

$$
\begin{array}{c|ccccccccccc}
n, k & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\hline
0 & 1 &  &  &  &  &  &  &  &  & \\1 & 1 & 2 &  &  &  &  &  &  &  & \\2 & 1 & 3 & 3 &  &  &  &  &  &  & \\3 & 1 & 4 & 6 & 4 &  &  &  &  &  & \\4 & 1 & 5 & 10 & 10 & 5 &  &  &  &  & \\5 & 1 & 6 & 15 & 20 & 15 & 6 &  &  &  & \\6 & 1 & 7 & 21 & 35 & 35 & 21 & 7 &  &  & \\7 & 1 & 8 & 28 & 56 & 70 & 56 & 28 & 8 &  & \\8 & 1 & 9 & 36 & 84 & 126 & 126 & 84 & 36 & 9 & \\9 & 1 & 10 & 45 & 120 & 210 & 252 & 210 & 120 & 45 & 10
\end{array}
$$


_Comments_:
   - This sequence counts the "almost triangular" partitions of n. A partition is triangular if it is of the form 0+1+2+...+k. Examples: 3=0+1+2, 6=0+1+2+3. An "almost triangular" partition is a triangular partition with at most 1 added to each of the parts. Examples: 7 = 1+1+2+3 = 0+2+2+3 = 0+1+3+3 = 0+1+2+4. Thus a(7)=4. 8 = 1+2+2+3 = 1+1+3+3 = 1+1+2+4 = 0+2+3+3 = 0+2+2+4 = 0+1+3+4 so a(8)=6. - _Moshe Shmuel Newman_, Dec 19 2002
   - The "almost triangular" partitions are the ones cycled by the operation of "Bulgarian solitaire", as defined by Martin Gardner.
   - Start with <a href="http://oeis.org/A007318">A007318</a> - I (I = Identity matrix), then delete right border of zeros. - _Gary W. Adamson_, Jun 15 2007
   - Also the number of increasing acyclic functions from {1..n-k+1} to {1..n+2}. A function f is acyclic if for every subset B of the domain the image of B under f does not equal B. For example, T(3,1)=4 since there are exactly 4 increasing acyclic functions from {1,2,3} to {1,2,3,4,5}: f1={(1,2),(2,3),(3,4)}, f2={(1,2),(2,3),(3,5)}, f3={(1,2),(2,4),(3,5)} and f4={(1,3),(2,4),(4,5)}. - _Dennis P. Walsh_, Mar 14 2008
   - Second Bernoulli polynomials are (from <a href="http://oeis.org/A164555">A164555</a> instead of <a href="http://oeis.org/A027641">A027641</a>) B2(n,x) = 1; 1/2, 1; 1/6, 1, 1; 0, 1/2, 3/2, 1; -1/30, 0, 1, 2, 1; 0, -1/6, 0, 5/3, 5/2, 1; ... . Then (B2(n,x)/<a href="http://oeis.org/A002260">A002260</a>) = 1; 1/2, 1/2; 1/6, 1/2, 1/3; 0, 1/4, 1/2, 1/4; -1/30, 0, 1/3, 1/2, 1/5; 0, -1/12, 0, 5/12, 1/2, 1/6; ... . See (from Faulhaber 1631) Jacob Bernoulli Summae Potestatum (sum of powers) in <a href="http://oeis.org/A159688">A159688</a>. Inverse polynomials are 1; -1, 2; 1, -3, 3; -1, 4, -6, 4; ... = <a href="http://oeis.org/A074909">A074909</a> with negative even diagonals. Reflected <a href="http://oeis.org/A053382">A053382</a>/<a href="http://oeis.org/A053383">A053383</a> = reflected B(n,x) = RB(n,x) = 1; -1/2, 1; 1/6, -1, 1; 0, 1/2, -3/2, 1; ... . <a href="http://oeis.org/A074909">A074909</a> is inverse of RB(n,x)/<a href="http://oeis.org/A002260">A002260</a> = 1; -1/2, 1/2; 1/6, -1/2, 1/3; 0, 1/4, -1/2, 1/4; ... . - _Paul Curtz_, Jun 21 2010
   - <a href="http://oeis.org/A054143">A054143</a> is the fission of the polynomial sequence (p(n,x)) given by p(n,x) = x^n + x^(n-1) + ... + x + 1 by the polynomial sequence ((x+1)^n). See <a href="http://oeis.org/A193842">A193842</a> for the definition of fission. - _Clark Kimberling_, Aug 07 2011
   - Reversal of <a href="http://oeis.org/A135278">A135278</a>. - _Philippe Deléham_, Feb 11 2012
   - For a closed-form formula for arbitrary left and right borders of Pascal-like triangles see <a href="http://oeis.org/A228196">A228196</a>. - _Boris Putievskiy_, Aug 19 2013
   - For a closed-form formula for generalized Pascal's triangle see <a href="http://oeis.org/A228576">A228576</a>. - _Boris Putievskiy_, Sep 09 2013
   - From <a href="http://oeis.org/A238363">A238363</a>, the operator equation d/d(:xD:)f(xD)={exp[d/d(xD)]-1}f(xD) = f(xD+1)-f(xD) follows. Choosing f(x) = x^n and using :xD:^n/n! = Binom(xD,n) and (xD)^n = Bell(n,:xD:), the Bell polynomials of <a href="http://oeis.org/A008277">A008277</a>, it follows that the lower triangular matrix [padded <a href="http://oeis.org/A074909">A074909</a>]

_Formulae_:
   - T(n, k) = Sum_{i=0..n} C(i, n-k) = C(n+1, k).
   - Row n has g.f. (1+x)^(n+1)-x^(n+1).
   - T(n, k) = T(n-1, k-1) + T(n-1, k) for k: 0<k<n, T(n, 0)=1, T(n, n)=n. - _Reinhard Zumkeller_, Apr 18 2005
   - T(n,k) = T(n-1,k) + 2*T(n-1,k-1) - T(n-2,k-1) - T(n-2,k-2), T(0,0)=1, T(1,0)=1, T(1,1)=2, T(n,k)=0 if k<0 or if k>n. - _Philippe Deléham_, Dec 27 2013
   - G.f. for column k (with leading zeros): x^(k-1)*(1/(1-x)^(k+1)-1), k >= 0. - _Wolfdieter Lang_, Nov 04 2014
   - Up(n, x+y) = (Up(.,x)+ y)^n = Sum_{k=0..n} binomial(n,k) Up(k,x)*y^(n-k), where Up(n,x) = ((x+1)^(n+1)-x^(n+1)) / (n+1) = P(n,x)/(n+1) with P(n,x) the n-th row polynomial of this entry. dUp(n,x)/dx = n * Up(n-1,x) and dP(n,x)/dx = (n+1)*P(n-1,x). - _Tom Copeland_, Nov 14 2014
   - The o.g.f. GF(x,t) = x / ((1-t*x)*(1-(1+t)x)) = x + (1+2t)*x^2 + (1+3t+3t^2)*x^3 + ... has the inverse GFinv(x,t) = (1+(1+2t)x-sqrt(1+(1+2t)*2x+x^2))/(2t(1+t)x) in x about 0, which generates the row polynomials (mod row signs) of <a href="http://oeis.org/A033282">A033282</a>. The reciprocal of the o.g.f., i.e., x/GF(x,t), gives the free cumulants (1, -(1+2t) , t(1+t) , 0, 0, ...) associated with the moments defined by GFinv, and, in fact, these free cumulants generate these moments through the noncrossing partitions of <a href="http://oeis.org/A134264">A134264</a>. The associated e.g.f. and relations to Grassmannians are described in <a href="http://oeis.org/A248727">A248727</a>, whose polynomials are the basis for an Appell sequence of polynomials that are umbral compositional inverses of the Appell sequence formed from this entry's polynomials (distinct from the one described in the comments above, without the normalizing reciprocal). - _Tom Copeland_, Jan 07 2015

_Cross references_:
   - Cf. <a href="http://oeis.org/A007318">A007318</a>, <a href="http://oeis.org/A181971">A181971</a>, <a href="http://oeis.org/A228196">A228196</a>, <a href="http://oeis.org/A228576">A228576</a>.
   - Row sums are <a href="http://oeis.org/A000225">A000225</a>, diagonal sums are <a href="http://oeis.org/A052952">A052952</a>.
   - The number of acyclic functions is <a href="http://oeis.org/A058127">A058127</a>.
   - Cf. <a href="http://oeis.org/A008292">A008292</a>, <a href="http://oeis.org/A090582">A090582</a>, <a href="http://oeis.org/A019538">A019538</a>, <a href="http://oeis.org/A049019">A049019</a>, <a href="http://oeis.org/A133314">A133314</a>, <a href="http://oeis.org/A135278">A135278</a>, <a href="http://oeis.org/A133437">A133437</a>, <a href="http://oeis.org/A111785">A111785</a>, <a href="http://oeis.org/A126216">A126216</a>, <a href="http://oeis.org/A134685">A134685</a>, <a href="http://oeis.org/A133932">A133932</a>, <a href="http://oeis.org/A248727">A248727</a>, <a href="http://oeis.org/A033282">A033282</a>, <a href="http://oeis.org/A134264">A134264</a>.
   - Cf. <a href="http://oeis.org/A000027">A000027</a>, <a href="http://oeis.org/A000217">A000217</a>, <a href="http://oeis.org/A000292">A000292</a>, <a href="http://oeis.org/A000332">A000332</a>, <a href="http://oeis.org/A000389">A000389</a>, <a href="http://oeis.org/A000579">A000579</a>, <a href="http://oeis.org/A000580">A000580</a>, <a href="http://oeis.org/A000581">A000581</a>, <a href="http://oeis.org/A000582">A000582</a>.

_Links_:
   - Reinhard Zumkeller, <a href="/<a href="http://oeis.org/A074909">A074909</a>/b074909.txt">Rows n=0..150 of triangle, flattened</a>
   - T. Copeland, <a href="https://tcjpn.wordpress.com/2014/12/23/appell-ops-cumulants-noncrossing-partitions-dyck-lattice-paths-and-inversion">Appell ops, cumulants, noncrossing partitions, Dyck lattice paths, and inversion</a>
   - T. Copeland, <a href="http://tcjpn.wordpress.com/2015/12/21/generators-inversion-and-matrix-binomial-and-integral-transforms/">Generators, Inversion, and Matrix, Binomial, and Integral Transforms</a>
   - J. R. Griggs, <a href="http://dx.doi.org/10.1006/aama.1998.0597">The Cycling of Partitions and Compositions under Repeated Shifts</a>, Advances in Applied Mathematics, Volume 21, Issue 2, August 1998, Pages 205-227.
   - P. Olver, <a href="http://www.math.umn.edu/~olver/di_/contact.pdf">The canonical contact form</a> pg. 7.
   - D. P. Walsh, <a href="http://www.mtsu.edu/~dwalsh/ACYCINCR.pdf">A short note on increasing acyclic functions</a>

<hr><div align='center'><b><a href='http://oeis.org/A132440'>A132440</a></b>: <i>Infinitesimal Pascal matrix: generator (lower triangular matrix representation) of the Pascal matrix, the classical operator xDx, iterated Laguerre transforms, associated matrices of the list partition transform and general Euler transformation for sequences.</i><br></div>

by _Tom Copeland_, Nov 13 2007, Nov 15 2007, Nov 22 2007, Dec 02 2007

_Keywords_: `easy,nonn,tabl`

_Data_:

$$
\begin{array}{c|ccccccccccc}
n, k & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\hline
0 & 0 &  &  &  &  &  &  &  &  & \\1 & 1 & 0 &  &  &  &  &  &  &  & \\2 & 0 & 2 & 0 &  &  &  &  &  &  & \\3 & 0 & 0 & 3 & 0 &  &  &  &  &  & \\4 & 0 & 0 & 0 & 4 & 0 &  &  &  &  & \\5 & 0 & 0 & 0 & 0 & 5 & 0 &  &  &  & \\6 & 0 & 0 & 0 & 0 & 0 & 6 & 0 &  &  & \\7 & 0 & 0 & 0 & 0 & 0 & 0 & 7 & 0 &  & \\8 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 8 & 0 & \\9 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 9 & 0
\end{array}
$$


_Comments_:
   - Let M(t) = exp(t*T) = Lim_{n->infinity} (1 + t*T/n)^n.
   - Pascal matrix = [ binomial(n,k) ] = M(1) = exp(T), truncating the series gives the n X n submatrices.
   - Inverse Pascal matrix = M(-1) = exp(-T) = matrix for inverse binomial transform.
   - A(j) = T^j / j! equals the matrix [bin(n,k) * delta(n-k-j)] where delta(n) = 1 if n=0 and vanishes otherwise (Kronecker delta); i.e., A(j) is a matrix with all the terms 0 except for the j-th lower (or main for j=0) diagonal which equals that of the Pascal triangle. Hence the A(j)'s form a linearly independent basis for all matrices of the form [binomial(n,k) d(n-k)] which include as a subset the invertible associated matrices of the list partition transform (LPT) of <a href="http://oeis.org/A133314">A133314</a>.
   - For sequences with b(0) = 1, umbrally,
   - M[b(.)] = exp(b(.)*T) = [ binomial(n,k) * b(n-k) ] = matrices associated to b by LPT.
   - [M[b(.)]]^(-1) = exp(c(.)*T) = [ binomial(n,k) * c(n-k) ] = matrices associated to c, where c = LPT(b) . Or,
   - [M[b(.)]]^(-1) = exp[LPT(b(.))*T] = LPT[M(b(.))] = M[LPT(b(.))]= M[c(.)].
   - This is related to xDx, the iterated Laguerre transform and the general Euler transformation of a sequence through the comments in <a href="http://oeis.org/A132013">A132013</a> and <a href="http://oeis.org/A132014">A132014</a> and the relation [Sum_{k=0..n} binomial(n,k) * b(n-k) * d(k)] = M(b)*d, (n-th term). See also <a href="http://oeis.org/A132382">A132382</a>.
   - If b(n,x) is a binomial type Sheffer sequence, then M[b(.,x)]*s(y) = s(x+y) when s(y) = (s(0,y),s(1,y),s(2,y),...) is an array for a Sheffer sequence with the same delta operator as b(n,x) and [M[b(.,x)]]^(-1) is given by the formulas above with b(n) replaced by b(n,x) as b(0,x)=1 for a binomial type Sheffer sequence.

_Formulae_:
   - T = log(P) with the Pascal matrix P:=<a href="http://oeis.org/A007318">A007318</a>. This should be read as T_N = log(P_N) with P_N the N X N matrix P, N>=2. Because P_N is lower triangular with all diagonal elements 1, the series log(1_N-(1_N-P_N)) stops after N-1 terms because (1_N-P_N)^N is the 0_N-matrix. - _Wolfdieter Lang_, Oct 14 2010
   - Given a polynomial sequence p_n(x) with p_0(x)=1 and the lowering and raising operators L and R defined by L p_n(x) = n * p_(n-1)(x) and R p_n(x) = p_(n+1)(x), the matrix T represents the action of R*L*R in the p_n(x) basis. For p_n(x) = x^n, L = D = d/dx and R = x. For p_n(x) = x^n/n!, L = DxD and R = D^(-1). - _Tom Copeland_, Oct 25 2012
   - From _Tom Copeland_, Apr 26 2014: 
       - A) T = exp(<a href="http://oeis.org/A238385">A238385</a>-I) - I
       - B)   = [St1]*P*[St2] - I
       - C)   = [St1]*P*[St1]^(-1) - I
       - D)   = [St2]^(-1)*P*[St2] - I
       - E)   = [St2]^(-1)*P*[St1]^(-1) - I
       - where P=<a href="http://oeis.org/A007318">A007318</a>, [St1]=padded <a href="http://oeis.org/A008275">A008275</a> just as [St2]=<a href="http://oeis.org/A048993">A048993</a>=padded <a href="http://oeis.org/A008277">A008277</a>, and I=identity matrix. 
   - From _Robert Israel_, Oct 02 2015: 
       - G.f. Sum_{k >= 1} k x^((k+3/2)^2/2 - 17/8) is related to Jacobi theta functions.
       - If 8*n+17 = y^2 is a square, then a(n) = (y-3)/2, otherwise a(n) = 0. 


_Links_:
   - Robert Israel, <a href="/<a href="http://oeis.org/A132440">A132440</a>/b132440.txt">Table of n, a(n) for n = 0..10000</a>
   - W. A. Al-Salam, <a href="http://projecteuclid.org/euclid.dmj/1077375084">Operational representations for the Laguerre and other polynomials</a>, Duke Math. Jour., vol 31 (1964), pp. 127-142
   - T. Copeland, <a href="http://tcjpn.wordpress.com/2015/09/09/fractional-calculus-gamma-classes-the-riemann-zeta-function-and-an-appell-pair-of-sequences/">Fractional Calculus, Gamma Classes, the Riemann Zeta Function, and an Appell Pair of Sequences</a>
   - T. Copeland, <a href="http://tcjpn.wordpress.com/2014/08/03/goin-with-the-flow-logarithm-of-the-derivative/">Goin' with the Flow: Logarithm of the Derivative Operator</a>
   - T. Copeland, <a href="http://tcjpn.wordpress.com/2012/11/29/infinigens-the-pascal-pyramid-and-the-witt-and-virasoro-algebras/">Infinigens, the Pascal Pyramid, and the Witt and Virasoro Algebras</a>
   - T. Copeland, <a href="http://tcjpn.files.wordpress.com/2011/11/the-mt-bell-dobinski-chgf.pdf">The Inverse Mellin Transform, Bell Polynomials, a Generalized Dobinski Relation, and the Confluent Hypergeometric Functions (pdf)</a>
   - T. Copeland, <a href="http://tcjpn.wordpress.com/2008/06/12/mathemagical-forests/">Mathemagical Forests</a>
   - T. Copeland, <a href="http://tcjpn.wordpress.com/2015/09/16/mellin-transform-interpolation-of-differential-operators/">Mellin Interpolation of Differential Ops and Associated Infinigens and Appell Polynomials: The Ordered, Laguerre, and Scherk-Witt-Lie Diff Ops</a>
   - G. Hetyei, <a href="http://arxiv.org/abs/0909.4352">Meixner polynomials of the second kind and quantum algebras representing su(1,1)</a>, arXiv preprint arXiv:0909.4352 [math.QA], 2009 (cf. Viennot's Laguerre histoires)
   - K. A. Penson, P. Blasiak, A. Horzela, G.H.E. Duchamp and A. I. Solomon, <a href="http://arxiv.org/abs/0904.0369">Laguerre-type derivatives: Dobinski relations and combinatorial identities</a>, arXiv:0904.0369 [math-ph], 2009.
   - K. A. Penson, P. Blasiak, A. Horzela, G.H.E. Duchamp and A. I. Solomon, <a href="http://dx.doi.org/10.1063/1.3155380">Laguerre-type derivatives: Dobinski relations and combinatorial identities</a>, Journal of Mathematical Physics vol.50, (2009) 083512.

<hr><div align='center'><b><a href='http://oeis.org/A083093'>A083093</a></b>: <i>Triangle formed by reading Pascal's triangle (A007318) mod 3.</i><br></div>

by _Benoit Cloitre_, Apr 22 2003

_Keywords_: `easy,nonn,tabl`

_Data_:

$$
\begin{array}{c|ccccccccccc}
n, k & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\hline
0 & 1 &  &  &  &  &  &  &  &  & \\1 & 1 & 1 &  &  &  &  &  &  &  & \\2 & 1 & 2 & 1 &  &  &  &  &  &  & \\3 & 1 & 0 & 0 & 1 &  &  &  &  &  & \\4 & 1 & 1 & 0 & 1 & 1 &  &  &  &  & \\5 & 1 & 2 & 1 & 1 & 2 & 1 &  &  &  & \\6 & 1 & 0 & 0 & 2 & 0 & 0 & 1 &  &  & \\7 & 1 & 1 & 0 & 2 & 2 & 0 & 1 & 1 &  & \\8 & 1 & 2 & 1 & 2 & 1 & 2 & 1 & 2 & 1 & \\9 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1
\end{array}
$$


_Comments_:
   - Start with [1], repeatedly apply the map 0 -> [000/000/000], 1 -> [111/120/100], 2 -> [222/210/200] . [_Philippe Deléham_, Apr 16 2009]

_Formulae_:
   - T(i, j) = binomial(i, j) mod 3.
   - T(n+1,k) = (T(n,k) + T(n,k-1)) mod 3. - _Reinhard Zumkeller_, Jul 11 2013

_Cross references_:
   - Cf. <a href="http://oeis.org/A007318">A007318</a>, <a href="http://oeis.org/A051638">A051638</a> (row sums), <a href="http://oeis.org/A090044">A090044</a>, <a href="http://oeis.org/A047999">A047999</a>, <a href="http://oeis.org/A034931">A034931</a>, <a href="http://oeis.org/A034930">A034930</a>, <a href="http://oeis.org/A008975">A008975</a>, <a href="http://oeis.org/A034932">A034932</a>, <a href="http://oeis.org/A062296">A062296</a>, <a href="http://oeis.org/A006047">A006047</a>.
   - Cf. <a href="http://oeis.org/A006996">A006996</a> (central terms), <a href="http://oeis.org/A173019">A173019</a>, <a href="http://oeis.org/A206424">A206424</a>, <a href="http://oeis.org/A227428">A227428</a>.

_Links_:
   - Reinhard Zumkeller, <a href="/<a href="http://oeis.org/A083093">A083093</a>/b083093.txt">Rows n = 0..120 of triangle, flattened</a>
   - Lin Jiu, Christophe Vignat, <a href="http://arxiv.org/abs/1602.04149">On Binomial Identities in Arbitrary Bases</a>, arXiv:1602.04149 [math.CO], 2016.
   - Y. Moshe, <a href="http://dx.doi.org/10.1016/S0022-314X(03)00103-3">The density of 0's in recurrence double sequences</a>, J. Number Theory, 103 (2003), 109-121.
   - Y. Moshe, <a href="http://dx.doi.org/10.1016/j.disc.2005.03.022">The distribution of elements in automatic double sequences</a>, Discr. Math., 297 (2005), 91-103.
   - <a href="/index/Pas#Pascal">Index entries for triangles and arrays related to Pascal's triangle</a>

_References_:
   - Michel Rigo, Formal Languages, Automata and Numeration Systems, 2 vols., Wiley, 2014. Mentions this sequence - see "List of Sequences" in Vol. 2.
<hr><div align='center'><b><a href='http://oeis.org/A130595'>A130595</a></b>: <i>Triangle read by rows: lower triangular matrix which is inverse to Pascal's triangle (A007318) regarded as a lower triangular matrix.</i><br></div>

by _Philippe Deléham_, Jun 17 2007

_Keywords_: `sign,nice,tabl`

_Data_:

$$
\begin{array}{c|ccccccccccc}
n, k & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\hline
0 & 1 &  &  &  &  &  &  &  &  & \\1 & -1 & 1 &  &  &  &  &  &  &  & \\2 & 1 & -2 & 1 &  &  &  &  &  &  & \\3 & -1 & 3 & -3 & 1 &  &  &  &  &  & \\4 & 1 & -4 & 6 & -4 & 1 &  &  &  &  & \\5 & -1 & 5 & -10 & 10 & -5 & 1 &  &  &  & \\6 & 1 & -6 & 15 & -20 & 15 & -6 & 1 &  &  & \\7 & -1 & 7 & -21 & 35 & -35 & 21 & -7 & 1 &  & \\8 & 1 & -8 & 28 & -56 & 70 & -56 & 28 & -8 & 1 & \\9 & -1 & 9 & -36 & 84 & -126 & 126 & -84 & 36 & -9 & 1
\end{array}
$$


_Comments_:
   - Triangle T(n,k), read by rows,given by [-1,0,0,0,0,0,0,0,...] DELTA [1,0,0,0,0,0,0,0,...] where DELTA is the operator defined in <a href="http://oeis.org/A084938">A084938</a>.
   - Coefficients of the polynomials generated by the e.g.f. exp(x*t)*exp(-t). - _Peter Luschny_, Jul 13 2009
   - Riordan array (1/(1+x), x/(1+x)). - _Philippe Deléham_, Nov 29 2009
   - Multiplication of a sequence (written as column vector) by this matrix (to the left) yields the inverse Binomial transform of the sequence. - _M. F. Hasler_, Nov 01 2014

_Formulae_:
   - T(n,k) = (-1)^(n-k)*binomial(n,k) = (-1)^(n-k)*<a href="http://oeis.org/A007318">A007318</a>(n,k).
   - T(n,k) = T(n-1,k-1)-T(n-1,k). - _Philippe Deléham_, Oct 10 2011.
   - G.f.: 1/(1+x-x*y). - _R. J. Mathar_, Aug 11 2015 [corrected by _Anders Claesson_, Nov 28 2015]
   - Conjecture from Dale Gerdemann, Nov 28 2015: T(n,k) = (n-k+1)*T(n-1,k-1) + (k-1)*T(n-1,k). Proof from Anders Claesson, Nov 29 2015: It follows from T(n,k) = T(n-1,k-1) - T(n-1,k) and n*T(n-1,k-1) = k*T(n,k) that (n-k+1)*T(n-1,k-1) + (k-1)*T(n-1,k) = n*T(n-1,k-1) - (k-1)*T(n-1,k-1) + (k-1)*T(n-1,k) = n*T(n-1,k-1) - (k-1)*(T(n-1,k-1) - T(n-1,k)) = n*T(n-1,k-1) - (k-1)*T(n,k) = n*T(n-1,k-1) - k*T(n,k) + T(n,k) = T(n,k). QED

_Cross references_:
   - Cf. <a href="http://oeis.org/A007318">A007318</a>.

_Links_:
   - Reinhard Zumkeller, <a href="/<a href="http://oeis.org/A130595">A130595</a>/b130595.txt">Rows n = 0..125 of triangle, flattened</a>
   - Tian-Xiao He and Renzo Sprugnoli, <a href="http://dx.doi.org/10.1016/j.disc.2008.11.021">Sequence characterization of Riordan arrays</a>, Discrete Math. 309 (2009), no. 12, 3962-3974. [_N. J. A. Sloane_, Nov 26 2011]
   - <a href="/index/Pas#Pascal">Index entries for triangles and arrays related to Pascal's triangle</a>


## Sandbox

In [13]:
from requests import get
from IPython.display import Markdown

In [7]:
code=45

payload = {"fmt": "json", 
           #"q": "id:A{:06d}".format(code)}
           "q": "fibonacci",
           "start": 0}

doc_result = get("https://oeis.org/search", params=payload,)
doc = doc_result.json()

In [8]:
doc_result.url

'https://oeis.org/search?fmt=json&q=fibonacci&start=0'

In [9]:
doc.keys()

dict_keys(['greeting', 'start', 'results', 'query', 'count'])

In [11]:
results = doc['results']
results[0]['comment'][:10]

["Also sometimes called Lamé's sequence.",
 "F(n+2) = number of binary sequences of length n that have no consecutive 0's.",
 'F(n+2) = number of subsets of {1,2,...,n} that contain no consecutive integers.',
 'F(n+1) = number of tilings of a 2 X n rectangle by 2 X 1 dominoes.',
 'F(n+1) = number of matchings (i.e., Hosoya index) in a path graph on n vertices: F(5)=5 because the matchings of the path graph on the vertices A, B, C, D are the empty set, {AB}, {BC}, {CD} and {AB, CD}. - _Emeric Deutsch_, Jun 18 2001',
 'F(n) = number of compositions of n+1 with no part equal to 1. [Cayley, Grimaldi]',
 'Positive terms are the solutions to z = 2*x*y^4 + (x^2)*y^3 - 2*(x^3)*y^2 - y^5 - (x^4)*y + 2*y for x,y >= 0 (Ribenboim, page 193). When x=F(n), y=F(n + 1) and z>0 then z=F(n + 1).',
 'For Fibonacci search see Knuth, Vol. 3; Horowitz and Sahni; etc.',
 "F(n) is the diagonal sum of the entries in Pascal's triangle at 45 degrees slope. - _Amarnath Murthy_, Dec 29 2001",
 'F(n+1) is the numbe

In [14]:
Markdown(pretty_print(doc['results'][0], 
                      comment=lambda i,c: "binomial" in c,
                      formula=lambda i,f: "theorem" in f,
                      link=lambda i,l: "paul barry" in l,
                      reference=lambda i,r: "riordan" in r))

<div align='center'><b><a href='http://oeis.org/A000045'>A000045</a></b>: <i>Fibonacci numbers: F(n) = F(n-1) + F(n-2) with F(0) = 0 and F(1) = 1.</i><br></div>

by _N. J. A. Sloane_, 1964

_Keywords_: `core,nonn,nice,easy,hear,changed`

_Data_:

$$
\begin{array}{c|cccccccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 & 15 & 16 & 17 & 18 & 19 \\
\hline
A000045(n) & 0 & 1 & 1 & 2 & 3 & 5 & 8 & 13 & 21 & 34 & 55 & 89 & 144 & 233 & 377 & 610 & 987 & 1597 & 2584 & 4181
\end{array}
$$


_Comments_:
   - Fib(n+2) = Sum_{k=0..n} binomial(floor((n+k)/2),k), row sums of <a href="http://oeis.org/A046854">A046854</a>. - _Paul Barry_, Mar 11 2003
   - The sequence F(n) is the binomial transformation of the alternating sequence (-1)^(n-1)*F(n), whereas the sequence F(n+1) is the binomial transformation of the alternating sequence (-1)^n*F(n-1). Both of these facts follow easily from the equalities a(n;1)=F(n+1) and b(n;1)=F(n) where a(n;d) and b(n;d) are so-called "delta-Fibonacci" numbers as defined in comments to <a href="http://oeis.org/A014445">A014445</a> (see also the papers of Witula et al.). - _Roman Witula_, Jul 24 2012
   - From _Tom Copeland_, Nov 02 2014: 
       - Let P(x) = x/(1+x) with comp. inverse Pinv(x) = x/(1-x) = -P[-x], and C(x)= [1-sqrt(1-4x)]/2, an o.g.f. for the shifted Catalan numbers <a href="http://oeis.org/A000108">A000108</a>, with inverse Cinv(x) = x * (1-x).
       - Fin(x) = P[C(x)] = C(x)/[1 + C(x)] is an o.g.f. for the Fine numbers, <a href="http://oeis.org/A000957">A000957</a> with inverse Fin^(-1)(x) = Cinv[Pinv(x)] = Cinv[-P(-x)].
       - Mot(x) = C[P(x)] = C[-Pinv(-x)] gives an o.g.f. for shifted <a href="http://oeis.org/A005043">A005043</a>, the Motzkin or Riordan numbers with comp. inverse Mot^(-1)(x) = Pinv[Cinv(x)] = (x - x^2) / (1 - x + x^2) (cf. <a href="http://oeis.org/A057078">A057078</a>).
       - BTC(x) = C[Pinv(x)] gives <a href="http://oeis.org/A007317">A007317</a>, a binomial transform of the Catalan numbers, with BTC^(-1)(x) = P[Cinv(x)].
       - Fib(x) = -Fin[Cinv(Cinv(-x))] = -P[Cinv(-x)] = x + 2 x^2 + 3 x^3 + 5 x^4 + ... = (x+x^2)/[1-x-x^2] is an o.g.f. for the shifted Fibonacci sequence <a href="http://oeis.org/A000045">A000045</a>, so the comp. inverse is Fib^(-1)(x) = -C[Pinv(-x)] = -BTC(-x) and Fib(x) = -BTC^(-1)(-x).
       - Generalizing to P(x,t) = x /(1 + t*x) and Pinv(x,t) = x /(1 - t*x) = -P(-x,t) gives other relations to lattice paths, such as the o.g.f. for <a href="http://oeis.org/A091867">A091867</a>, C[P[x,1-t]], and that for <a href="http://oeis.org/A104597">A104597</a>, Pinv[Cinv(x),t+1].

_Formulae_:
   - A sufficient condition for F(m) to be divisible by a prime p is (p - 1) divides m, if p == 1 or 4 (mod 5); (p + 1) divides m, if p == 2 or 3 (mod 5); or 5 divides m, if p = 5. (This is essentially Theorem 180 in Hardy and Wright.) - Fred W. Helenius (fredh(AT)ix.netcom.com), Jun 29 2001
   - _Kurmang. Aziz. Rashid_, Feb 21 2004, makes 4 conjectures and gives 3 theorems:
   - Theorem 1: for n>=0, {F(n+3)^ 2 - F(n+1)^ 2}/F(n+2)={F(n+3)+ F(n+1)}.
   - Theorem 2: for n>=0, F(n+10) = 11*F(n+5) + F(n).
   - Theorem 3: for n>=6, F(n) = 4*F(n-3) + F(n-6).

_Cross references_:
   - Cf. <a href="http://oeis.org/A039834">A039834</a> (signed Fibonacci numbers), <a href="http://oeis.org/A001690">A001690</a> (complement), <a href="http://oeis.org/A000213">A000213</a>, <a href="http://oeis.org/A000288">A000288</a>, <a href="http://oeis.org/A000322">A000322</a>, <a href="http://oeis.org/A000383">A000383</a>, <a href="http://oeis.org/A060455">A060455</a>, <a href="http://oeis.org/A030186">A030186</a>, <a href="http://oeis.org/A020695">A020695</a>, <a href="http://oeis.org/A020701">A020701</a>, <a href="http://oeis.org/A071679">A071679</a>, <a href="http://oeis.org/A099731">A099731</a>, <a href="http://oeis.org/A100492">A100492</a>, <a href="http://oeis.org/A094216">A094216</a>, <a href="http://oeis.org/A094638">A094638</a>, <a href="http://oeis.org/A000108">A000108</a>, <a href="http://oeis.org/A101399">A101399</a>, <a href="http://oeis.org/A101400">A101400</a>, <a href="http://oeis.org/A001611">A001611</a>, <a href="http://oeis.org/A000071">A000071</a>, <a href="http://oeis.org/A157725">A157725</a>, <a href="http://oeis.org/A001911">A001911</a>, <a href="http://oeis.org/A157726">A157726</a>, <a href="http://oeis.org/A006327">A006327</a>, <a href="http://oeis.org/A157727">A157727</a>, <a href="http://oeis.org/A157728">A157728</a>, <a href="http://oeis.org/A157729">A157729</a>, <a href="http://oeis.org/A167616">A167616</a>, <a href="http://oeis.org/A059929">A059929</a>, <a href="http://oeis.org/A144152">A144152</a>, <a href="http://oeis.org/A152063">A152063</a>, <a href="http://oeis.org/A114690">A114690</a>, <a href="http://oeis.org/A003893">A003893</a>, <a href="http://oeis.org/A000032">A000032</a>, <a href="http://oeis.org/A060441">A060441</a>, <a href="http://oeis.org/A000930">A000930</a>, <a href="http://oeis.org/A003269">A003269</a>, <a href="http://oeis.org/A000957">A000957</a>, <a href="http://oeis.org/A057078">A057078</a>, <a href="http://oeis.org/A007317">A007317</a>, <a href="http://oeis.org/A091867">A091867</a>, <a href="http://oeis.org/A104597">A104597</a>, <a href="http://oeis.org/A249548">A249548</a>, <a href="http://oeis.org/A262342">A262342</a>.
   - First row of arrays <a href="http://oeis.org/A103323">A103323</a>, <a href="http://oeis.org/A234357">A234357</a>. Second row of arrays <a href="http://oeis.org/A099390">A099390</a>, <a href="http://oeis.org/A048887">A048887</a>, and <a href="http://oeis.org/A092921">A092921</a> (k-generalized Fibonacci numbers).
   - a(n) = <a href="http://oeis.org/A094718">A094718</a>(4, n). a(n) = <a href="http://oeis.org/A101220">A101220</a>(0, j, n).
   - a(n) = <a href="http://oeis.org/A090888">A090888</a>(0, n+1) = <a href="http://oeis.org/A118654">A118654</a>(0, n+1) = <a href="http://oeis.org/A118654">A118654</a>(1, n-1) = <a href="http://oeis.org/A109754">A109754</a>(0, n) = <a href="http://oeis.org/A109754">A109754</a>(1, n-1), for n > 0.
   - Fibonacci-Pascal triangles: <a href="http://oeis.org/A027926">A027926</a>, <a href="http://oeis.org/A036355">A036355</a>, <a href="http://oeis.org/A037027">A037027</a>, <a href="http://oeis.org/A074829">A074829</a>, <a href="http://oeis.org/A105809">A105809</a>, <a href="http://oeis.org/A109906">A109906</a>, <a href="http://oeis.org/A111006">A111006</a>, <a href="http://oeis.org/A114197">A114197</a>, <a href="http://oeis.org/A162741">A162741</a>, <a href="http://oeis.org/A228074">A228074</a>.
   - Boustrophedon transforms: <a href="http://oeis.org/A000738">A000738</a>, <a href="http://oeis.org/A000744">A000744</a>.
   - Powers: <a href="http://oeis.org/A103323">A103323</a>, <a href="http://oeis.org/A105317">A105317</a>, <a href="http://oeis.org/A254719">A254719</a>.
   - Numbers of prime factors: <a href="http://oeis.org/A022307">A022307</a> and <a href="http://oeis.org/A038575">A038575</a>.

_Links_:
   - Paul Barry, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL8/Barry/barry84.html">A Catalan Transform and Related Transformations on Integer Sequences</a>, Journal of Integer Sequences, Vol. 8 (2005), Article 05.4.5.
   - Paul Barry, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL9/Barry/barry91.html">On Integer-Sequence-Based Constructions of Generalized Pascal Triangles</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.2.4.

_References_:
   - J. Riordan, An Introduction to Combinatorial Analysis, Princeton University Press, Princeton, NJ, 1978.

## Under the hood

In [2]:
from inpututils import python_code

In [3]:
python_code(oeis_search)

```python
def oeis_search(id=None, seq=None, query="", start=0,):

    # the following imports are too specific to appear at the top of the module.
    from requests import get
    from IPython.display import Markdown

    payload = {"fmt": "json", "start": start}
    
    if id: payload.update({"q": "id:A{:06d}".format(id)})
    elif seq: payload.update({"q": ", ".join(map(str,seq))})
    else: payload.update({"fmt": "json", "q": query})
    
    try:
        doc_result = get("https://oeis.org/search", params=payload,)
        doc = doc_result.json()
    except:
        return lambda **pp_kwds: Markdown("<hr>__Connection Error__<hr>")
    
    def searchable(**pp_kwds):
        results_description = "_Results for query: <a href='{url}'>{url}</a>_<br><hr>".format(url=doc_result.url)
        inner_results = [pretty_print(result, **pp_kwds) for result in doc['results']]
        return Markdown(results_description + "\n<hr>".join(inner_results))
    
    return searchable

```

In [4]:
python_code(pretty_print)

```python
def pretty_print(doc, 
                 head=None, 
                 keyword=None, 
                 data_upper_limit=20,
                 comment=lambda i, c: True,
                 formula=lambda i, c: True,
                 xref=lambda i, c: True,
                 link=lambda i, c: "broken link" not in c,
                 reference=lambda i, c: True,):
    
    builders = [head_builder(), 
                keyword_builder(), 
                data_builder(upper_limit=data_upper_limit), 
                comment_builder(filter_pred=comment), 
                formula_builder(filter_pred=formula), 
                xref_builder(filter_pred=xref),
                link_builder(filter_pred=link), 
                reference_builder(filter_pred=reference)]
    
    descr = "\n".join([builder(doc) for builder in builders])
    return descr

```

---
<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title"><i>oeis-interaction notebook</i></span> by <span xmlns:cc="http://creativecommons.org/ns#" property="cc:attributionName">Massimo Nocentini</span> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/massimo-nocentini/competitive-programming/blob/master/python-libs/oeis.py" rel="dct:source">https://github.com/massimo-nocentini/competitive-programming/blob/master/python-libs/oeis.py</a>.